In [1]:
cp /kaggle/input/logger/logger.py /kaggle/working/

In [2]:
!wandb login 8d3272b3799965d019ae97e7cc2411cc23e4d87d

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
cd /kaggle/working

/kaggle/working


In [4]:
ACCOUNT='yuyututa'
NOTE='drseg'

In [5]:
!pip install torcheval
!pip install torch-dct
!pip install einops
!pip install segmentation-models-pytorch
# !pip install ml_collections
# !pip install wget
# !pip install medpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.7 MB/s eta 0:00:00a 0:00:01
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=8bcafb062fbb16330f8ec2867cf5a7b6bc26c37704d8b72524bbb36a397e53e4
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=51c62bc5c339e8504084644bb4227cf34a3

In [6]:
import sys, os
import argparse
import csv
import random
import numpy as np
import torch
import itertools
import json
from torch.utils.data.sampler import Sampler

# from const import *


def seed_torch(seed):
    seed = int(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False

def stable(dataloader, seed):
    seed_torch(seed)
    return dataloader


def add_dict_to_argparser(parser, default_dict):
    for k, v in default_dict.items():
        v_type = type(v)
        if v is None:
            v_type = str
        elif isinstance(v, bool):
            v_type = str2bool
        parser.add_argument(f"--{k}", default=v, type=v_type)
        
class Aobj():
    def __init__(self,default_dict):
        for k, v in default_dict.items():
            setattr(self,k,v)
    def get_dict(self):
        return self.__dict__
    
    def update(self, default_dict):
        for k, v in default_dict.items():
            setattr(self,k,v)


def mkdir(path):
    isExists = os.path.exists(path)
    if not isExists:
        os.makedirs(path)
        print(path + ' ----- folder created')
        return True
    else:
        return False

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()
        self.warning=0
        self.min_avg=sys.float_info.max

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
    
    def is_overfitting(self):
        if self.min_avg < self.avg:
            print(f'warning val_loss:{self.avg} > minimum val_loss:{self.min_avg}')
            self.warning+=1
            print(f'warning count: {self.warning}')
        elif self.min_avg > self.avg:
            self.warning=0
            self.min_avg=self.avg
        return self.warning>=3
    
    def shall_save(self):
        return self.warning==1

class Timer:
    def __init__(self):
        self.start_time=0

    def start_timer(self,start_time):
        self.start_time=start_time

    def compute(self,end_time, name=''):
        self.print_duration(end_time-self.start_time,name)
        self.start_time=0

    def print_duration(self, total_time, name):
        minutes, seconds = divmod(total_time, 60)
        print(f'{name} Time: {int(minutes)}m {int(seconds)}s, total:{total_time}')
    
def count_class(label, num_classes):
    if not isinstance(num_classes, list):
        num_classes=[num_classes]
        label=np.expand_dims(label, axis=0)
    if label.shape[0] != len(num_classes):
        label=label.transpose()
    results={}
    for i,n in enumerate(num_classes):
        l=label[i].tolist()
        result={}
        for j in range(n):
            result[f'第{j}类']=l.count(j)
        results.update({f'标签{i}':result})
    # return json.dumps(results, indent=4, ensure_ascii=False)
    return results

print('utils.py')

utils.py


In [7]:
#!coding:utf-8
import math
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.optim import lr_scheduler
from torch.nn.modules.loss import BCEWithLogitsLoss

# from const import *

def get_criterion(args, **kwargs):
    if args.loss_method == 'ce':
        criterion=torch.nn.CrossEntropyLoss()
    elif args.loss_method == 'wce':
        criterion=torch.nn.CrossEntropyLoss(weight=torch.tensor(args.ce_weight).to(kwargs['device']))
    elif args.loss_method == 'dice':
        criterion=DiceLoss()
    elif args.loss_method == 'ce_dice':
        criterion=CE_DiceLoss(weight=torch.tensor(args.ce_weight).to(kwargs['device']))
    elif args.loss_method == 'focal':
        criterion=FocalLoss(gamma=args.focal_gamma, alpha=torch.tensor(args.ce_weight).to(kwargs['device']))
    elif args.loss_method == 'bce':
        weight=torch.tensor(args.bce_weight).to(kwargs['device'])
        criterion=nn.BCEWithLogitsLoss(pos_weight=weight)
    elif args.loss_method == 'bce_dice':
        weight=torch.tensor(args.bce_weight).to(kwargs['device'])
        criterion=BCE_DiceLoss(weight=weight)
    elif args.loss_method == 'wbce':
        weight=torch.tensor(args.bce_weight).to(kwargs['device'])
        criterion=WeightedBCE(weight=weight)
    elif args.loss_method == 'cwbce':
        criterion=CustomWeightedLoss()
    return criterion

#学习率调整
def get_scheduler(args, optimizer, last_epoch=-1):
    if args.scheduler == 'cos':
        scheduler= lr_scheduler.CosineAnnealingLR(optimizer,T_max=args.num_epochs,eta_min=args.min_lr,last_epoch=last_epoch)
    elif args.scheduler == 'lam':
        def ad_lr(epoch):
            t=8
            if epoch <=t:
                return 1
            else:
                return math.pow(0.1,((epoch-t)//4)+1)
        scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=ad_lr,last_epoch=last_epoch)
    elif args.scheduler == 'multi':
        scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=args.milestones, gamma=args.lr_gamma,last_epoch=last_epoch)
    elif args.scheduler == 'step':
        scheduler = lr_scheduler.StepLR(optimizer, step_size=args.step_size, gamma=args.lr_gamma,last_epoch=last_epoch)
    else:
        scheduler=None
    return scheduler

def make_one_hot(labels, classes):
    one_hot = torch.FloatTensor(labels.size()[0], classes, labels.size()[2], labels.size()[3]).zero_().to(labels.device)
    target = one_hot.scatter_(1, labels.data, 1)
    return target

class CustomWeightedLoss(nn.Module):
    def __init__(self):
        super(CustomWeightedLoss, self).__init__()
        self.beta = 0.8

    def forward(self, preds, targets):
        """
        Args:
            preds (Tensor): 模型预测的概率，形状为 (batch_size, c, H, W)。
            targets (Tensor): 目标标签，形状为 (batch_size, c, H, W)。
        """
        preds = torch.clamp(preds,min=1e-8,max=1-1e-8)
        preds = F.sigmoid(preds)
        
        self.beta=(targets==0).sum(dim=(0,2,3))/(targets.shape[0]*targets.shape[2]*targets.shape[3])
        self.beta=self.beta.view(1,targets.shape[1],1,1)
        # 计算每个类别的损失
        bce = - self.beta * targets * torch.log(preds) - (1 - targets) * (1-self.beta) * torch.log(1 - preds)
        
        loss = torch.mean(bce)
        # print(self.beta)
        # print(f'(targets==0).sum(dim=(0,2,3)):{(targets==0).sum(dim=(0,2,3))}')
        # print(bce.shape)
        # print(loss)
        # print(loss.item())
        return loss

class WeightedBCE(nn.Module):
    def __init__(self, weight):
        super(WeightedBCE, self).__init__()
        self.weight=weight

    def forward(self, output, target):
        batch_weight = self.weight[target.data.view(-1).long()].view_as(target)
        bce = nn.BCELoss(weight=batch_weight)
        return bce(output, target)
        
        
class DiceLoss(nn.Module):
    def __init__(self, smooth=1., ignore_index=255, binary=False):
        super(DiceLoss, self).__init__()
        self.ignore_index = ignore_index
        self.smooth = smooth
        self.binary=binary

    def forward(self, output, target):
        if self.binary:
            output = F.sigmoid(output)
            output = output.contiguous().view(-1)
            target = target.contiguous().view(-1)
            
            pos_inter=output*target
            neg_inter=(1-output)*(1-target)
            intersection = pos_inter.sum()+neg_inter.sum()
            loss = 1-((2 * intersection + self.smooth) / (2*target.numel()+ self.smooth))
        else:
            if self.ignore_index not in range(target.min(), target.max()):
                if (target == self.ignore_index).sum() > 0:
                    target[target == self.ignore_index] = target.min()
            target = make_one_hot(target.unsqueeze(dim=1), classes=output.size()[1])
            output = F.softmax(output, dim=1)
            output_flat = output.contiguous().view(-1)
            target_flat = target.contiguous().view(-1)
            intersection = (output_flat * target_flat).sum()
            loss = 1 - ((2. * intersection + self.smooth) /
                        (output_flat.sum() + target_flat.sum() + self.smooth))
        return loss

class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None, ignore_index=255, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.size_average = size_average
        self.CE_loss = nn.CrossEntropyLoss(reduce=False, ignore_index=ignore_index, weight=alpha)

    def forward(self, output, target):
        logpt = self.CE_loss(output, target)
        pt = torch.exp(-logpt)
        loss = ((1-pt)**self.gamma) * logpt
        if self.size_average:
            return loss.mean()
        return loss.sum()

class CE_DiceLoss(nn.Module):
    def __init__(self, reduction='mean', ignore_index=255, weight=None):
        super(CE_DiceLoss, self).__init__()
        self.dice = DiceLoss()
        self.cross_entropy = nn.CrossEntropyLoss(weight=weight, reduction=reduction, ignore_index=ignore_index)
    
    def forward(self, output, target):
        CE_loss = self.cross_entropy(output, target)
        dice_loss = self.dice(output, target)
        return CE_loss + dice_loss

class BCE_DiceLoss(nn.Module):
    def __init__(self, weight, smooth=1):
        super(BCE_DiceLoss, self).__init__()
        self.dice = DiceLoss(smooth = smooth, binary=True)
        self.bce = BCEWithLogitsLoss(pos_weight=weight)
        
    def forward(self, output, target):
        bce_loss = self.bce(output, target)
        dice_loss = self.dice(output, target)
        return bce_loss+dice_loss
        

print('loss.py')

loss.py


In [8]:
import torch
import torcheval.metrics as tm
import numpy as np
from sklearn.metrics import *
import matplotlib.pyplot as plt
from torch.nn import functional as F

# from utils import utils
# from const import *


class Estimator():
    def __init__(self, metrics, num_classes, binary=False, input_logits=True):
        
        self.binary=binary
        self.input_logits=input_logits
        self.threshold=0.5
        if self.binary:
            self.metrics_names = {
                'acc': tm.BinaryAccuracy,
                'f1': tm.BinaryF1Score,
                'auc': tm.BinaryAUROC,
                'auprc': tm.BinaryAUPRC,
                'precision': tm.BinaryPrecision,
                'recall': tm.BinaryRecall,
                'cm':tm.BinaryConfusionMatrix
            }
        else:
            self.metrics_names = {
                'acc': tm.MulticlassAccuracy,
                'f1': tm.MulticlassF1Score,
                'auc': tm.MulticlassAUROC,
                'auprc': tm.MulticlassAUPRC,
                'precision': tm.MulticlassPrecision,
                'recall': tm.MulticlassRecall,
                'kappa': QuadraticWeightedKappa,
                'SegMetrics': SegMetrics
            }
        
        self.return_dict_metrics = ['SegMetrics']
        self.logits_required_metrics = ['auc', 'auprc']
        self.need_kwargs_change=['f1','precision','recall']
        self.need_named_metrics=['auc', 'auprc', 'PA']
        
        
        self.num_classes = num_classes
        self.metrics = metrics
        self.metrics_fn={}
        
        if self.binary:
            for m in metrics:
                self.metrics_fn[m]=self.metrics_names[m]()
        else:
            for m in metrics:
                if m in self.need_kwargs_change:
                    self.metrics_fn[m]=self.metrics_names[m](num_classes=num_classes, average='macro')
                if m in self.logits_required_metrics:
                    self.metrics_fn[m]=self.metrics_names[m](num_classes=num_classes, average=None)
                else:
                    self.metrics_fn[m]=self.metrics_names[m](num_classes=num_classes)
                
        # self.y = np.empty(0)
        # self.y_pred = np.empty(0)
        # if self.binary:
        #     self.logits = np.empty(0)
        # else:
        #     self.logits = np.empty((0,self.num_classes))
            

    def update(self, predictions, targets):
        targets = targets.detach().cpu().long()
        logits = predictions.detach().cpu()
        predictions = self.to_prediction(logits)
        
        if len(logits.shape) == 4:
            logits = logits.permute(0,2,3,1).reshape(-1, self.num_classes)
            logits = F.softmax(logits, dim=1)
        elif 'kappa' in self.metrics:
            logits = F.softmax(logits, dim=1)
        else:
            if self.input_logits:
                logits = F.sigmoid(logits)
            logits = logits.contiguous().view(-1)
        
            
        predictions = predictions.contiguous().view(-1)
        targets = targets.contiguous().view(-1)
        
        # self.y_pred = np.concatenate((self.y_pred, predictions.numpy()), axis=0)
        # self.y = np.concatenate((self.y, targets.numpy()), axis=0)
        # self.logits = np.concatenate((self.logits, logits.numpy()), axis=0)

        # update metrics
        for m in self.metrics_fn.keys():
            if m in self.logits_required_metrics:
                self.metrics_fn[m].update(logits, targets)
            else:
                self.metrics_fn[m].update(predictions, targets)

    def get_scores(self, digits=-1):
        
#         for i in range(self.num_classes):
#             self.plot_auprc(self.y, self.logits, i)
        
        scores={}
        for m in self.metrics:
            scores.update(self._compute(m, digits))
        return scores

    # def count(self, name):
    #     if name == 'gt' :
    #         return count_class(self.y, self.num_classes)
    #     elif name == 'pred': 
    #         return count_class(self.y_pred, self.num_classes)

    def _compute(self, metric, digits=-1):
        if not self.binary:
            if metric in self.return_dict_metrics:
                return self.metrics_fn[metric].compute(digits)
            if metric in self.logits_required_metrics:
                l=self.metrics_fn[metric].compute().tolist()
                l.append(np.mean(l[1:]))
                l=np.round(l,digits)
                return {metric:l}
            
        score = self.metrics_fn[metric].compute().item()
        score = score if digits == -1 else round(score, digits)
        return {metric:score}

    def reset(self):
        for m in self.metrics_fn.keys():
            self.metrics_fn[m].reset()
        # self.y = np.empty(0)
        # self.y_pred = np.empty(0)
        # if self.binary:
        #     self.logits = np.empty(0)
        # else:
        #     self.logits = np.empty((0,self.num_classes))
    
    def to_prediction(self, predictions):
        if self.binary:
            predictions = (predictions>self.threshold).to(torch.int16)
        else:
            predictions = torch.argmax(predictions, dim=1).long()
        return predictions
    
    def name_val(self, label, metric, val):
        l=label
        label={'BK':None}
        label.update(l)
        named_val={}
        for i,(k,v) in enumerate(label.items()):
            named_val.update({metric+'_'+k:val[i]})
        named_val.update({metric:val[-1]})
        return named_val
    
    def plot_auprc(self, targets, logits, c):
        targets=(targets==c).astype(np.int32)
        logits=logits[:,c]
        PrecisionRecallDisplay.from_predictions(targets ,logits)
        precision, recall, th = precision_recall_curve(targets, logits)
        print('precision:{}\n'.format(precision))
        print('recall:{}\n'.format(recall))
        print('threshhold:{}\n'.format(th))
        auprc=auc(recall, precision)
        
        plt.legend([f'AUPRC = {auprc:.2f}'], loc="lower left")
        plt.title(f'{c}th PR Curve')
        plt.show()

class BinaryAUPRC():
    def __init__(self):
        self.logits = np.empty(0)
        self.targets = np.empty(0)
        
    def update(self, logits, targets):
        self.targets = np.concatenate((self.targets, targets.numpy()), axis=0)
        self.logits = np.concatenate((self.logits, logits.numpy()), axis=0)
        
    def compute(self, digits=4):
        precision, recall, thresholds = precision_recall_curve(self.targets, self.logits)
        auprc = auc(recall, precision)
        return np.round(auprc, digits)
        
    def reset(self):
        self.logits = np.empty(0)
        self.targets = np.empty(0)
            
class SegMetrics():
    def __init__(self, num_classes):
        self.num_classes=num_classes
        self.confusion_matrix = np.zeros((num_classes,num_classes))
        self.cm=tm.MulticlassConfusionMatrix(num_classes=num_classes)
        
    def PA(self):
        return np.diag(self.confusion_matrix) / self.confusion_matrix.sum(axis=1)
    
    def PAC(self):
        acc = np.diag(self.confusion_matrix) / self.confusion_matrix.sum(axis=1)
        acc_class = np.nanmean(acc)
        return acc_class
    
    def MIoU(self):
        IoU = np.diag(self.confusion_matrix) / (
            np.sum(self.confusion_matrix, axis=1) + np.sum(self.confusion_matrix, axis=0) -
            np.diag(self.confusion_matrix))
        MIoU = np.nanmean(IoU)
        MIoU_noback = np.nanmean(IoU[1:])
        return MIoU, MIoU_noback
    
    def update(self, predictions, targets):
        self.cm.update(predictions, targets)
        
    def reset(self):
        self.cm = tm.MulticlassConfusionMatrix(num_classes=self.num_classes)

    def compute(self, digits):
        self.confusion_matrix=self.cm.compute().numpy().astype(np.int32)
        cm=self.cm.normalized('true').numpy()*100
        np.set_printoptions(suppress=True)
        print(np.round(cm,2))
        MIoU, MIoU_noback = self.MIoU()
        return {
            "PA": np.round(self.PA(), digits),
            "PAC": np.round(self.PAC(), digits),
            "MIoU": np.round(MIoU, digits),
            "MIoU_NoBack": np.round(MIoU_noback, digits)
        }

class QuadraticWeightedKappa():
    def __init__(self, num_classes):
        self.num_classes = num_classes
        self.conf_mat = torch.zeros((self.num_classes, self.num_classes), dtype=int)

    def update(self, predictions, targets):
        for i, p in enumerate(predictions):
            self.conf_mat[int(targets[i])][int(p.item())] += 1

    def compute(self):
        return self.quadratic_weighted_kappa(self.conf_mat)

    def reset(self):
        self.conf_mat = torch.zeros((self.num_classes, self.num_classes), dtype=int)

    def quadratic_weighted_kappa(self, conf_mat):
        assert conf_mat.shape[0] == conf_mat.shape[1]
        cate_num = conf_mat.shape[0]

        # Quadratic weighted matrix
        weighted_matrix = torch.zeros((cate_num, cate_num))
        for i in range(cate_num):
            for j in range(cate_num):
                weighted_matrix[i][j] = 1 - float(((i - j)**2) / ((cate_num - 1)**2))

        # Expected matrix
        ground_truth_count = torch.sum(conf_mat, axis=1)
        pred_count = torch.sum(conf_mat, axis=0)
        expected_matrix = torch.outer(ground_truth_count, pred_count)

        # Normalization
        conf_mat = conf_mat / conf_mat.sum()
        expected_matrix = expected_matrix / expected_matrix.sum()

        observed = (conf_mat * weighted_matrix).sum()
        expected = (expected_matrix * weighted_matrix).sum()
        return (observed - expected) / (1 - expected)

print('metircs.py')


metircs.py


In [9]:
from torch import nn
import torch
import torch.nn.functional as F

def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


class ConvBNReLU(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, dilation=1, groups=1):
        #padding = (kernel_size - 1) // 2
        if kernel_size==1:
            padding=0
        else:
            padding=dilation
        super(ConvBNReLU, self).__init__(
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, dilation=dilation, groups=groups, bias=False),
            nn.BatchNorm2d(out_planes),
            nn.ReLU6(inplace=True)
        )

class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, dilation, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = int(round(inp * expand_ratio))
        self.use_res_connect = self.stride == 1 and inp == oup

        layers = []
        if expand_ratio != 1:
            # pw
            layers.append(ConvBNReLU(inp, hidden_dim, kernel_size=1))

        layers.extend([
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, stride=stride, dilation=dilation, groups=hidden_dim),
            # pw-linear
            nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
            nn.BatchNorm2d(oup),
        ])
        self.conv = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)

class MobileNetV2(nn.Module):
    def __init__(self, num_classes=1000, downsample_factor=8, width_mult=1.0, inverted_residual_setting=None, round_nearest=8):
        """
        MobileNet V2 main class

        Args:
            num_classes (int): Number of classes
            width_mult (float): Width multiplier - adjusts number of channels in each layer by this amount
            inverted_residual_setting: Network structure
            round_nearest (int): Round the number of channels in each layer to be a multiple of this number
            Set to 1 to turn off rounding
        """
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = 32
        last_channel = 1280
        if inverted_residual_setting is None:
            inverted_residual_setting = [
                # t, c, n, s
                [1, 16, 1, 1],
                [6, 24, 2, 2],
                [6, 32, 3, 2],
                [6, 64, 4, 2],
                [6, 96, 3, 1],
                [6, 160, 3, 2],
                [6, 320, 1, 1],
            ]

        # only check the first element, assuming user knows t,c,n,s are required
        if len(inverted_residual_setting) == 0 or len(inverted_residual_setting[0]) != 4:
            raise ValueError("inverted_residual_setting should be non-empty "
                             "or a 4-element list, got {}".format(inverted_residual_setting))

        # building first layer
        input_channel = _make_divisible(input_channel * width_mult, round_nearest)
        self.last_channel = _make_divisible(last_channel * max(1.0, width_mult), round_nearest)
        features = [ConvBNReLU(3, input_channel, stride=2)]
        cur_downsample_factor = 2
        dilation=1
        previous_dilation = 1

        # building inverted residual blocks
        for t, c, n, s in inverted_residual_setting:
            output_channel = _make_divisible(c * width_mult, round_nearest)
            previous_dilation = dilation
            if cur_downsample_factor == downsample_factor:
                stride = 1
                dilation *= s
            else:
                stride = s
                cur_downsample_factor *= s
            output_channel = int(c * width_mult)

            for i in range(n):
                if i==0:
                    features.append(block(input_channel, output_channel, stride, previous_dilation, expand_ratio=t))
                else:
                    features.append(block(input_channel, output_channel, 1, dilation, expand_ratio=t))
                input_channel = output_channel
        # building last several layers
        features.append(ConvBNReLU(input_channel, self.last_channel, kernel_size=1))
        # make it nn.Sequential
        self.features = nn.Sequential(*features)

        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.last_channel, num_classes),
        )

        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        low_level_feature = self.features[:4](x)
        x = self.features[4:-1](low_level_feature)
        return low_level_feature, x


def _mobilenetv2(**kwargs):
    print(kwargs)
    url = kwargs.pop('url', -1)
    
    model = MobileNetV2(**kwargs)
    
    if url != -1:
        weight_dict=torch.hub.load_state_dict_from_url(url, progress=True)
        del_key = []
        for key, _ in weight_dict.items():  # 遍历预训练权重的有序字典
            if "fc" in key:  # 如果key中包含'fc'这个字段
                del_key.append(key)

        for key in del_key:  # 遍历要删除字段的list
            del weight_dict[key]
        # 抽出现有模型中的K,V
        model_dict=model.state_dict()
        # 新建权重字典，并更新
        state_dict={k:v for k,v in weight_dict.items() if k in model_dict.keys()}
        # 更新现有模型的权重字典
        model_dict.update(state_dict)
        # 载入更新后的权重字典
        model.load_state_dict(model_dict)
    return model


def mobilenetv2(**kwargs):
    pretrained = kwargs.pop('pretrained', -1)
    if pretrained != -1 and pretrained:
        kwargs['url'] = 'https://download.pytorch.org/models/mobilenet_v2-b0353104.pth'
    
    model = _mobilenetv2(**kwargs)
    return model

print('mobilenetv2.py')

mobilenetv2.py


In [10]:

"""
Xception is adapted from https://github.com/Cadene/pretrained-models.pytorch/blob/master/pretrainedmodels/models/xception.py

Ported to pytorch thanks to [tstandley](https://github.com/tstandley/Xception-PyTorch)
@author: tstandley
Adapted by cadene
Creates an Xception Model as defined in:
Francois Chollet
Xception: Deep Learning with Depthwise Separable Convolutions
https://arxiv.org/pdf/1610.02357.pdf
This weights ported from the Keras implementation. Achieves the following performance on the validation set:
Loss:0.9173 Prec@1:78.892 Prec@5:94.292
REMEMBER to set your image size to 3x299x299 for both test and validation
normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                  std=[0.5, 0.5, 0.5])
The resize parameter of the validation transform should be 333, and make sure to center crop at 299x299
"""
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init


class SeparableConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=1,stride=1,padding=0,dilation=1,bias=False):
        super(SeparableConv2d,self).__init__()

        self.conv1 = nn.Conv2d(in_channels,in_channels,kernel_size,stride,padding,dilation,groups=in_channels,bias=bias)
        self.pointwise = nn.Conv2d(in_channels,out_channels,1,1,0,1,1,bias=bias)

    def forward(self,x):
        x = self.conv1(x)
        x = self.pointwise(x)
        return x


class Block(nn.Module):
    def __init__(self,in_filters,out_filters,reps,strides=1,start_with_relu=True,grow_first=True, dilation=1):
        super(Block, self).__init__()

        if out_filters != in_filters or strides!=1:
            self.skip = nn.Conv2d(in_filters,out_filters,1,stride=strides, bias=False)
            self.skipbn = nn.BatchNorm2d(out_filters)
        else:
            self.skip=None

        rep=[]

        filters=in_filters
        if grow_first:
            rep.append(nn.ReLU(inplace=True))
            rep.append(SeparableConv2d(in_filters,out_filters,3,stride=1,padding=dilation, dilation=dilation, bias=False))
            rep.append(nn.BatchNorm2d(out_filters))
            filters = out_filters

        for i in range(reps-1):
            rep.append(nn.ReLU(inplace=True))
            rep.append(SeparableConv2d(filters,filters,3,stride=1,padding=dilation,dilation=dilation,bias=False))
            rep.append(nn.BatchNorm2d(filters))

        if not grow_first:
            rep.append(nn.ReLU(inplace=True))
            rep.append(SeparableConv2d(in_filters,out_filters,3,stride=1,padding=dilation,dilation=dilation,bias=False))
            rep.append(nn.BatchNorm2d(out_filters))

        if not start_with_relu:
            rep = rep[1:]
        else:
            rep[0] = nn.ReLU(inplace=False)

        if strides != 1:
            rep.append(nn.MaxPool2d(3,strides,1))
        self.rep = nn.Sequential(*rep)

    def forward(self,inp):
        x = self.rep(inp)

        if self.skip is not None:
            skip = self.skip(inp)
            skip = self.skipbn(skip)
        else:
            skip = inp
        x+=skip
        return x


class Xception(nn.Module):
    """
    Xception optimized for the ImageNet dataset, as specified in
    https://arxiv.org/pdf/1610.02357.pdf
    """
    def __init__(self, num_classes=1000, downsample_factor=8):
        """ Constructor
        Args:
            num_classes: number of classes
        """
        super(Xception, self).__init__()

        self.num_classes = num_classes
        self.dilation = 1
        if downsample_factor==8:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, True, True]
        else:
            replace_stride_with_dilation = [False, False, False, True]

        self.conv1 = nn.Conv2d(3, 32, 3,2, 0, bias=False) # 1 / 2
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(32,64,3,bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=True)
        #do relu here

        self.block1=self._make_block(64,128,2,2,start_with_relu=False,grow_first=True, dilate=replace_stride_with_dilation[0]) # 1 / 4
        self.block2=self._make_block(128,256,2,2,start_with_relu=True,grow_first=True, dilate=replace_stride_with_dilation[1]) # 1 / 8
        self.block3=self._make_block(256,728,2,2,start_with_relu=True,grow_first=True, dilate=replace_stride_with_dilation[2]) # 1 / 16

        self.block4=self._make_block(728,728,3,1,start_with_relu=True,grow_first=True, dilate=replace_stride_with_dilation[2])
        self.block5=self._make_block(728,728,3,1,start_with_relu=True,grow_first=True, dilate=replace_stride_with_dilation[2])
        self.block6=self._make_block(728,728,3,1,start_with_relu=True,grow_first=True, dilate=replace_stride_with_dilation[2])
        self.block7=self._make_block(728,728,3,1,start_with_relu=True,grow_first=True, dilate=replace_stride_with_dilation[2])

        self.block8=self._make_block(728,728,3,1,start_with_relu=True,grow_first=True, dilate=replace_stride_with_dilation[2])
        self.block9=self._make_block(728,728,3,1,start_with_relu=True,grow_first=True, dilate=replace_stride_with_dilation[2])
        self.block10=self._make_block(728,728,3,1,start_with_relu=True,grow_first=True, dilate=replace_stride_with_dilation[2])
        self.block11=self._make_block(728,728,3,1,start_with_relu=True,grow_first=True, dilate=replace_stride_with_dilation[2])

        self.block12=self._make_block(728,1024,2,2,start_with_relu=True,grow_first=False, dilate=replace_stride_with_dilation[3]) # 1 / 32

        self.conv3 = SeparableConv2d(1024,1536,3,1,1, dilation=self.dilation)
        self.bn3 = nn.BatchNorm2d(1536)
        self.relu3 = nn.ReLU(inplace=True)

        #do relu here
        self.conv4 = SeparableConv2d(1536,2048,3,1,1, dilation=self.dilation)
        self.bn4 = nn.BatchNorm2d(2048)

        self.fc = nn.Linear(2048, num_classes)

        # #------- init weights --------
        # for m in self.modules():
        #     if isinstance(m, nn.Conv2d):
        #         n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        #         m.weight.data.normal_(0, math.sqrt(2. / n))
        #     elif isinstance(m, nn.BatchNorm2d):
        #         m.weight.data.fill_(1)
        #         m.bias.data.zero_()
        # #-----------------------------

    def _make_block(self, in_filters,out_filters,reps,strides=1,start_with_relu=True,grow_first=True, dilate=False):
        if dilate:
            self.dilation *= strides
            strides = 1
        return Block(in_filters,out_filters,reps,strides,start_with_relu=start_with_relu,grow_first=grow_first, dilation=self.dilation)

    def features(self, input):
        x = self.conv1(input)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        low_level_feature = self.block1(x)
        x = low_level_feature
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        x = self.block7(x)
        x = self.block8(x)
        x = self.block9(x)
        x = self.block10(x)
        x = self.block11(x)
        x = self.block12(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)

        x = self.conv4(x)
        x = self.bn4(x)
        return low_level_feature, x

    def logits(self, features):
        x = nn.ReLU(inplace=True)(features)

        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        return x

    def forward(self, input):
        low_level_feature, x = self.features(input)
        # x = self.logits(x)
        return low_level_feature, x


def _xception(**kwargs):
    print(kwargs)
    url = kwargs.pop('url', -1)
    
    model = Xception(**kwargs)
    
    if url != -1:
        try:
            weight_dict=torch.hub.load_state_dict_from_url(url, progress=True)
        except:
            weight_dict=torch.load('/kaggle/input/pretrained_model/pytorch/xception/1/xception-43020ad28.pth')
            # for k,v in weight_dict.items():
            #     print(f'{k} -- {v.shape}')
        del_key = []
        for key, _ in weight_dict.items():  # 遍历预训练权重的有序字典
            if "fc" in key:  # 如果key中包含'fc'这个字段
                del_key.append(key)

        for key in del_key:  # 遍历要删除字段的list
            del weight_dict[key]
        # 抽出现有模型中的K,V
        model_dict=model.state_dict()
        # 新建权重字典，并更新
        state_dict={k:v for k,v in weight_dict.items() if k in model_dict.keys()}
        # 更新现有模型的权重字典
        model_dict.update(state_dict)
        # 载入更新后的权重字典
        model.load_state_dict(model_dict)
    return model


def xception(**kwargs):
    pretrained = kwargs.pop('pretrained', -1)
    if pretrained != -1 and pretrained:
        kwargs['url'] = 'http://data.lip6.fr/cadene/pretrainedmodels/xception-43020ad28.pth'
    
    model = _xception(**kwargs)
    return model

print('xception.py')

xception.py


In [11]:
from collections import OrderedDict
from functools import partial
from typing import Any, List, Optional, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
NOKEY=-1

# from model.modules import *


class _DenseAtrousLayer(nn.Module):
    def __init__(
        self, num_input_features: int, growth_rate: int, bn_size: int, drop_rate: float, memory_efficient: bool = False, dilation: int = 1 
    ) -> None:
        super().__init__()
        self.norm1 = nn.BatchNorm2d(num_input_features)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(num_input_features, bn_size * growth_rate, kernel_size=1, stride=1, bias=False)

        self.norm2 = nn.BatchNorm2d(bn_size * growth_rate)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(bn_size * growth_rate, growth_rate, kernel_size=3, stride=1, padding=dilation, dilation=dilation, bias=False)

        self.drop_rate = float(drop_rate)
        self.memory_efficient = memory_efficient

    def bn_function(self, inputs: List[Tensor]) -> Tensor:
        concated_features = torch.cat(inputs, 1)
        bottleneck_output = self.conv1(self.relu1(self.norm1(concated_features)))  # noqa: T484
        return bottleneck_output

    # torchscript does not yet support *args, so we overload method
    # allowing it to take either a List[Tensor] or single Tensor
    def forward(self, input: Tensor) -> Tensor:  # noqa: F811
        if isinstance(input, Tensor):
            prev_features = [input]
        else:
            prev_features = input

        if self.memory_efficient and self.any_requires_grad(prev_features):
            if torch.jit.is_scripting():
                raise Exception("Memory Efficient not supported in JIT")

            bottleneck_output = self.call_checkpoint_bottleneck(prev_features)
        else:
            bottleneck_output = self.bn_function(prev_features)

        new_features = self.conv2(self.relu2(self.norm2(bottleneck_output)))
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return new_features


class _DenseAtrousBlock(nn.ModuleDict):
    _version = 2

    def __init__(
        self,
        num_layers: int,
        num_input_features: int,
        bn_size: int,
        growth_rate: int,
        drop_rate: float,
        memory_efficient: bool = False,
        dilation: int = 1,
    ) -> None:
        super().__init__()
        for i in range(num_layers):
            layer = _DenseAtrousLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient,
                dilation=dilation
            )
            self.add_module("denselayer%d" % (i + 1), layer)

    def forward(self, init_features: Tensor) -> Tensor:
        features = [init_features]
        for name, layer in self.items():
            new_features = layer(features)
            features.append(new_features)
        return torch.cat(features, 1)


class _AtrousTransition(nn.Sequential):
    def __init__(self, num_input_features: int, num_output_features: int, dilation: int = 1) -> None:
        super().__init__()
        self.norm = nn.BatchNorm2d(num_input_features)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(num_input_features, num_output_features, kernel_size=1, stride=1, bias=False)
        if dilation == 1:
            self.pool = nn.AvgPool2d(kernel_size=2, stride=2)#减半尺寸


class DenseNetAtrous(nn.Module):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_.

    Args:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
        memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
          but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_.
    """

    def __init__(
        self,
        growth_rate: int = 32,
        block_config: Tuple[int, int, int, int] = (6, 12, 24, 16),
        num_init_features: int = 64,
        bn_size: int = 4,
        drop_rate: float = 0,
        num_classes: int = 1000,
        memory_efficient: bool = False,
        module_list = None,
        downsample_factor=8,
    ) -> None:

        super().__init__()
        
        # self.module_list = module_list
        # self.attention = module_list['attention']
        
        #x = [3,512,512]
        # First convolution
        self.low_features = nn.Sequential(
            OrderedDict(
                [
                    ("conv0", nn.Conv2d(3, num_init_features, kernel_size=7, stride=2, padding=3, bias=False)),#x/2
                    ("norm0", nn.BatchNorm2d(num_init_features)),
                    ("relu0", nn.ReLU(inplace=True)),
                    ("pool0", nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
                ]
            )
        )
        self.features = nn.Sequential()
            
        self.cur_downsample_factor=2
        self.dilation=1
        # Each denseblock
        num_features = num_init_features #初始通道数
        for i, num_layers in enumerate(block_config):
            if self.cur_downsample_factor == downsample_factor:
                self.dilation*=2
            else:
                self.dilation=1
                self.cur_downsample_factor*=2
            block = _DenseAtrousBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient,
                dilation=self.dilation
            )
            if i==0:
                self.low_features.add_module("denseblock%d" % (i + 1), block)
            else:
                self.features.add_module("denseblock%d" % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _AtrousTransition(num_input_features=num_features, num_output_features=num_features // 2, dilation=self.dilation)#减半
                if i==0:
                    self.low_features.add_module("transition%d" % (i + 1), trans)
                    self.low_channels=num_features // 2
                else:
                    self.features.add_module("transition%d" % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.output_channels=num_features
        self.features.add_module("norm5", nn.BatchNorm2d(num_features))
        
        print(f'num_features:{num_features}')
        # self.mymodules=get_modules(self.module_list, num_features, num_classes=num_classes)

        # Linear layer
        if isinstance(num_classes, list):
            self.classifier=nn.ModuleList([nn.Linear(num_features, n) for n in num_classes])
        else:
            self.classifier = nn.Linear(num_features, num_classes)

        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x: Tensor) -> Tensor:
        low_level_feature=self.low_features(x)
        features = self.features(low_level_feature)
        
        # for i in self.mymodules:
        #     features = i(features)
        
        # out = F.relu(features, inplace=True)
        # out = F.adaptive_avg_pool2d(out, (1, 1))
        # out = torch.flatten(out, 1)
        # if isinstance(self.classifier, nn.ModuleList):
        #     x = [classifier(out) for classifier in self.classifier]
        # else:
        #     x = self.classifier(out)
        return low_level_feature, features




def _densenet_atrous(
    growth_rate: int,
    block_config: Tuple[int, int, int, int],
    num_init_features: int,
    **kwargs: Any,
) -> DenseNetAtrous:
    print(kwargs)
    url = kwargs.pop('url', NOKEY)

    model = DenseNetAtrous(growth_rate, block_config, num_init_features, **kwargs)
    
    if url != NOKEY:
        weight_dict=torch.hub.load_state_dict_from_url(url, progress=True)
        del_key = []
        for key, _ in weight_dict.items():  # 遍历预训练权重的有序字典
            if "classifier" in key:  # 如果key中包含'classifier'这个字段
                del_key.append(key)

        for key in del_key:  # 遍历要删除字段的list
            del weight_dict[key]
        # 抽出现有模型中的K,V
        model_dict=model.state_dict()
        # 新建权重字典，并更新
        state_dict={k:v for k,v in weight_dict.items() if k in model_dict.keys()}
        # 更新现有模型的权重字典
        model_dict.update(state_dict)
        # 载入更新后的权重字典
        model.load_state_dict(model_dict)

    return model

DenseNetAtrousWeight={
        'densenet121':'https://download.pytorch.org/models/densenet121-a639ec97.pth',
        'densenet161':'https://download.pytorch.org/models/densenet161-8d451a50.pth',
        'densenet169':'https://download.pytorch.org/models/densenet169-b2777c0a.pth',
        'densenet201':'https://download.pytorch.org/models/densenet201-c1103571.pth'
    }


def densenet_atrous121(**kwargs: Any) -> DenseNetAtrous:
    pretrained = kwargs.pop('pretrained',NOKEY)
    if pretrained != NOKEY and pretrained:
        kwargs['url'] = DenseNetAtrousWeight['densenet121']

    return _densenet_atrous(32, (6, 12, 24, 16), 64, **kwargs)


def densenet_atrous161(**kwargs: Any) -> DenseNetAtrous:
    pretrained = kwargs.pop('pretrained',NOKEY)
    if pretrained != NOKEY and pretrained:
        kwargs['url'] = DenseNetAtrousWeight['densenet161']

    return _densenet_atrous(48, (6, 12, 36, 24), 96, **kwargs)

def densenet_atrous169(**kwargs: Any) -> DenseNetAtrous:
    pretrained = kwargs.pop('pretrained',NOKEY)
    if pretrained != NOKEY and pretrained:
        kwargs['url'] = DenseNetAtrousWeight['densenet169']

    return _densenet_atrous(32, (6, 12, 32, 32), 64, **kwargs)

def densenet_atrous201(**kwargs: Any) -> DenseNetAtrous:
    pretrained = kwargs.pop('pretrained',NOKEY)
    if pretrained != NOKEY and pretrained:
        kwargs['url'] = DenseNetAtrousWeight['densenet201']

    return _densenet_atrous(32, (6, 12, 48, 32), 64, **kwargs)

print('densenet_atrous.py')

densenet_atrous.py


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from model.densenet import densenet161
# import xception
# import mobilenetv2

#-----------------------------------------#
#   ASPP特征提取模块
#   利用不同膨胀率的膨胀卷积进行特征提取
#-----------------------------------------#
class ASPP(nn.Module):
    def __init__(self, dim_in, dim_out, rate=1, bn_mom=0.1):
        super(ASPP, self).__init__()
        self.branch1 = nn.Sequential(
                nn.Conv2d(dim_in, dim_out, 1, 1, padding=0, bias=True),
                nn.BatchNorm2d(dim_out, momentum=bn_mom),
                nn.ReLU(inplace=True),
        )
        self.branch2 = nn.Sequential(
                nn.Conv2d(dim_in, dim_out, 3, 1, padding=6*rate, dilation=6*rate, bias=True),
                nn.BatchNorm2d(dim_out, momentum=bn_mom),
                nn.ReLU(inplace=True),    
        )
        self.branch3 = nn.Sequential(
                nn.Conv2d(dim_in, dim_out, 3, 1, padding=12*rate, dilation=12*rate, bias=True),
                nn.BatchNorm2d(dim_out, momentum=bn_mom),
                nn.ReLU(inplace=True),    
        )
        self.branch4 = nn.Sequential(
                nn.Conv2d(dim_in, dim_out, 3, 1, padding=18*rate, dilation=18*rate, bias=True),
                nn.BatchNorm2d(dim_out, momentum=bn_mom),
                nn.ReLU(inplace=True),    
        )
        self.branch5_conv = nn.Conv2d(dim_in, dim_out, 1, 1, 0,bias=True)
        self.branch5_bn = nn.BatchNorm2d(dim_out, momentum=bn_mom)
        self.branch5_relu = nn.ReLU(inplace=True)
        
        self.conv_cat = nn.Sequential(
                nn.Conv2d(dim_out*5, dim_out, 1, 1, padding=0,bias=True),
                nn.BatchNorm2d(dim_out, momentum=bn_mom),
                nn.ReLU(inplace=True),        
        )

    def forward(self, x):
        [b, c, row, col] = x.size()
        # print('x.shape:{}'.format(x.size()))
        #-----------------------------------------#
        #   一共五个分支
        #-----------------------------------------#
        conv1x1 = self.branch1(x)
        conv3x3_1 = self.branch2(x)
        conv3x3_2 = self.branch3(x)
        conv3x3_3 = self.branch4(x)
        #-----------------------------------------#
        #   第五个分支，全局平均池化(image pooling)+卷积
        #-----------------------------------------#
        global_feature = torch.mean(x,2,True)
        global_feature = torch.mean(global_feature,3,True)#shape=(1,dim_out,1,1)
        global_feature = self.branch5_conv(global_feature)
        global_feature = self.branch5_bn(global_feature)
        global_feature = self.branch5_relu(global_feature)
        global_feature = F.interpolate(global_feature, (row, col), None, 'bilinear', True)#恢复原图尺寸
        
        #-----------------------------------------#
        #   将五个分支的内容堆叠起来
        #   然后1x1卷积整合特征。
        #-----------------------------------------#
        feature_cat = torch.cat([conv1x1, conv3x3_1, conv3x3_2, conv3x3_3, global_feature], dim=1)#在channel维度上拼接所有特征图
        result = self.conv_cat(feature_cat)
        return result

class DeepLab(nn.Module):
    def __init__(self, num_classes, backbone="mobilenet", pretrained=True, downsample_factor=16, **kwargs):
        super(DeepLab, self).__init__()
        if backbone=="xception":
            #----------------------------------#
            #   获得两个特征层
            #   浅层特征    [128,128,256]
            #   主干部分    [30,30,2048]
            #----------------------------------#
            self.backbone = xception(downsample_factor=downsample_factor, pretrained=pretrained)
            in_channels = 2048
            low_level_channels = 128
        elif backbone=="mobilenet":
            #----------------------------------#
            #   获得两个特征层
            #   浅层特征    [128,128,24]
            #   主干部分    [30,30,320]
            #----------------------------------#
            self.backbone = mobilenetv2(downsample_factor=downsample_factor, pretrained=pretrained)
            in_channels = 320
            low_level_channels = 24
        elif backbone=='densenet':
            self.backbone = densenet_atrous161(downsample_factor=downsample_factor, pretrained=pretrained)
            in_channels = self.backbone.output_channels
            low_level_channels = self.backbone.low_channels
        else:
            raise ValueError('Unsupported backbone - `{}`, Use mobilenet, xception.'.format(backbone))

        #-----------------------------------------#
        #   ASPP特征提取模块
        #   利用不同膨胀率的膨胀卷积进行特征提取
        #   这里的rate可理解为如果downsample_factor=8的话，图片尺寸相比downsample_ractor=16要大一倍，所以可以用更大的空洞卷积核去卷积
        #-----------------------------------------#
        self.aspp = ASPP(dim_in=in_channels, dim_out=256, rate=16//downsample_factor)
        
        #----------------------------------#
        #   浅层特征边
        #----------------------------------#
        self.shortcut_conv = nn.Sequential(
            nn.Conv2d(low_level_channels, 48, 1),
            nn.BatchNorm2d(48),
            nn.ReLU(inplace=True)
        )        

        self.cat_conv = nn.Sequential(
            nn.Conv2d(48+256, 256, 3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),

            nn.Conv2d(256, 256, 3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.Dropout(0.1),
        )
        self.cls_conv = nn.Conv2d(256, num_classes, 1, stride=1)

    def forward(self, x):
        H, W = x.size(2), x.size(3)
        #-----------------------------------------#
        #   获得两个特征层
        #   low_level_features: 浅层特征-进行卷积处理
        #   x : 主干部分-利用ASPP结构进行加强特征提取
        #-----------------------------------------#
        low_level_features, x = self.backbone(x)
        x = self.aspp(x)
        low_level_features = self.shortcut_conv(low_level_features)
        
        #-----------------------------------------#
        #   将加强特征边上采样
        #   与浅层特征堆叠后利用卷积进行特征提取
        #-----------------------------------------#
        x = F.interpolate(x, size=(low_level_features.size(2), low_level_features.size(3)), mode='bilinear', align_corners=True)
        x = self.cat_conv(torch.cat((x, low_level_features), dim=1))
        x = self.cls_conv(x)
        x = F.interpolate(x, size=(H, W), mode='bilinear', align_corners=True)
        return x

print('deeplabv3+.py')

deeplabv3+.py


In [13]:
import re
from collections import OrderedDict
from functools import partial
from typing import Any, List, Optional, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from torch import Tensor
NOKEY=-1

# from model.modules import *


class _DenseLayer(nn.Module):
    def __init__(
        self, num_input_features: int, growth_rate: int, bn_size: int, drop_rate: float, memory_efficient: bool = False
    ) -> None:
        super().__init__()
        self.norm1 = nn.BatchNorm2d(num_input_features)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(num_input_features, bn_size * growth_rate, kernel_size=1, stride=1, bias=False)

        self.norm2 = nn.BatchNorm2d(bn_size * growth_rate)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(bn_size * growth_rate, growth_rate, kernel_size=3, stride=1, padding=1, bias=False)

        self.drop_rate = float(drop_rate)
        self.memory_efficient = memory_efficient

    def bn_function(self, inputs: List[Tensor]) -> Tensor:
        concated_features = torch.cat(inputs, 1)
        bottleneck_output = self.conv1(self.relu1(self.norm1(concated_features)))  # noqa: T484
        return bottleneck_output

    # torchscript does not yet support *args, so we overload method
    # allowing it to take either a List[Tensor] or single Tensor
    def forward(self, input: Tensor) -> Tensor:  # noqa: F811
        if isinstance(input, Tensor):
            prev_features = [input]
        else:
            prev_features = input

        if self.memory_efficient and self.any_requires_grad(prev_features):
            if torch.jit.is_scripting():
                raise Exception("Memory Efficient not supported in JIT")

            bottleneck_output = self.call_checkpoint_bottleneck(prev_features)
        else:
            bottleneck_output = self.bn_function(prev_features)

        new_features = self.conv2(self.relu2(self.norm2(bottleneck_output)))
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return new_features


class _DenseBlock(nn.ModuleDict):
    _version = 2

    def __init__(
        self,
        num_layers: int,
        num_input_features: int,
        bn_size: int,
        growth_rate: int,
        drop_rate: float,
        memory_efficient: bool = False,
    ) -> None:
        super().__init__()
        for i in range(num_layers):
            layer = _DenseLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient,
            )
            self.add_module("denselayer%d" % (i + 1), layer)

    def forward(self, init_features: Tensor) -> Tensor:
        features = [init_features]
        for name, layer in self.items():
            new_features = layer(features)
            features.append(new_features)
        return torch.cat(features, 1)


class _Transition(nn.Sequential):
    def __init__(self, num_input_features: int, num_output_features: int) -> None:
        super().__init__()
        self.norm = nn.BatchNorm2d(num_input_features)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(num_input_features, num_output_features, kernel_size=1, stride=1, bias=False)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)


class DenseNet(nn.Module):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_.

    Args:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
        memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
          but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_.
    """

    def __init__(
        self,
        growth_rate: int = 32,
        block_config: Tuple[int, int, int, int] = (6, 12, 24, 16),
        num_init_features: int = 64,
        bn_size: int = 4,
        drop_rate: float = 0,
        num_classes: int = 1000,
        memory_efficient: bool = False,
        module_list = None,
    ) -> None:

        super().__init__()
        
        # self.module_list = module_list
        # self.attention = module_list['attention']
        
        # First convolution
        self.features = nn.Sequential(
            OrderedDict(
                [
                    ("conv0", nn.Conv2d(3, num_init_features, kernel_size=7, stride=2, padding=3, bias=False)),
                    ("norm0", nn.BatchNorm2d(num_init_features)),
                    ("relu0", nn.ReLU(inplace=True)),
                    ("pool0", nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
                ]
            )
        )

        # Each denseblock
        num_features = num_init_features #初始通道数
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient,
            )
            self.features.add_module("denseblock%d" % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2)
                self.features.add_module("transition%d" % (i + 1), trans)
                num_features = num_features // 2
        self.last_num_features=num_features

        # Final batch norm
        self.features.add_module("norm5", nn.BatchNorm2d(num_features))
        
        # self.mymodules=get_modules(self.module_list, num_features, num_classes=num_classes)

        # Linear layer
        if isinstance(num_classes, list):
            self.classifier=nn.ModuleList([nn.Linear(num_features, n) for n in num_classes])
        else:
            self.classifier = nn.Linear(num_features, num_classes)

        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x: Tensor) -> Tensor:
        features = self.features(x)
        
        # for i in self.mymodules:
        #     features = i(features)
        
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = torch.flatten(out, 1)
        if isinstance(self.classifier, nn.ModuleList):
            x = [classifier(out) for classifier in self.classifier]
        else:
            x = self.classifier(out)
        return x




def _densenet(
    growth_rate: int,
    block_config: Tuple[int, int, int, int],
    num_init_features: int,
    **kwargs: Any,
) -> DenseNet:
    print(kwargs)
    url = kwargs.pop('url', NOKEY)

    model = DenseNet(growth_rate, block_config, num_init_features, **kwargs)
    
    if url != NOKEY:
        weight_dict=torch.hub.load_state_dict_from_url(url, progress=True)
        del_key = []
        for key, _ in weight_dict.items():  # 遍历预训练权重的有序字典
            if "classifier" in key:  # 如果key中包含'classifier'这个字段
                del_key.append(key)

        for key in del_key:  # 遍历要删除字段的list
            del weight_dict[key]
        # 抽出现有模型中的K,V
        model_dict=model.state_dict()
        # 新建权重字典，并更新
        state_dict={k:v for k,v in weight_dict.items() if k in model_dict.keys()}
        # 更新现有模型的权重字典
        model_dict.update(state_dict)
        # 载入更新后的权重字典
        model.load_state_dict(model_dict)

    return model

DenseNetWeight={
        'densenet121':'https://download.pytorch.org/models/densenet121-a639ec97.pth',
        'densenet161':'https://download.pytorch.org/models/densenet161-8d451a50.pth',
        'densenet169':'https://download.pytorch.org/models/densenet169-b2777c0a.pth',
        'densenet201':'https://download.pytorch.org/models/densenet201-c1103571.pth'
    }


def densenet121(**kwargs: Any) -> DenseNet:
    pretrained = kwargs.pop('pretrained',NOKEY)
    if pretrained != NOKEY and pretrained:
        kwargs['url'] = DenseNetWeight['densenet121']

    return _densenet(32, (6, 12, 24, 16), 64, **kwargs)


def densenet161(**kwargs: Any) -> DenseNet:
    pretrained = kwargs.pop('pretrained',NOKEY)
    if pretrained != NOKEY and pretrained:
        kwargs['url'] = DenseNetWeight['densenet161']

    return _densenet(48, (6, 12, 36, 24), 96, **kwargs)

def densenet169(**kwargs: Any) -> DenseNet:
    pretrained = kwargs.pop('pretrained',NOKEY)
    if pretrained != NOKEY and pretrained:
        kwargs['url'] = DenseNetWeight['densenet169']

    return _densenet(32, (6, 12, 32, 32), 64, **kwargs)

def densenet201(**kwargs: Any) -> DenseNet:
    pretrained = kwargs.pop('pretrained',NOKEY)
    if pretrained != NOKEY and pretrained:
        kwargs['url'] = DenseNetWeight['densenet201']

    return _densenet(32, (6, 12, 48, 32), 64, **kwargs)

print('densenet.py')

densenet.py


In [14]:
from functools import partial
from typing import Any, Callable, List, Optional

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.parameter import Parameter
NOKEY=-1

# from model.modules import *


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class Bottleneck(nn.Module):
    expansion = 4
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        
        branch_features = planes * self.expansion
        
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None, module_list = None):
        super(ResNet, self).__init__()
        self.module_list = module_list
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
            self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group

        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        if isinstance(num_classes, list):
            self.fc=nn.ModuleList([nn.Linear(512 * block.expansion, n) for n in num_classes])
        else:
            self.fc = nn.Linear(512 * block.expansion, num_classes)

        if self.module_list:
            self.mymodules=get_modules(self.module_list, 2048, num_classes=num_classes)
        
        self.attmap=self.module_list.get('use_attmap', False)
        if self.attmap:
            self.attmap_low = nn.ModuleList([nn.Sequential(
                                                            nn.Conv2d(1, self.conv1.out_channels//2, 3, 2, 1),
                                                            nn.Conv2d(self.conv1.out_channels//2, self.conv1.out_channels, 3, padding=1),
                                                            nn.BatchNorm2d(self.conv1.out_channels),
                                                            nn.ReLU(inplace=True),
                                                            )
                                            for _ in range(self.attmap)])
            
            self.attmap_high = nn.ModuleList([nn.Sequential(
                                                            nn.Conv2d(self.conv1.out_channels, 1, 1),
                                                            nn.BatchNorm2d(1),
                                                            nn.ReLU(inplace=True)
                                                            )
                                            for _ in range(self.attmap)])
            self.upconv = nn.ModuleList([
                nn.Sequential(
                            nn.Upsample(scale_factor=2),
                            nn.Conv2d(1,1,1),
                            nn.BatchNorm2d(1),
                            nn.ReLU(inplace=True)
                        )
                for _ in range(self.attmap)
            ])
            self.cat_attmap = nn.Linear(512 * block.expansion * self.attmap, 512 * block.expansion)
            self.high_conv1x1 = nn.Conv2d(512 * block.expansion, self.conv1.out_channels, 1)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x, attmaps=None):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        if attmaps != None:
            low_feature=x
            for i in range(len(attmaps)):
                attmap = attmaps[i].to(low_feature.device)
                attmap = attmap.unsqueeze(1)
                attmap = self.attmap_low[i](attmap)
                
                attmaps[i] = attmap + low_feature
        
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        if self.module_list:
            for i in self.mymodules:
                x = i(x)

        x = self.avgpool(x)
        
        if attmaps != None:
            refined_attmaps=[]
            high_feature = self.high_conv1x1(x)
            for i, attmap in enumerate(attmaps):
                refined_attmap=self.attmap_high[i](attmap*high_feature)
                # print(f'refined_attmap:{refined_attmap.shape}')
                refined_attmaps.append(self.upconv[i](refined_attmap))
                attmap = refined_attmap+low_feature
                attmap = self.layer1(attmap)
                attmap = self.layer2(attmap)
                attmap = self.layer3(attmap)
                attmap = self.layer4(attmap)
                
                if self.module_list:
                    for m in self.mymodules:
                        attmap = m(attmap)
                attmaps[i] = self.avgpool(attmap)
                
            x = torch.cat(attmaps, dim=1)
            x = torch.flatten(x, 1)
            x = self.cat_attmap(x)
        
        x = torch.flatten(x, 1)
        
        if isinstance(self.fc, nn.ModuleList):
            out = [fc(x) for fc in self.fc]
        else:
            out = self.fc(x)
        
        if attmaps != None:
            return out, torch.cat(refined_attmaps, dim=1)
        return out


def _resnet(arch, block, layers, **kwargs):
    print(kwargs)
    url = kwargs.pop('url', NOKEY)
    weights = kwargs.pop('weights', NOKEY)
    
    model = ResNet(block, layers, **kwargs)
    
    if url != NOKEY or weights != NOKEY:
        if url != NOKEY:
            print('backbone loading Imagenet weights')
            weight_dict=torch.hub.load_state_dict_from_url(url, progress=True)
        elif weights != NOKEY and weights != None:
            print('backbone loading custome weights')
            weight_dict=torch.load(weights, map_location=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
            weight_dict=weight_dict['model_state_dict']
        else:
            return model
        del_key = []
        for key, _ in weight_dict.items():  # 遍历预训练权重的有序字典
            if "fc" in key:  # 如果key中包含'fc'这个字段
                del_key.append(key)

        for key in del_key:  # 遍历要删除字段的list
            del weight_dict[key]
        # 抽出现有模型中的K,V
        model_dict=model.state_dict()
        # 新建权重字典，并更新
        state_dict={k:v for k,v in weight_dict.items() if k in model_dict.keys()}
        # print(model_dict.keys())
        # print(weight_dict.keys())
        # print(state_dict.keys())
        # 更新现有模型的权重字典
        model_dict.update(state_dict)
        # 载入更新后的权重字典
        model.load_state_dict(model_dict)
    return model


def resnet50(**kwargs):
    pretrained = kwargs.pop('pretrained',NOKEY)
    if pretrained != NOKEY and pretrained:
        kwargs['url'] = 'https://download.pytorch.org/models/resnet50-0676ba61.pth'
    
    model = _resnet('ResNet-50', Bottleneck, [3, 4, 6, 3], **kwargs)
    return model


def resnet101(**kwargs):
    pretrained = kwargs.pop('pretrained',NOKEY)
    if pretrained != NOKEY and pretrained:
        kwargs['url'] = 'https://download.pytorch.org/models/resnet101-cd907fc2.pth'
    
    model = _resnet('ResNet-101', Bottleneck, [3, 4, 23, 3], **kwargs)
    return model


def resnet152(**kwargs):
    pretrained = kwargs.pop('pretrained',NOKEY)
    if pretrained != NOKEY and pretrained:
        kwargs['url'] = 'https://download.pytorch.org/models/resnet152-f82ba261.pth'
    
    model = _resnet('ResNet-152', Bottleneck, [3, 8, 36, 3], **kwargs)
    return model

def get_resnet_stages(downsample_factor, backbone, module_list, args, **kwargs):
    rsd=[downsample_factor<(2**i) for i in range(3,6)]
    if backbone == 'rs50':
        encoder = resnet50(pretrained=args.pretrained, module_list=module_list, weights=args.backbone_weights, replace_stride_with_dilation=rsd)
    elif backbone == 'rs101':
        encoder = resnet101(pretrained=args.pretrained, module_list=module_list, replace_stride_with_dilation=rsd)
    elif backbone == 'rs152':
        encoder = resnet152(pretrained=args.pretrained, module_list=module_list, replace_stride_with_dilation=rsd)
    else:
        assert f'{backbone} does not exists!'
    
    conv1 = nn.Sequential(encoder.conv1,
                               encoder.bn1,
                               encoder.relu)#x/2
    
    conv2=nn.Sequential(encoder.maxpool,
                             encoder.layer1)#x/4
    
    conv3=nn.Sequential(encoder.layer2)#x/8
    
    conv4=nn.Sequential(encoder.layer3)#x/16
    
    conv5=nn.Sequential(encoder.layer4)#x/32

    features = [conv1, conv2, conv3, conv4, conv5]
    channels = [64, 256, 512, 1024, 2048]

    return features, channels

print('resnet.py')


resnet.py


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import einops

class conv_block(nn.Module):
    def __init__(self,
                 in_features,
                 out_features,
                 kernel_size=(3, 3),
                 stride=(1, 1),
                 padding=(1, 1),
                 dilation=(1, 1),
                 norm_type='bn',
                 activation=True,
                 use_bias=True, 
                 ):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=in_features,
                              out_channels=out_features,
                              kernel_size=kernel_size,
                              stride=stride,
                              padding=padding,
                              dilation=dilation,
                              bias=use_bias)

        self.norm_type = norm_type
        self.act = activation

        if self.norm_type == 'gn':
            self.norm = nn.GroupNorm(32 if out_features >= 32 else out_features, out_features)
        if self.norm_type == 'bn':
            self.norm = nn.BatchNorm2d(out_features)
        if self.act:
            self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x = self.conv(x)
        if self.norm_type is not None:
            x = self.norm(x)
        if self.act:
            x = self.relu(x)
        return x

class depthwise_conv_block(nn.Module):
    def __init__(self, 
                in_features, 
                out_features,
                kernel_size=(3, 3),
                stride=(1, 1), 
                padding=(1, 1), 
                dilation=(1, 1),
                groups=None, 
                norm_type='bn',
                activation=True, 
                use_bias=True,
                pointwise=False, 
                ):
        super().__init__()
        self.pointwise = pointwise
        self.norm = norm_type
        self.act = activation
        self.depthwise = nn.Conv2d(
            in_channels=in_features,
            out_channels=in_features if pointwise else out_features,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            groups=groups,
            dilation=dilation, 
            bias=use_bias)
        if pointwise:
            self.pointwise = nn.Conv2d(in_features, 
                                        out_features, 
                                        kernel_size=(1, 1), 
                                        stride=(1, 1), 
                                        padding=(0, 0),
                                        dilation=(1, 1), 
                                        bias=use_bias)

        self.norm_type = norm_type
        self.act = activation

        if self.norm_type == 'gn':
            self.norm = nn.GroupNorm(32 if out_features >= 32 else out_features, out_features)
        if self.norm_type == 'bn':
            self.norm = nn.BatchNorm2d(out_features)
        if self.act:
            self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x = self.depthwise(x)
        if self.pointwise:
            x = self.pointwise(x)
        if self.norm_type is not None:
            x = self.norm(x)
        if self.act:
            x = self.relu(x)
        return x

class depthwise_projection(nn.Module):
    def __init__(self, 
                in_features, 
                out_features, 
                groups,
                kernel_size=(1, 1), 
                padding=(0, 0), 
                norm_type=None, 
                activation=False, 
                pointwise=False) -> None:
        super().__init__()

        self.proj = depthwise_conv_block(in_features=in_features, 
                                        out_features=out_features, 
                                        kernel_size=kernel_size,
                                        padding=padding,
                                        groups=groups,
                                        pointwise=pointwise, 
                                        norm_type=norm_type,
                                        activation=activation)
                            
    def forward(self, x):
        P = int(x.shape[1] ** 0.5)
        x = einops.rearrange(x, 'B (H W) C-> B C H W', H=P) 
        x = self.proj(x)
        x = einops.rearrange(x, 'B C H W -> B (H W) C')
        return x
    
class ScaleDotProduct(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.softmax = nn.Softmax(dim=-1)
                                                    
    def forward(self, x1, x2, x3, scale):
        x2 = x2.transpose(-2, -1) #b n p*p ck || b n ck p*p
        x12 = torch.einsum('bhcw, bhwk -> bhck', x1, x2) * scale #可以用torch.matmul代替
        att = self.softmax(x12) #b n cq ck | x3: b n cv p*p || #b n p*p p*p | x3:b n p*p cv
        x123 = torch.einsum('bhcw, bhwk -> bhck', att, x3) 
        return x123 #b n cq p*p || b n p*p cv

class PoolEmbedding(nn.Module):
    def __init__(self,
                pooling,
                patch,
                ) -> None:
        super().__init__()
        self.projection = pooling(output_size=(patch, patch))

    def forward(self, x):
        x = self.projection(x)
        x = einops.rearrange(x, 'B C H W -> B (H W) C')        
        return x

class ChannelCrossAttention(nn.Module):
    def __init__(self, in_features, out_features, n_heads=1) -> None:
        super().__init__()
        self.n_heads = n_heads
        self.q_map = depthwise_projection(in_features=out_features, 
                                            out_features=out_features, 
                                            groups=out_features)
        self.k_map = depthwise_projection(in_features=in_features, 
                                            out_features=in_features, 
                                            groups=in_features)
        self.v_map = depthwise_projection(in_features=in_features, 
                                            out_features=in_features, 
                                            groups=in_features) 

        self.projection = depthwise_projection(in_features=out_features, 
                                    out_features=out_features, 
                                    groups=out_features)
        self.sdp = ScaleDotProduct()        
        

    def forward(self, x):
        q, k, v = x[0], x[1], x[2]
        q = self.q_map(q)
        k = self.k_map(k)
        v = self.v_map(v)
        b, hw, c_q = q.shape
        c = k.shape[2]
        scale = c ** -0.5                     
        q = q.reshape(b, hw, self.n_heads, c_q // self.n_heads).permute(0, 2, 1, 3).transpose(2, 3)# b n c_q p*p
        k = k.reshape(b, hw, self.n_heads, c // self.n_heads).permute(0, 2, 1, 3).transpose(2, 3)
        v = v.reshape(b, hw, self.n_heads, c // self.n_heads).permute(0, 2, 1, 3).transpose(2, 3)
        att = self.sdp(q, k ,v, scale).permute(0, 3, 1, 2).flatten(2) #b p*p n c_q | flatten(2) -> b p*p n*c_q
        att = self.projection(att)
        return att# b p*p c_q

class SpatialCrossAttention(nn.Module):
    def __init__(self, in_features, out_features, n_heads=4) -> None:
        super().__init__()
        self.n_heads = n_heads

        self.q_map = depthwise_projection(in_features=in_features, 
                                            out_features=in_features, 
                                            groups=in_features)
        self.k_map = depthwise_projection(in_features=in_features, 
                                            out_features=in_features, 
                                            groups=in_features)
        self.v_map = depthwise_projection(in_features=out_features, 
                                            out_features=out_features, 
                                            groups=out_features)       

        self.projection = depthwise_projection(in_features=out_features, 
                                    out_features=out_features, 
                                    groups=out_features)                                             
        self.sdp = ScaleDotProduct()        

    def forward(self, x):
        q, k, v = x[0], x[1], x[2]
        q = self.q_map(q)
        k = self.k_map(k)
        v = self.v_map(v)  
        b, hw, c = q.shape
        c_v = v.shape[2]
        scale = (c // self.n_heads) ** -0.5        
        q = q.reshape(b, hw, self.n_heads, c // self.n_heads).permute(0, 2, 1, 3) #b n p*p c/n
        k = k.reshape(b, hw, self.n_heads, c // self.n_heads).permute(0, 2, 1, 3)
        v = v.reshape(b, hw, self.n_heads, c_v // self.n_heads).permute(0, 2, 1, 3) #b n p*p c_v/n
        att = self.sdp(q, k ,v, scale).transpose(1, 2).flatten(2)#b p*p n c_v/n | flatten(2) -> b p*p n*c_v/n
        x = self.projection(att) # b p*p c_v
        return x

class CCSABlock(nn.Module):
    def __init__(self, 
                features, 
                channel_head, 
                spatial_head, 
                spatial_att=True, 
                channel_att=True) -> None:
        super().__init__()
        self.channel_att = channel_att
        self.spatial_att = spatial_att
        if self.channel_att:
            self.channel_norm = nn.ModuleList([nn.LayerNorm(in_features,
                                                    eps=1e-6) 
                                                    for in_features in features])#对每张图片进行标准化   

            self.c_attention = nn.ModuleList([ChannelCrossAttention(
                                                in_features=sum(features),
                                                out_features=feature,
                                                n_heads=head, 
                                        ) for feature, head in zip(features, channel_head)])
        if self.spatial_att:
            self.spatial_norm = nn.ModuleList([nn.LayerNorm(in_features,
                                                    eps=1e-6) 
                                                    for in_features in features])          
          
            self.s_attention = nn.ModuleList([SpatialCrossAttention(
                                                    in_features=sum(features),
                                                    out_features=feature,
                                                    n_heads=head, 
                                                    ) 
                                                    for feature, head in zip(features, spatial_head)])

    def forward(self, x):
        if self.channel_att:
            x_ca = self.channel_attention(x)
            x = self.m_sum(x, x_ca)   
        if self.spatial_att:
            x_sa = self.spatial_attention(x)
            x = self.m_sum(x, x_sa)   
        return x

    def channel_attention(self, x):
        x_c = self.m_apply(x, self.channel_norm) #b p*p c
        x_cin = self.cat(*x_c) #b p*p c*4
        x_in = [[q, x_cin, x_cin] for q in x_c]
        x_att = self.m_apply(x_in, self.c_attention)
        return x_att    

    def spatial_attention(self, x):
        x_c = self.m_apply(x, self.spatial_norm)
        x_cin = self.cat(*x_c)
        x_in = [[x_cin, x_cin, v] for v in x_c]        
        x_att = self.m_apply(x_in, self.s_attention)
        return x_att 
        

    def m_apply(self, x, module):
        return [module[i](j) for i, j in enumerate(x)]

    def m_sum(self, x, y):
        return [xi + xj for xi, xj in zip(x, y)]    

    def cat(self, *args):
        return torch.cat((args), dim=2)



class DCA(nn.Module):
    def __init__(self,
                features,
                patch=28,
                n=1,
                ):
        super().__init__()
        self.n = n
        self.features = features
        self.spatial_head = [4, 4, 4, 4]
        self.channel_head = [1, 1, 1, 1]
        self.channel_att = True
        self.spatial_att = True
        self.patch = patch
        self.patch_avg = nn.ModuleList([PoolEmbedding(
                                                    pooling = nn.AdaptiveAvgPool2d,            
                                                    patch=patch,
                                                    ) #b h w c->b p*p c
                                                    for _ in features])                
        self.avg_map = nn.ModuleList([depthwise_projection(in_features=feature,
                                                            out_features=feature, 
                                                            kernel_size=(1, 1),
                                                            padding=(0, 0),
                                                            groups=feature
                                                            ) #1*1 DW
                                                    for feature in features])         
                                
        self.attention = nn.ModuleList([
                                        CCSABlock(features=features, 
                                                  channel_head=self.channel_head, 
                                                  spatial_head=self.spatial_head, 
                                                  channel_att=self.channel_att, 
                                                  spatial_att=self.spatial_att) 
                                                  for _ in range(n)])                                                   
        self.bn_relu = nn.ModuleList([nn.Sequential(
                                                    nn.BatchNorm2d(feature), 
                                                    nn.ReLU()
                                                    ) 
                                                    for feature in features])
    
    def forward(self, raw):
        x = self.m_apply(raw, self.patch_avg)
        x = self.m_apply(x, self.avg_map)
        for block in self.attention:
            x = block(x)
        x = [self.reshape(i) for i in x]
#         for i in range(len(x)):
#             print('after block-----{}----------'.format(i))
#             print(x[i].shape)
        x = self.upsample(x, raw)
        x_out = self.m_sum(x, raw)
        x_out = self.m_apply(x_out, self.bn_relu)
        return (*x_out, )      

    def m_apply(self, x, module):
        return [module[i](j) for i, j in enumerate(x)]

    def m_sum(self, x, y):
        # for i in range(len(x)):
        #     print('-----{}----------'.format(i))
        #     print(x[i].shape)
        #     print(y[i].shape)
        return [xi + xj for xi, xj in zip(x, y)]  
        
    def reshape(self, x):
        return einops.rearrange(x, 'B (H W) C-> B C H W', H=self.patch) 

    def upsample(self, x, raw):
        for i in range(len(x)):
            x[i] = F.interpolate(x[i], scale_factor=raw[i].shape[2]//x[i].shape[2], mode="nearest")
        return x

print('dca')

dca


In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.parameter import Parameter
from torch import Tensor
import math
import cv2
import torch_dct as dct
import einops
# from model.dca import *
# from sklearn.cross_decomposition._pls import CCA
NOKEY=-1

c2wh = dict([(64,56), (128,28), (256,14) ,(512,7)])
my_c2wh = dict([(512,16),(256,32)])
freq_method='top16'
end_modules=['mccsa']

def channel2wh(channels):
    channels=channels//4
    wh=my_c2wh.get(channels, -1)
    if wh == -1:
        if channels < 256:
            wh=64
        else:
            wh=16
    return wh

def get_modules(module_dict, out_c, **kwargs):
    default_values={'fm':'top16'}
    mymodules=nn.ModuleList()
    module_list=list(module_dict.values())
    for i in module_list:
        m=None
        if i == 'aspp':
            m=ASPPModule(out_c)
        elif i == 'biaspp':
            m=ASPPModule(out_c,bi=True)
        elif i == 'sa':
            m=sa_layer(out_c)
        elif i == 'cbam':
            m=nn.Sequential(
                ChannelAttention(out_c),
                SpatialAttention(),
            )
        elif i == 'myffs':
            m=nn.Sequential(
                DFCAM(out_c, channel2wh(out_c), channel2wh(out_c), freq_size=8, freq_sel_method = module_dict.get('freq_method', 'dct16'), fc='conv'),
                FSpatialAttention(transform='dct'),
            )
        elif i == 'fcbam':
            m=nn.Sequential(
                MultiSpectralAttentionLayer(out_c, channel2wh(out_c), channel2wh(out_c), freq_sel_method = module_dict.get('freq_method', default_values['fm'])),
                SpatialAttention(),
            )
        elif i == 'fsa':
            m=nn.Sequential(
                DFCAM(out_c, channel2wh(out_c), channel2wh(out_c), freq_sel_method = module_dict.get('freq_method', default_values['fm']), fc='conv'),
                SpatialAttention(),
            )
        elif i == 'cafs':
            m=nn.Sequential(
                ChannelAttention(out_c),
                FSpatialAttention(transform='dct'),
            )
        
            
        if m!=None: mymodules.append(m)
    return mymodules

def transformTo(x, transform='fft'):
    if transform == 'dct':
        x_np=x.detach().cpu().numpy()
#         for b in range(x_np.shape[0]):
#             for c in range(x_np.shape[1]):
#                 x_np[b,c]=cv2.dct(x_np[b,c])
        x=dct.dct_2d(x)
        x=torch.from_numpy(x_np).to(x.device)
        return x
    elif transform == 'fft':
        return torch.real(torch.fft.fft2(x, dim=(-2, -1),norm='forward'))

'''
#ASPP
'''
   
class ASPPConv(nn.Sequential):
    def __init__(self, in_channels, out_channels, dilation):
        modules = [
            nn.Conv2d(in_channels, out_channels, 3, padding=dilation, dilation=dilation, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        ]
        super(ASPPConv, self).__init__(*modules)

class ASPPPooling(nn.Sequential):
    def __init__(self, in_channels, out_channels):
        super(ASPPPooling, self).__init__(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True))

    def forward(self, x):
        size = x.shape[-2:]
        x = super(ASPPPooling, self).forward(x)
        return F.interpolate(x, size=size, mode='bilinear', align_corners=False)

class ASPPModule(nn.Module):
    def __init__(self, in_channels, atrous_rates=[6, 12, 18], bi=False):
        super(ASPPModule, self).__init__()
        self.bi=bi
        out_channels = 256
        modules = []
        modules.append(nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)))

        rate1, rate2, rate3 = tuple(atrous_rates)
        modules.append(ASPPConv(in_channels, out_channels, rate1))
        modules.append(ASPPConv(in_channels, out_channels, rate2))
        modules.append(ASPPConv(in_channels, out_channels, rate3))
        modules.append(ASPPPooling(in_channels, out_channels))

        self.convs = nn.ModuleList(modules)

        self.project = nn.Sequential(
            nn.Conv2d(5 * out_channels, in_channels, 1, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),)

    def forward(self, x):
#         print('aspp')
        res = []
        for conv in self.convs:
            res.append(conv(x))
            
        if self.bi:
            mid=res[1]+res[2]
            res[3]=mid+res[3]
            res[2]=mid+res[3]
            res[1]=res[1]+res[2]
        
        res = torch.cat(res, dim=1)
        return self.project(res)


'''
Shuffle Attention
'''

class sa_layer(nn.Module):
    """Constructs a Channel Spatial Group module.

    Args:
        k_size: Adaptive selection of kernel size
    """

    def __init__(self, channel, groups=64):
        super(sa_layer, self).__init__()
        self.groups = groups
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.cweight = Parameter(torch.zeros(1, channel // (2 * groups), 1, 1))
        self.cbias = Parameter(torch.ones(1, channel // (2 * groups), 1, 1))
        self.sweight = Parameter(torch.zeros(1, channel // (2 * groups), 1, 1))
        self.sbias = Parameter(torch.ones(1, channel // (2 * groups), 1, 1))

        self.sigmoid = nn.Sigmoid()
        self.gn = nn.GroupNorm(channel // (2 * groups), channel // (2 * groups))

    @staticmethod
    def channel_shuffle(x, groups):
        b, c, h, w = x.shape

        x = x.reshape(b, groups, -1, h, w)
        x = x.permute(0, 2, 1, 3, 4)

        # flatten
        x = x.reshape(b, -1, h, w)

        return x

    def forward(self, x):
#         print('sa')
        b, c, h, w = x.shape

        x = x.reshape(b * self.groups, -1, h, w)
        x_0, x_1 = x.chunk(2, dim=1)

        # channel attention
        xn = self.avg_pool(x_0)
        xn = self.cweight * xn + self.cbias
        xn = x_0 * self.sigmoid(xn)

        # spatial attention
        xs = self.gn(x_1)
        xs = self.sweight * xs + self.sbias
        xs = x_1 * self.sigmoid(xs)

        # concatenate along channel axis
        out = torch.cat([xn, xs], dim=1)
        out = out.reshape(b, -1, h, w)

        out = self.channel_shuffle(out, 2)
        return out
    

    
'''
CBAM
'''

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
           
        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // 16, 1, bias=False),
                               nn.ReLU(),
                               nn.Conv2d(in_planes // 16, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
#         print('cabm')
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        out = self.sigmoid(out)*x
        return out

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        out = torch.cat([avg_out, max_out], dim=1)
        out = self.conv1(out)
        return self.sigmoid(out)*x
    
class FChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(FChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
           
        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // 16, 1, bias=False),
                               nn.ReLU(),
                               nn.Conv2d(in_planes // 16, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        f_x=transformTo(x)
        f_avg_out = self.fc(self.avg_pool(f_x))
        f_max_out = self.fc(self.max_pool(f_x))
        f_out=f_avg_out + f_max_out
        f_out=self.sigmoid(f_out)*f_x
        return f_out
    
class FSpatialAttention(nn.Module):
    def __init__(self,transform='fft', kernel_size=7):
        super(FSpatialAttention, self).__init__()

        self.transform=transform
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        f_x=transformTo(x, self.transform)
        f_avg_out = torch.mean(f_x, dim=1, keepdim=True)
        f_max_out, _ = torch.max(f_x, dim=1, keepdim=True)
        f_out = torch.cat([f_avg_out, f_max_out], dim=1)
        
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        out = torch.cat([avg_out, max_out], dim=1)
        
        out = out*f_out
        out = self.conv1(out)
        
        return self.sigmoid(out)*x

'''
FcaNet
'''
   
def get_freq_indices(method):
    assert method in ['top1','top2','top4','top8','top16','top32',
                      'bot1','bot2','bot4','bot8','bot16','bot32',
                      'low1','low2','low4','low8','low16','low32',
                      'dct2','dct4','dct8','dct16','dct32',
                      'idr2','idr4','idr8','idr16','idr32',]
    num_freq = int(method[3:])
    if 'top' in method:
        all_top_indices_x = [0,0,6,0,0,1,1,4,5,1,3,0,0,0,3,2,4,6,3,5,5,2,6,5,5,3,3,4,2,2,6,1]
        all_top_indices_y = [0,1,0,5,2,0,2,0,0,6,0,4,6,3,5,2,6,3,3,3,5,1,1,2,4,2,1,1,3,0,5,3]
        mapper_x = all_top_indices_x[:num_freq]
        mapper_y = all_top_indices_y[:num_freq]
    elif 'low' in method:
        all_low_indices_x = [0,0,1,1,0,2,2,1,2,0,3,4,0,1,3,0,1,2,3,4,5,0,1,2,3,4,5,6,1,2,3,4]
        all_low_indices_y = [0,1,0,1,2,0,1,2,2,3,0,0,4,3,1,5,4,3,2,1,0,6,5,4,3,2,1,0,6,5,4,3]
        mapper_x = all_low_indices_x[:num_freq]
        mapper_y = all_low_indices_y[:num_freq]
    elif 'bot' in method:
        all_bot_indices_x = [6,1,3,3,2,4,1,2,4,4,5,1,4,6,2,5,6,1,6,2,2,4,3,3,5,5,6,2,5,5,3,6]
        all_bot_indices_y = [6,4,4,6,6,3,1,4,4,5,6,5,2,2,5,1,4,3,5,0,3,1,1,2,4,2,1,1,5,3,3,3]
        mapper_x = all_bot_indices_x[:num_freq]
        mapper_y = all_bot_indices_y[:num_freq]
    elif 'dct' in method:
        all_bot_indices_x = [0, 4, 1, 3, 2, 0, 6, 0, 5, 2, 4, 3, 7, 6, 2, 2, 6, 2, 6, 6, 1, 5, 5, 6, 7, 5, 0, 4, 3, 4, 0, 5]
        all_bot_indices_y = [0, 2, 4, 7, 4, 6, 0, 5, 1, 6, 4, 0, 6, 5, 3, 7, 4, 1, 3, 6, 0, 3, 4, 1, 3, 7, 7, 5, 6, 1, 1, 6]
        mapper_x = all_bot_indices_x[:num_freq]
        mapper_y = all_bot_indices_y[:num_freq]
    elif 'idr' in method:
        all_bot_indices_x = [0, 0, 1, 0, 2, 3, 4, 4, 6, 6, 0, 0, 2, 3, 7, 0, 1, 6, 6, 6, 7, 7, 0, 1, 1, 2, 2, 4, 5, 5, 5, 6]
        all_bot_indices_y = [0, 1, 6, 3, 0, 0, 1, 6, 0, 1, 4, 7, 7, 6, 2, 2, 3, 3, 4, 6, 4, 6, 5, 0, 2, 3, 6, 5, 0, 2, 7, 5]
        mapper_x = all_bot_indices_x[:num_freq]
        mapper_y = all_bot_indices_y[:num_freq]
    else:
        raise NotImplementedError
    return mapper_x, mapper_y

class MultiSpectralAttentionLayer(torch.nn.Module):
    def __init__(self, channel, dct_h, dct_w, reduction = 16, freq_size=7, freq_sel_method = 'top16'):
        super(MultiSpectralAttentionLayer, self).__init__()
        self.reduction = reduction
        self.dct_h = dct_h
        self.dct_w = dct_w

        mapper_x, mapper_y = get_freq_indices(freq_sel_method)
        self.num_split = len(mapper_x)
        mapper_x = [temp_x * (dct_h // freq_size) for temp_x in mapper_x]
        mapper_y = [temp_y * (dct_w // freq_size) for temp_y in mapper_y]
        # make the frequencies in different sizes are identical to a 7x7 frequency space
        # eg, (2,2) in 14x14 is identical to (1,1) in 7x7

        self.dct_layer = MultiSpectralDCTLayer(dct_h, dct_w, mapper_x, mapper_y, channel)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        n,c,h,w = x.shape
        x_pooled = x
        if h != self.dct_h or w != self.dct_w:
            x_pooled = torch.nn.functional.adaptive_avg_pool2d(x, (self.dct_h, self.dct_w))
            # If you have concerns about one-line-change, don't worry.   :)
            # In the ImageNet models, this line will never be triggered. 
            # This is for compatibility in instance segmentation and object detection.
        y = self.dct_layer(x_pooled)

        y = self.fc(y).view(n, c, 1, 1)
        return x * y.expand_as(x)
    
class DCTSpectralLayer(torch.nn.Module):
    def __init__(self, channel, dct_h, dct_w, reduction = 16, freq_size=8, mapper=[0,0]):
        super(DCTSpectralLayer, self).__init__()
        self.reduction = reduction
        self.dct_h = dct_h
        self.dct_w = dct_w

        mapper_x, mapper_y = [mapper[0]],[mapper[1]]
        self.num_split = len(mapper_x)
        mapper_x = [temp_x * (dct_h // freq_size) for temp_x in mapper_x]
        mapper_y = [temp_y * (dct_w // freq_size) for temp_y in mapper_y]
        # make the frequencies in different sizes are identical to a 7x7 frequency space
        # eg, (2,2) in 14x14 is identical to (1,1) in 7x7

        self.dct_layer = MultiSpectralDCTLayer(dct_h, dct_w, mapper_x, mapper_y, channel)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        n,c,h,w = x.shape
        x_pooled = x
        if h != self.dct_h or w != self.dct_w:
            x_pooled = torch.nn.functional.adaptive_avg_pool2d(x, (self.dct_h, self.dct_w))
            # If you have concerns about one-line-change, don't worry.   :)
            # In the ImageNet models, this line will never be triggered. 
            # This is for compatibility in instance segmentation and object detection.
        y = self.dct_layer(x_pooled)

        y = self.fc(y).view(n, c, 1, 1)
        return x * y.expand_as(x)


class MultiSpectralDCTLayer(nn.Module):
    """
    Generate dct filters
    """
    def __init__(self, height, width, mapper_x, mapper_y, channel):
        super(MultiSpectralDCTLayer, self).__init__()
        
        assert len(mapper_x) == len(mapper_y)
        assert channel % len(mapper_x) == 0

        self.num_freq = len(mapper_x)

        # fixed DCT init
        self.register_buffer('weight', self.get_dct_filter(height, width, mapper_x, mapper_y, channel))
        
        # fixed random init
        # self.register_buffer('weight', torch.rand(channel, height, width))

        # learnable DCT init
        # self.register_parameter('weight', self.get_dct_filter(height, width, mapper_x, mapper_y, channel))
        
        # learnable random init
        # self.register_parameter('weight', torch.rand(channel, height, width))

        # num_freq, h, w

    def forward(self, x):
        assert len(x.shape) == 4, 'x must been 4 dimensions, but got ' + str(len(x.shape))
        # n, c, h, w = x.shape

        x = x * self.weight

        result = torch.sum(x, dim=[2,3])
        return result

    def build_filter(self, pos, freq, POS): #即基础函数
        result = math.cos(math.pi * freq * (pos + 0.5) / POS) / math.sqrt(POS) 
        if freq == 0:
            return result
        else:
            return result * math.sqrt(2)
    
    def get_dct_filter(self, tile_size_x, tile_size_y, mapper_x, mapper_y, channel):
        dct_filter = torch.zeros(channel, tile_size_x, tile_size_y)

        c_part = channel // len(mapper_x)

        for i, (u_x, v_y) in enumerate(zip(mapper_x, mapper_y)):
            for t_x in range(tile_size_x):
                for t_y in range(tile_size_y):
                    dct_filter[i * c_part: (i+1)*c_part, t_x, t_y] = self.build_filter(t_x, u_x, tile_size_x) * self.build_filter(t_y, v_y, tile_size_y)
                    #dct_filter即得到每个channel对应的基础函数
                        
        return dct_filter
    
'''
    FCAM
'''
class DFCAM(nn.Module):
    def __init__(self, in_planes, dct_h, dct_w, reduction = 16,freq_size=8, freq_sel_method = 'top16', fc='conv', transform=None):
        super(DFCAM, self).__init__()
        
        self.transform=transform
        self.reduction = reduction
        self.dct_h = dct_h
        self.dct_w = dct_w

        mapper_x, mapper_y = get_freq_indices(freq_sel_method)
        self.num_split = len(mapper_x)
        mapper_x = [temp_x * (dct_h // freq_size) for temp_x in mapper_x]
        mapper_y = [temp_y * (dct_w // freq_size) for temp_y in mapper_y]

        self.dct_layer = MultiSpectralDCTLayer(dct_h, dct_w, mapper_x, mapper_y, in_planes)
        
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc_method = fc
        
        if fc == 'linear':
            self.fc = nn.Sequential(
                nn.Linear(in_planes, in_planes // reduction, bias=False),#这里用的是Linear，所以输入形状必须是【bs,channels】
                nn.ReLU(inplace=True),
                nn.Linear(in_planes // reduction, in_planes, bias=False),
            )
        elif fc == 'conv':
            self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // reduction, 1, bias=False),
                               nn.ReLU(inplace=True),
                               nn.Conv2d(in_planes // reduction, in_planes, 1, bias=False))
        
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        n,c,h,w = x.shape
        x_pooled = x
        if h != self.dct_h or w != self.dct_w:
            x_pooled = torch.nn.functional.adaptive_avg_pool2d(x, (self.dct_h, self.dct_w))
            # If you have concerns about one-line-change, don't worry.   :)
            # In the ImageNet models, this line will never be triggered. 
            # This is for compatibility in instance segmentation and object detection.
        y = self.dct_layer(x_pooled)

        if self.fc_method == 'conv':
            y=y.view(n,c,1,1)
        y = self.sigmoid(self.fc(y)).view(n, c, 1, 1)
        dct_att = x * y.expand_as(x)
        
        
        if self.transform:
            f_x=transformTo(x, self.transform)
        else:
            f_x=x
        
        if self.fc_method == 'linear':
            f_avg_out = self.fc(self.avg_pool(f_x).squeeze())
            f_max_out = self.fc(self.max_pool(f_x).squeeze())
        else:
            f_avg_out = self.fc(self.avg_pool(f_x))
            f_max_out = self.fc(self.max_pool(f_x))
        f_out=f_avg_out + f_max_out
        f_out=f_out.view(n, c, 1, 1)
        f_out=self.sigmoid(f_out)*f_x
        
        return dct_att + f_out
    
    
'''
Global Transformer block
'''
class ChannelSelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(ChannelSelfAttention, self).__init__()
        
        self.inter_channels=in_channels//8
        self.conv1 = nn.Conv2d(in_channels, self.inter_channels, kernel_size=3, padding=1, bias=False)
        self.conv2 = nn.Conv2d(in_channels, self.inter_channels, kernel_size=3, padding=1, bias=False)
        self.conv3 = nn.Conv2d(in_channels, self.inter_channels, kernel_size=3, padding=1, bias=False)
        self.lastconv = nn.Conv2d(self.inter_channels, in_channels, 1, bias=False)
        self.gap = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        q=self.gap(self.conv1(x)).flatten(2) # [b,c,1]
        
        k_T=self.conv2(x).flatten(2).permute(0,2,1) # [b,h*w,c]
        
        v=self.conv3(x).flatten(2) # [b,c,h*w]
        
        att = torch.matmul(k_T, q) # [b,h*w,1]
        att = F.softmax(att, dim=1)
        out = torch.matmul(v, att)
        out = out.view(out.shape[0],out.shape[1],1,1)
        out = self.lastconv(out)
        out = out + x
        
        return out
    
class SpatialSelfAttention(nn.Module):
    def __init__(self, in_channels, extra_channels=None):
        super(SpatialSelfAttention, self).__init__()
        
        self.inter_channels=in_channels//2
        self.conv1 = nn.Conv2d(in_channels, self.inter_channels, kernel_size=3, padding=1, bias=False)
        self.conv2 = nn.Conv2d(in_channels, self.inter_channels, kernel_size=3, padding=1, bias=False)
        self.conv3 = nn.Conv2d(in_channels, self.inter_channels, kernel_size=3, padding=1, bias=False)
        self.lastconv = nn.Conv2d(self.inter_channels, in_channels, 1, bias=False)
        
        if extra_channels is not None:
            self.extra_project=nn.Conv2d(extra_channels, in_channels, 1, bias=False)
            

    def forward(self, x, extra=None):
        b, c, h, w = x.shape
        
        q=self.conv1(x).view(b, self.inter_channels, -1)
        
        if extra is not None:
            extra=self.extra_project(extra)
            feature=extra
        else:
            feature=x
            
        k_T=self.conv2(feature).view(b, self.inter_channels, -1).permute(0,2,1)
        v=self.conv3(feature).view(b, self.inter_channels, -1)
        att = torch.matmul(k_T, q)
        att = F.softmax(att, dim=-1)
        
        out = torch.matmul(v, att)
        out = out.view(b, self.inter_channels, h, w)
        
        out = self.lastconv(out)
        out = out + x
        
        return out
    
class RTBlock(nn.Module):
    def __init__(self, in_channels, patch=None):
        super().__init__()
        self.patch=patch
        if self.patch:
            self.patch_avg = nn.ModuleList([nn.AdaptiveAvgPool2d(output_size=(patch, patch)) for _ in range(2)]) #b h w c->b p*p c
        
            self.avg_map = nn.ModuleList([
                    nn.Sequential(nn.Conv2d(in_channels, in_channels, 1),
                                  nn.BatchNorm2d(in_channels), 
                                  nn.ReLU(inplace=True))
                    for _ in range(2)
                ])

            # self.patch_avg = nn.ModuleList([nn.Conv2d(in_channels,in_channels,kernel_size=patch,stride=patch) for _ in range(2)])
            # self.avg_map = nn.ModuleList([nn.Identity() for _ in range(2)])
            
            self.upconv = nn.ModuleList([
                    nn.Conv2d(in_channels, in_channels, 1) for _ in range(2)
                ])
            
        # self.csa=ChannelSelfAttention(in_channels)
        # self.csa_extra=ChannelSelfAttention(in_channels)
        self.ssa=SpatialSelfAttention(in_channels)
        self.ssa_cross=SpatialSelfAttention(in_channels, in_channels)
        
        self.conv = nn.Conv2d(in_channels*2, in_channels, kernel_size=1, bias=False)
        
    def forward(self, x, extra=None):
        raw_size = x.shape[2]
        if self.patch:
            x = self.patch_avg[0](x)
            x = self.avg_map[0](x)
            if extra != None:
                extra = self.patch_avg[1](extra)
                extra = self.avg_map[1](extra)
        
        # if extra != None:
        #     x = self.csa(x)
        #     extra = self.csa_extra(extra)
        # else:
        #     raw_featrues=x
        #     x = self.csa(x)
        #     extra = self.csa_extra(raw_featrues)
        
        self_att = self.ssa(x)
        cross_att = self.ssa_cross(x, extra)
        
        x = self.conv(torch.cat((self_att,cross_att), dim=1))
        
        if self.patch:
            x = self.upsample(x, raw_size)
            extra = self.upsample(extra, raw_size)
            x = self.upconv[0](x)
            extra = self.upconv[1](extra)
            
        return x, extra

    def upsample(self, x, raw_size):
        if raw_size == x.shape[2]:
            return x
        x = F.interpolate(x, scale_factor=raw_size//x.shape[2], mode="nearest")
        return x
    
class DualCrossAttentionBlock(nn.Module):
    def __init__(self, hidden_channels, patch=None, net_type='linear'):
        super().__init__()
        self.hidden_size = hidden_channels
        self.attention_norm = nn.ModuleList([LayerNorm(hidden_channels, eps=1e-6) for _ in range(2)])
        self.ffn_norm = LayerNorm(hidden_channels, eps=1e-6)
        self.ffn = TranUnetMlp(hidden_channels, net_type=net_type)
        self.attn = RTBlock(hidden_channels, patch=patch)

    def forward(self, x, x_dual):
        raw = x
        b, p2, c= x.shape
        p = int(math.sqrt(p2))
        x = self.attention_norm[0](x)
        x_dual = self.attention_norm[1](x_dual)
        
        x = einops.rearrange(x, 'B (H W) C-> B C H W', H=p) 
        x_dual = einops.rearrange(x_dual, 'B (H W) C-> B C H W', H=p) 
        x, x_dual = self.attn(x, x_dual)
        x = einops.rearrange(x, 'B C H W-> B (H W) C') 
        x_dual = einops.rearrange(x_dual, 'B C H W-> B (H W) C')
        x = x + raw

        raw = x
        x = self.ffn_norm(x)
        x = self.ffn(x)
        x = x + raw
        return x, x_dual
    
class DualCrossTransformer(nn.Module):
    def __init__(self, in_channels, num_layer=1, patch=None, net_type='linear'):
        super().__init__()
        self.blocks=nn.ModuleList([
                DualCrossAttentionBlock(in_channels, patch=patch) for _ in range(num_layer)
            ])
        
    def forward(self, x, x_dual):
        b, c, h, w= x.shape
        x = einops.rearrange(x, 'B C H W-> B (H W) C') 
        x_dual = einops.rearrange(x_dual, 'B C H W-> B (H W) C')
        
        for i,b in enumerate(self.blocks):
            x, x_dual = b(x, x_dual)
            
        x = einops.rearrange(x, 'B (H W) C-> B C H W', H=h) 
        x_dual = einops.rearrange(x_dual, 'B (H W) C-> B C H W', H=h)

        return x, x_dual
        

class MultiScaleFusion(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MultiScaleFusion, self).__init__()

        self.myffs = nn.ModuleList([nn.Sequential(
                        DFCAM(inc, channel2wh(out_channels), channel2wh(out_channels), freq_sel_method = freq_method),
                        FSpatialAttention(transform='dct')
                      ) for inc in in_channels])

        self.upconv = nn.ModuleList([nn.Conv2d(inc, out_channels, 1) for inc in in_channels])
        
        self.conv = nn.Sequential(nn.Conv2d(len(in_channels)*out_channels, out_channels, kernel_size=1, bias=False),
                                  nn.BatchNorm2d(out_channels),
                                  nn.ReLU(inplace = True))


    def forward(self, x):
        # x = self.m_apply(raw, self.myffs)
        max_size = x[-1].shape[2]

        for i,_x in enumerate(x):
            x[i] = self.upsample(_x, max_size)
        x=self.m_apply(x, self.upconv)
        x = self.m_apply(x, self.myffs)
        x = self.conv(torch.cat((x), dim=1))
        
        return x
    
    def m_apply(self, x, module):
        return [module[i](j) for i, j in enumerate(x)]

    def upsample(self, x, size):
        x = F.interpolate(x, scale_factor=size//x.shape[2], mode="nearest")
        return x

class MultiScaleFreqAttention(nn.Module):
    def __init__(self, in_channels, out_channels, freq_method='dct8'):
        super(MultiScaleFreqAttention, self).__init__()

        self.myffs = nn.ModuleList([nn.Sequential(
                        DFCAM(out_channels, channel2wh(out_channels), channel2wh(out_channels), freq_sel_method = freq_method),
                        FSpatialAttention(transform='dct')
                      ) for inc in in_channels])
        
        self.upconv = nn.ModuleList([nn.Conv2d(inc, out_channels, 1) for inc in in_channels])
        
        self.conv = nn.Conv2d(len(in_channels)*out_channels, out_channels, kernel_size=1, bias=False)

    def forward(self, features):
        max_size = features[-1].shape[2]
        
        for i,f in enumerate(features):
            features[i] = self.upsample(f, max_size)
        
        features=self.m_apply(features, self.upconv)
        features=self.m_apply(features, self.myffs)
        
        out = self.conv(torch.cat((features), dim=1))
        
        return out
    
    def m_apply(self, x, module):
        return [module[i](j) for i, j in enumerate(x)]

    def upsample(self, x, size):
        x = F.interpolate(x, scale_factor=size//x.shape[2], mode="nearest")
        return x

# class MultiScaleRelationFusion(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(MultiScaleFusion, self).__init__()
#
#         self.dct = nn.ModuleList([DualCrossTransformer(inc) for inc in in_channels])
#
#         self.upconv = nn.ModuleList([nn.Conv2d(inc, out_channels, 1) for inc in in_channels])
#
#         self.conv = nn.Sequential(nn.Conv2d(len(in_channels)*out_channels, out_channels, kernel_size=1, bias=False),
#                                   nn.BatchNorm2d(out_channels),
#                                   nn.ReLU(inplace = True))
#
#
#     def forward(self, x):
#         # x = self.m_apply(raw, self.myffs)
#         max_size = x[-1].shape[2]
#
#         for i,_x in enumerate(x):
#             x[i] = self.upsample(_x, max_size)
#         x=self.m_apply(x, self.upconv)
#         x = self.m_apply(x, self.myffs)
#         x = self.conv(torch.cat((x), dim=1))
#
#         return x
#
#     def m_apply(self, x, module):
#         return [module[i](j) for i, j in enumerate(x)]
#
#     def upsample(self, x, size):
#         x = F.interpolate(x, scale_factor=size//x.shape[2], mode="nearest")
#         return x


print('modules')

modules


In [17]:
import torch
import math
import torch.nn.functional as F


class KANLinear(torch.nn.Module):
    def __init__(
        self,
        in_features,
        out_features,
        grid_size=5,
        spline_order=3,
        scale_noise=0.1,
        scale_base=1.0,
        scale_spline=1.0,
        enable_standalone_scale_spline=True,
        base_activation=torch.nn.SiLU,
        grid_eps=0.02,
        grid_range=[-1, 1],
    ):
        super(KANLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.grid_size = grid_size
        self.spline_order = spline_order

        h = (grid_range[1] - grid_range[0]) / grid_size
        grid = (
            (
                torch.arange(-spline_order, grid_size + spline_order + 1) * h
                + grid_range[0]
            )
            .expand(in_features, -1)
            .contiguous()
        )
        self.register_buffer("grid", grid)

        self.base_weight = torch.nn.Parameter(torch.Tensor(out_features, in_features))
        self.spline_weight = torch.nn.Parameter(
            torch.Tensor(out_features, in_features, grid_size + spline_order)
        )
        if enable_standalone_scale_spline:
            self.spline_scaler = torch.nn.Parameter(
                torch.Tensor(out_features, in_features)
            )

        self.scale_noise = scale_noise
        self.scale_base = scale_base
        self.scale_spline = scale_spline
        self.enable_standalone_scale_spline = enable_standalone_scale_spline
        self.base_activation = base_activation()
        self.grid_eps = grid_eps

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.kaiming_uniform_(self.base_weight, a=math.sqrt(5) * self.scale_base)
        with torch.no_grad():
            noise = (
                (
                    torch.rand(self.grid_size + 1, self.in_features, self.out_features)
                    - 1 / 2
                )
                * self.scale_noise
                / self.grid_size
            )
            self.spline_weight.data.copy_(
                (self.scale_spline if not self.enable_standalone_scale_spline else 1.0)
                * self.curve2coeff(
                    self.grid.T[self.spline_order : -self.spline_order],
                    noise,
                )
            )
            if self.enable_standalone_scale_spline:
                # torch.nn.init.constant_(self.spline_scaler, self.scale_spline)
                torch.nn.init.kaiming_uniform_(self.spline_scaler, a=math.sqrt(5) * self.scale_spline)

    def b_splines(self, x: torch.Tensor):
        """
        Compute the B-spline bases for the given input tensor.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_features).

        Returns:
            torch.Tensor: B-spline bases tensor of shape (batch_size, in_features, grid_size + spline_order).
        """
        assert x.dim() == 2 and x.size(1) == self.in_features

        grid: torch.Tensor = (
            self.grid
        )  # (in_features, grid_size + 2 * spline_order + 1)
        x = x.unsqueeze(-1)
        bases = ((x >= grid[:, :-1]) & (x < grid[:, 1:])).to(x.dtype)
        for k in range(1, self.spline_order + 1):
            bases = (
                (x - grid[:, : -(k + 1)])
                / (grid[:, k:-1] - grid[:, : -(k + 1)])
                * bases[:, :, :-1]
            ) + (
                (grid[:, k + 1 :] - x)
                / (grid[:, k + 1 :] - grid[:, 1:(-k)])
                * bases[:, :, 1:]
            )

        assert bases.size() == (
            x.size(0),
            self.in_features,
            self.grid_size + self.spline_order,
        )
        return bases.contiguous()

    def curve2coeff(self, x: torch.Tensor, y: torch.Tensor):
        """
        Compute the coefficients of the curve that interpolates the given points.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_features).
            y (torch.Tensor): Output tensor of shape (batch_size, in_features, out_features).

        Returns:
            torch.Tensor: Coefficients tensor of shape (out_features, in_features, grid_size + spline_order).
        """
        assert x.dim() == 2 and x.size(1) == self.in_features
        assert y.size() == (x.size(0), self.in_features, self.out_features)

        A = self.b_splines(x).transpose(
            0, 1
        )  # (in_features, batch_size, grid_size + spline_order)
        B = y.transpose(0, 1)  # (in_features, batch_size, out_features)
        solution = torch.linalg.lstsq(
            A, B
        ).solution  # (in_features, grid_size + spline_order, out_features)
        result = solution.permute(
            2, 0, 1
        )  # (out_features, in_features, grid_size + spline_order)

        assert result.size() == (
            self.out_features,
            self.in_features,
            self.grid_size + self.spline_order,
        )
        return result.contiguous()

    @property
    def scaled_spline_weight(self):
        return self.spline_weight * (
            self.spline_scaler.unsqueeze(-1)
            if self.enable_standalone_scale_spline
            else 1.0
        )

    def forward(self, x: torch.Tensor):
        assert x.size(-1) == self.in_features
        original_shape = x.shape
        x = x.reshape(-1, self.in_features)

        base_output = F.linear(self.base_activation(x), self.base_weight)
        spline_output = F.linear(
            self.b_splines(x).view(x.size(0), -1),
            self.scaled_spline_weight.view(self.out_features, -1),
        )
        output = base_output + spline_output
        
        output = output.reshape(*original_shape[:-1], self.out_features)
        return output

    @torch.no_grad()
    def update_grid(self, x: torch.Tensor, margin=0.01):
        assert x.dim() == 2 and x.size(1) == self.in_features
        batch = x.size(0)

        splines = self.b_splines(x)  # (batch, in, coeff)
        splines = splines.permute(1, 0, 2)  # (in, batch, coeff)
        orig_coeff = self.scaled_spline_weight  # (out, in, coeff)
        orig_coeff = orig_coeff.permute(1, 2, 0)  # (in, coeff, out)
        unreduced_spline_output = torch.bmm(splines, orig_coeff)  # (in, batch, out)
        unreduced_spline_output = unreduced_spline_output.permute(
            1, 0, 2
        )  # (batch, in, out)

        # sort each channel individually to collect data distribution
        x_sorted = torch.sort(x, dim=0)[0]
        grid_adaptive = x_sorted[
            torch.linspace(
                0, batch - 1, self.grid_size + 1, dtype=torch.int64, device=x.device
            )
        ]

        uniform_step = (x_sorted[-1] - x_sorted[0] + 2 * margin) / self.grid_size
        grid_uniform = (
            torch.arange(
                self.grid_size + 1, dtype=torch.float32, device=x.device
            ).unsqueeze(1)
            * uniform_step
            + x_sorted[0]
            - margin
        )

        grid = self.grid_eps * grid_uniform + (1 - self.grid_eps) * grid_adaptive
        grid = torch.concatenate(
            [
                grid[:1]
                - uniform_step
                * torch.arange(self.spline_order, 0, -1, device=x.device).unsqueeze(1),
                grid,
                grid[-1:]
                + uniform_step
                * torch.arange(1, self.spline_order + 1, device=x.device).unsqueeze(1),
            ],
            dim=0,
        )

        self.grid.copy_(grid.T)
        self.spline_weight.data.copy_(self.curve2coeff(x, unreduced_spline_output))

    def regularization_loss(self, regularize_activation=1.0, regularize_entropy=1.0):
        """
        Compute the regularization loss.

        This is a dumb simulation of the original L1 regularization as stated in the
        paper, since the original one requires computing absolutes and entropy from the
        expanded (batch, in_features, out_features) intermediate tensor, which is hidden
        behind the F.linear function if we want an memory efficient implementation.

        The L1 regularization is now computed as mean absolute value of the spline
        weights. The authors implementation also includes this term in addition to the
        sample-based regularization.
        """
        l1_fake = self.spline_weight.abs().mean(-1)
        regularization_loss_activation = l1_fake.sum()
        p = l1_fake / regularization_loss_activation
        regularization_loss_entropy = -torch.sum(p * p.log())
        return (
            regularize_activation * regularization_loss_activation
            + regularize_entropy * regularization_loss_entropy
        )
        
print('kan.py')

kan.py


In [18]:
# coding=utf-8
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import copy
import logging
import math

from os.path import join as pjoin

import torch
import torch.nn as nn
import numpy as np

from torch.nn import CrossEntropyLoss, Dropout, Softmax, Linear, Conv2d, LayerNorm
from torch.nn.modules.utils import _pair
from scipy import ndimage
from timm.models.layers import DropPath, to_2tuple, trunc_normal_

ACT2FN = {"gelu": torch.nn.functional.gelu, "relu": torch.nn.functional.relu}


class TranUnetAttention(nn.Module):
    def __init__(self, hidden_channels, num_heads=8, net_type='linear'):
        super(TranUnetAttention, self).__init__()
        self.num_attention_heads = num_heads
        self.attention_head_size = int(hidden_channels / self.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        if net_type == 'linear':
            net=Linear
        elif net_type == 'kan':
            net=KANLinear
            
        self.query = net(hidden_channels, self.all_head_size)
        self.key = net(hidden_channels, self.all_head_size)
        self.value = net(hidden_channels, self.all_head_size)

        self.out = net(hidden_channels, hidden_channels)

        self.softmax = Softmax(dim=-1)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states):
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        attention_probs = self.softmax(attention_scores)

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)
        attention_output = self.out(context_layer)
        return attention_output


class TranUnetMlp(nn.Module):
    def __init__(self, hidden_channels, expand_ratio=4, net_type='linear'):
        super(TranUnetMlp, self).__init__()
        self.net_type=net_type
        if net_type == 'linear':
            net=Linear
        elif net_type == 'kan':
            net=KANLinear
            
        self.fc1 = net(hidden_channels, hidden_channels*expand_ratio)
        self.fc2 = net(hidden_channels*expand_ratio, hidden_channels)
        self.act_fn = ACT2FN["gelu"]
        self.dropout = Dropout(0.1)

        self._init_weights()

    def _init_weights(self):
        if self.net_type == 'linear':
            nn.init.xavier_uniform_(self.fc1.weight)
            nn.init.xavier_uniform_(self.fc2.weight)
            nn.init.normal_(self.fc1.bias, std=1e-6)
            nn.init.normal_(self.fc2.bias, std=1e-6)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act_fn(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x


class Embeddings(nn.Module):
    """Construct the embeddings from patch, position embeddings.
    """
    def __init__(self, patch_size, img_size, downsample_factor, pretrained=True):
        super(Embeddings, self).__init__()
        self.downsample_factor = downsample_factor
        img_size = img_size//downsample_factor
        patch_size = patch_size
        patch = img_size // patch_size       
        n_patches = patch**2
        # print(f'img_size:{img_size}')
        self.img_size=img_size
        self.patch=patch
        self.hidden_channels=patch_size**2 * 3
        backbone='rs50'
        rsd=[downsample_factor<(2**i) for i in range(3,6)]
        if backbone == 'rs50':
            encoder = resnet50(pretrained=pretrained, replace_stride_with_dilation=rsd)
        elif backbone == 'rs101':
            encoder = resnet101(pretrained=pretrained, replace_stride_with_dilation=rsd)
        elif backbone == 'rs152':
            encoder = resnet152(pretrained=pretrained, replace_stride_with_dilation=rsd)
        else:
            assert f'{backbone} does not exists!'
        
        conv1 = nn.Sequential(encoder.conv1,
                                   encoder.bn1,
                                   encoder.relu)#x/2
        
        conv2=nn.Sequential(encoder.maxpool,
                                 encoder.layer1)#x/4
        
        conv3=nn.Sequential(encoder.layer2)#x/8
        
        conv4=nn.Sequential(encoder.layer3)#x/16
        
        conv5=nn.Sequential(encoder.layer4)#x/32
    
        stages = [conv1, conv2, conv3, conv4, conv5]
        self.stages = nn.ModuleList(stages)
        self.in_channels = [0, 64, 256, 512, 1024]
        self.skip_channels = self.in_channels[::-1]
        
        
        self.patch_embeddings = Conv2d(in_channels=2048,
                                       out_channels=self.hidden_channels,
                                       kernel_size=patch_size,
                                       stride=patch_size)
        self.position_embeddings = nn.Parameter(torch.zeros(1, n_patches, self.hidden_channels))

        self.dropout = Dropout(0.1)


    def forward(self, x):
        
        features=[]
        for m in self.stages:
            x=m(x)
            features.append(x)
        #feature不需要conv5，conv5不属于skip
        last_conv=features[-1]
        features=features[:-1]
        features=features[::-1]
        x = self.patch_embeddings(last_conv)  # (B, hidden. n_patches^(1/2), n_patches^(1/2))
        # print(x.shape)
        x = x.flatten(2)
        x = x.transpose(-1, -2)  # (B, n_patches, hidden)
        # print(x.shape)

        embeddings = x + self.position_embeddings
        embeddings = self.dropout(embeddings)
        return embeddings, features


def window_partition(x, window_size):
    """
    Args:
        x: (B, H, W, C)
        window_size (int): window size
    Returns:
        windows: (num_windows*B, window_size, window_size, C)
    """
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)
    return windows


def window_reverse(windows, window_size, H, W):
    """
    Args:
        windows: (num_windows*B, window_size, window_size, C)
        window_size (int): Window size
        H (int): Height of image
        W (int): Width of image
    Returns:
        x: (B, H, W, C)
    """
    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
    return x


class WindowAttention(nn.Module): # W-MSA in the paper
    r""" Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.
    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self, dim, window_size, num_heads, net_type='linear', qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):

        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wh, Ww
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads))  # 2*Wh-1 * 2*Ww-1, nH

        # get pair-wise relative position index for each token inside the window
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        coords = torch.stack(torch.meshgrid([coords_h, coords_w], indexing='ij'))  # 2, Wh, Ww
        coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
        relative_coords[:, :, 0] += self.window_size[0] - 1  # shift to start from 0
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        self.register_buffer("relative_position_index", relative_position_index)
        
        self.net_type=net_type
        if net_type == 'linear':
            net=Linear
        elif net_type == 'kan':
            net=KANLinear
        
        self.qkv = net(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = net(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        """
        Args:
            x: input features with shape of (num_windows*B, N, C) >>> (B * 32*32, 4*4, 192)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x.shape
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)  #AMBIGUOUS X)
        q, k, v = qkv[0], qkv[1], qkv[2]  # make torchscript happy (cannot use tensor as tuple)

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))

        relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)  # Wh*Ww,Wh*Ww,nH
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
        attn = attn + relative_position_bias.unsqueeze(0)

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class SwinTransformerBlock(nn.Module):
    r""" Swin Transformer Block.
    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resulotion.
        num_heads (int): Number of attention heads.
        window_size (int): Window size.
        shift_size (int): Shift size for SW-MSA.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set.
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float, optional): Stochastic depth rate. Default: 0.0
        act_layer (nn.Module, optional): Activation layer. Default: nn.GELU
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, dim, input_resolution, num_heads=12, window_size=7, shift_size=3,
                 qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0., norm_layer=nn.LayerNorm, net_type='linear'):
        super().__init__()
        self.dim = dim
        self.input_resolution = (input_resolution,input_resolution)
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        if min(self.input_resolution) <= self.window_size:
            # if window size is larger than input resolution, we don't partition windows
            self.shift_size = 0
            self.window_size = min(self.input_resolution)
        assert 0 <= self.shift_size < self.window_size, "shift_size must in 0-window_size"

        self.norm1 = norm_layer(dim)
        self.attn = WindowAttention(
            dim, window_size=to_2tuple(self.window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop, net_type=net_type)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        self.mlp = TranUnetMlp(dim, net_type=net_type)

        if self.shift_size > 0:
            # calculate attention mask for SW-MSA
            H, W = self.input_resolution
            img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
            h_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1

            mask_windows = window_partition(img_mask, self.window_size)  # nW, window_size, window_size, 1
            mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None

        self.register_buffer("attn_mask", attn_mask)

    def forward(self, x):
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"

        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)

        # cyclic shift
        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
        else:
            shifted_x = x

        # partition windows
        x_windows = window_partition(shifted_x, self.window_size)  # nW*B, window_size, window_size, C
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)  # nW*B, window_size*window_size, C

        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=self.attn_mask)  # nW*B, window_size*window_size, C

        # merge windows
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)
        shifted_x = window_reverse(attn_windows, self.window_size, H, W)  # B H' W' C

        # reverse cyclic shift
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x
        x = x.view(B, H * W, C)

        # FFN
        x = shortcut + self.drop_path(x)
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x


class TranUnetBlock(nn.Module):
    def __init__(self, hidden_channels, net_type='linear'):
        super(TranUnetBlock, self).__init__()
        self.hidden_size = hidden_channels
        self.attention_norm = LayerNorm(hidden_channels, eps=1e-6)
        self.ffn_norm = LayerNorm(hidden_channels, eps=1e-6)
        self.ffn = TranUnetMlp(hidden_channels, net_type=net_type)
        self.attn = TranUnetAttention(hidden_channels, net_type=net_type)

    def forward(self, x):
        h = x
        x = self.attention_norm(x)
        x = self.attn(x)
        x = x + h

        h = x
        x = self.ffn_norm(x)
        x = self.ffn(x)
        x = x + h
        return x


class Encoder(nn.Module):
    def __init__(self, hidden_channels, num_layers=12, net_type='linear', transformer_block='vit', **kwargs):
        super(Encoder, self).__init__()
        self.layer = nn.ModuleList()
        self.encoder_norm = LayerNorm(hidden_channels, eps=1e-6)
        
        if transformer_block == 'vit':
            for _ in range(num_layers):
                layer = TranUnetBlock(hidden_channels, net_type=net_type)
                self.layer.append(copy.deepcopy(layer))
        elif transformer_block == 'swin':
            for i in range(num_layers):
                layer = SwinTransformerBlock(dim=hidden_channels, input_resolution=kwargs['input_resolution'],
                                     shift_size=0 if (i % 2 == 0) else 3)
                self.layer.append(copy.deepcopy(layer))

    def forward(self, hidden_states):
        for layer_block in self.layer:
            # print(hidden_states.shape)
            hidden_states = layer_block(hidden_states)
        encoded = self.encoder_norm(hidden_states)
        return encoded

class Transformer(nn.Module):
    def __init__(self, patch_size, img_size, downsample_factor, num_layers, transformer_block='vit', net_type='linear'):
        super(Transformer, self).__init__()
        self.embeddings = Embeddings(patch_size=patch_size, img_size=img_size, downsample_factor=downsample_factor)
        self.encoder = Encoder(self.embeddings.hidden_channels, num_layers, net_type=net_type, transformer_block=transformer_block, input_resolution=self.embeddings.patch)
        self.skip_channels=self.embeddings.skip_channels
        self.hidden_channels=self.embeddings.hidden_channels
        self.patch=self.embeddings.patch

    def forward(self, input_ids):
        embedding_output, features = self.embeddings(input_ids)
        encoded = self.encoder(embedding_output)  # (B, n_patch, hidden)
        return encoded, features


class Conv2dReLU(nn.Sequential):
    def __init__(
            self,
            in_channels,
            out_channels,
            kernel_size,
            padding=0,
            stride=1,
            use_batchnorm=True,
    ):
        conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride=stride,
            padding=padding,
            bias=not (use_batchnorm),
        )
        relu = nn.ReLU(inplace=True)

        bn = nn.BatchNorm2d(out_channels)

        super(Conv2dReLU, self).__init__(conv, bn, relu)


class TranUnetDecoderBlock(nn.Module):
    def __init__(
            self,
            in_channels,
            out_channels,
            skip_channels=0,
            use_batchnorm=True,
    ):
        super().__init__()
        self.conv1 = Conv2dReLU(
            in_channels + skip_channels,
            out_channels,
            kernel_size=3,
            padding=1,
            use_batchnorm=use_batchnorm,
        )
        self.conv2 = Conv2dReLU(
            out_channels,
            out_channels,
            kernel_size=3,
            padding=1,
            use_batchnorm=use_batchnorm,
        )
        self.up = nn.UpsamplingBilinear2d(scale_factor=2)

    def forward(self, x, skip=None):
        x = self.up(x)
        # print(f'x:{x.shape}')
        if skip is not None:
            # print(f'skip:{skip.shape}')
            x = torch.cat([x, skip], dim=1)
        x = self.conv1(x)
        x = self.conv2(x)
        return x


class SegmentationHead(nn.Sequential):

    def __init__(self, in_channels, out_channels, kernel_size=3, upsampling=1):
        conv2d = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size // 2)
        upsampling = nn.UpsamplingBilinear2d(scale_factor=upsampling) if upsampling > 1 else nn.Identity()
        super().__init__(conv2d, upsampling)


class DecoderCup(nn.Module):
    def __init__(self, hidden_channels, skip_channels, upsample_mode):
        super().__init__()
        head_channels = skip_channels[0]
        self.conv_more = Conv2dReLU(
            hidden_channels,
            head_channels,
            kernel_size=3,
            padding=1,
            use_batchnorm=True,
        )
        self.decoder_channels = [1024, 512, 256, 128, 64, 16]

        self.decoder_channels = self.decoder_channels[6-len(skip_channels):]
        in_channels = [head_channels] + list(self.decoder_channels[:-1]) #[1024, 512,256,128,64]
        out_channels = self.decoder_channels
        self.skip_channels =skip_channels #[1024,512,256,64,0]

        blocks = [
            DecoderBlock(in_ch, out_ch, sk_ch, mode=upsample_mode) for in_ch, out_ch, sk_ch in zip(in_channels, out_channels, skip_channels)
        ]

        self.blocks = nn.ModuleList(blocks)

    def forward(self, hidden_states, features=None):
        x = self.conv_more(hidden_states)
        for i, decoder_block in enumerate(self.blocks):
            if self.skip_channels[i] != 0:
                x = decoder_block(x, skip=features[i])
            else:
                x = decoder_block(x)
        return x
    
class UnetPPDecoderCup(nn.Module):
    def __init__(
        self, hidden_channels, skip_channels, upsample_mode
    ):
        super().__init__()
        head_channels = skip_channels[0]
        self.conv_more = Conv2dReLU(
            hidden_channels,
            head_channels,
            kernel_size=3,
            padding=1,
            use_batchnorm=True,
        )
        self.decoder_channels = [1024, 512, 256, 128, 64, 16]

        self.decoder_channels = self.decoder_channels[6-len(skip_channels):]
        self.in_channels = [head_channels] + list(self.decoder_channels[:-1])#[512,256,128,64]
        self.out_channels = self.decoder_channels
        self.skip_channels = skip_channels#[512,256,64,0]
        self.depth = len(self.in_channels) - 1

        blocks = {}
        for layer_idx in range(len(self.in_channels) - 1):
            for depth_idx in range(layer_idx + 1):
                if depth_idx == 0:
                    in_ch = self.in_channels[layer_idx]
                    skip_ch = self.skip_channels[layer_idx] * (layer_idx + 1)
                    out_ch = self.out_channels[layer_idx]
                    # print(f'x_{depth_idx}_{layer_idx}--in_ch:{in_ch}')
                    # print(f'x_{depth_idx}_{layer_idx}--skip_ch:{skip_ch}')
                    # print(f'x_{depth_idx}_{layer_idx}--out_ch:{out_ch}')
                else:
                    out_ch = self.skip_channels[layer_idx]
                    skip_ch = self.skip_channels[layer_idx] * (
                        layer_idx + 1 - depth_idx
                    )
                    in_ch = self.skip_channels[layer_idx - 1]
                blocks[f"x_{depth_idx}_{layer_idx}"] = DecoderBlock(
                    in_ch, out_ch, skip_ch, mode=upsample_mode
                )
        blocks[f"x_{0}_{len(self.in_channels)-1}"] = DecoderBlock(
            self.in_channels[-1], self.out_channels[-1], 0, mode='interp'
        )
        self.blocks = nn.ModuleDict(blocks)
        # print(blocks)

    def forward(self, hidden_states, features=None):
        x = self.conv_more(hidden_states)
        features = [x] + features
        # for i,f in enumerate(features):
        #     print(f'f[i]:{f.shape}')
        # start building dense connections
        dense_x = {}
        for layer_idx in range(len(self.in_channels) - 1):
            for depth_idx in range(self.depth - layer_idx):
                if layer_idx == 0:
                    # print(f'features[{depth_idx}].shape:{features[depth_idx].shape}')
                    # print(f'features[{depth_idx+1}].shape:{features[depth_idx+1].shape}')
                    output = self.blocks[f"x_{depth_idx}_{depth_idx}"](
                        features[depth_idx], features[depth_idx + 1]
                    )
                    dense_x[f"x_{depth_idx}_{depth_idx}"] = output
                else:
                    dense_l_i = depth_idx + layer_idx
                    cat_features = [
                        dense_x[f"x_{idx}_{dense_l_i}"]
                        for idx in range(depth_idx + 1, dense_l_i + 1)
                    ]
                    cat_features = torch.cat(
                        cat_features + [features[dense_l_i + 1]], dim=1
                    )
                    dense_x[f"x_{depth_idx}_{dense_l_i}"] = self.blocks[
                        f"x_{depth_idx}_{dense_l_i}"
                    ](dense_x[f"x_{depth_idx}_{dense_l_i-1}"], cat_features)
        dense_x[f"x_{0}_{self.depth}"] = self.blocks[f"x_{0}_{self.depth}"](
            dense_x[f"x_{0}_{self.depth-1}"]
        )

        last_out = dense_x[f"x_{0}_{self.depth}"]
        return last_out


class VisionTransformer(nn.Module):
    def __init__(self, img_size=224, num_classes=4, downsample_factor=8, upsample_mode='interp', transformer_block='vit', net_type='linear', decoder='unet'):
        super(VisionTransformer, self).__init__()
        self.num_classes = num_classes
        self.patch_size= 8 if img_size<=512 else 16
        self.transformer = Transformer(patch_size=self.patch_size, img_size=img_size, downsample_factor=downsample_factor, num_layers=12, transformer_block=transformer_block, net_type=net_type)
        self.downsample_factor=downsample_factor
        if decoder == 'unet':
            self.decoder = DecoderCup(self.transformer.hidden_channels, self.transformer.skip_channels, upsample_mode)
        elif decoder == 'unetpp':
            self.decoder = UnetPPDecoderCup(self.transformer.hidden_channels, self.transformer.skip_channels, upsample_mode)
        self.segmentation_head = SegmentationHead(
            in_channels=self.decoder.decoder_channels[-1],
            out_channels=num_classes,
            kernel_size=3,
        )

    def forward(self, x):
        x, features = self.transformer(x)  # (B, n_patch, hidden)
        # print(f'transformer x:{x.shape}')
        B, n_patch, hidden = x.size()  # reshape from (B, n_patch, hidden) to (B, h, w, hidden)
        h, w = int(np.sqrt(n_patch)), int(np.sqrt(n_patch))
        x = x.permute(0, 2, 1)
        x = x.contiguous().view(B, hidden, h, w)
        # print(f'before up:{x.shape}')
        x = F.interpolate(x, scale_factor=self.patch_size, mode="nearest")
        x = self.decoder(x, features)
        logits = self.segmentation_head(x)
        return logits
    
print('transunet.py')

transunet.py


In [19]:
from torch import nn
from torch.nn import functional as F
import torch
import einops
import math



class ConvRelu(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.conv1 = nn.Conv2d(in_c, out_c, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_c)

        self.conv2 = nn.Conv2d(out_c, out_c, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_c)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, skip_channels=0, is_up=True, scale_factor=2, mode='conv', fusion='cat'):
        super(DecoderBlock, self).__init__()
        self.in_channels = in_channels
        self.is_up=is_up
        self.mode = mode
        self.fusion = fusion

        if mode == 'conv':
            self.up = nn.ConvTranspose2d(in_channels, in_channels, kernel_size=2*scale_factor, stride=scale_factor, padding=scale_factor//2)
        elif mode == 'interp':
            self.up = nn.Upsample(scale_factor=scale_factor)
        
        self.conv = ConvRelu(in_channels + skip_channels, out_channels)
        
    def forward(self, x, skip=None):
        if skip is not None:
            if skip.shape[2] == x.shape[2]:
                x = x
            else:
                x = self.up(x)
        else:
            x = self.up(x)
            
            
        if skip != None:
            # print(f'x:{x.shape}')
            # print(f'skip:{skip.shape}')

            x = torch.cat([x, skip], axis=1)
            # print(f'cat_x:{x.shape}')
            
        x = self.conv(x)
            
        return x
    
class UNetResNet(nn.Module):
    def __init__(self, num_classes=1, pretrained=False, module_list = None, downsample_factor=8, upsample_mode='conv', dual=False, **kwargs):
        super(UNetResNet, self).__init__()

        self.args = kwargs['args']
        self.downsample_factor=downsample_factor
        self.module_list = module_list
        self.backbone=kwargs['backbone']
        self.resolution=kwargs['resolution']
        self.num_classes=num_classes
        self.dual=dual
        if self.backbone[:2] == 'rs':
            self.stages, encoder_channels=get_resnet_stages(self.downsample_factor, self.backbone, self.module_list, self.args)
        elif 'wtconvnext' in self.backbone:
            self.stages, encoder_channels=get_wtconvnext_stages(self.downsample_factor, self.backbone, self.args)
        self.stages=nn.ModuleList(self.stages)
        
        decoder_channels = [1024,512,256,128,64,32]
        decoder_channels = decoder_channels[6-len(encoder_channels):]
        
        encoder_channels = encoder_channels[::-1]
        
        head_channels = encoder_channels[0]
        self.in_channels = [head_channels] + list(decoder_channels[:-1]) # 2048, 256, 128, 64, 32, 16
        self.skip_channels = list(encoder_channels[1:]) + [0] # 1024,512,128,64,0
        self.out_channels = decoder_channels 
        
        blocks = [
            DecoderBlock(in_ch, out_ch, sk_ch, mode=upsample_mode) for in_ch, out_ch, sk_ch in zip(self.in_channels, self.out_channels, self.skip_channels)
        ]
        self.blocks = nn.ModuleList(blocks)

        if self.dual:
            if self.args.seg_vessel:
                self.blocks_dual = nn.ModuleList([
                    DecoderBlock(in_ch, out_ch, sk_ch, mode=upsample_mode) for in_ch, out_ch, sk_ch in zip(self.in_channels, self.out_channels, self.skip_channels)
                ])
                self.final_dual = nn.Conv2d(decoder_channels[-1], 1, 1)                            
            else:
                self.stages_dual, _=get_resnet_stages(self.downsample_factor, self.backbone, self.module_list, self.args)
                self.stages_dual=nn.ModuleList(self.stages)
                if self.module_list:
                    if self.module_list.get('serial_eam', False):
                        if self.module_list['serial_eam'] == 'dct':
                            dual_cross_att=DualCrossTransformer
                        elif self.module_list['serial_eam'] == 'rtb':
                            dual_cross_att=RTBlock
                    else:
                        dual_cross_att=RTBlock
                else:
                    dual_cross_att=RTBlock
                self.dual_cross_att=nn.ModuleList([dual_cross_att(in_c) for in_c in encoder_channels[1:3][::-1]])
            

        self.final = nn.Conv2d(decoder_channels[-1], num_classes, kernel_size=1)
            
        if self.module_list:
            self.att_modules = nn.Sequential(*get_modules(self.module_list, head_channels))
            if self.module_list.get('msfm', False):
                if self.module_list['msfm'] == 'dca':
                    patch=int(self.resolution/min(16,self.downsample_factor))#应该变为crop_size=512，resolution是1024
                    self.dca=DCA(n=1,
                                 features = encoder_channels[::-1][:-1],
                                 patch=patch)
                elif self.module_list['msfm'] == 'msf':
                    patch=int(self.resolution/self.downsample_factor)
                    self.msf=MultiScaleFusion(in_channels=encoder_channels[::-1])
                if self.dual and self.args.seg_vessel:
                    if self.module_list['msfm'] == 'rtb':
                        self.rtb=RTBlock(encoder_channels[0])
            if self.module_list.get('dam', False):
                if self.module_list['dam'] == 'msfa':
                    self.msfa=MultiScaleFreqAttention(decoder_channels[:-1], decoder_channels[-2], freq_method=self.module_list.get('freq_method','dct8'))
                if self.dual and self.args.seg_vessel:
                    if self.module_list['dam'] == 'rtb':
                        self.rtb=RTBlock(decoder_channels[-1], 16)
                if self.module_list['dam'] == 'msf':
                    self.msf=MultiScaleFusion(decoder_channels[:-1], decoder_channels[-2])
            if self.module_list.get('serial_dam', False):
                if self.module_list['serial_dam'] == 'dct':
                    dual_cross_att=DualCrossTransformer
                elif self.module_list['serial_dam'] == 'rtb':
                    dual_cross_att=RTBlock
                self.dual_cross_att=nn.ModuleList([dual_cross_att(in_c, patch=None if i<2 else 32) for i,in_c in enumerate(decoder_channels)])
            
        
    
    def forward(self, x, x_dual=None):
        encoder_features=[]
        if self.dual and not self.args.seg_vessel:
            x_dual = x_dual.unsqueeze(1).repeat(1,3,1,1)
            for i,(m, m_dual) in enumerate(zip(self.stages, self.stages_dual)):
                x=m(x)
                x_dual=m_dual(x_dual)
                encoder_features.append(x)
                if i>=2 and i<4:
                    x, x_dual = self.dual_cross_att[i-2](x, x_dual)
        else:
            for m in self.stages:
                x=m(x)
                encoder_features.append(x)

        if self.module_list:
            encoder_features[-1] = self.att_modules(encoder_features[-1])
            if self.module_list.get('msfm', False):
                if self.module_list['msfm'] == 'dca':
                    encoder_features[:-1] = self.dca(encoder_features[:-1])
                elif self.module_list['msfm'] == 'msf':
                    encoder_features = self.msf(encoder_features)
                if self.dual and self.args.seg_vessel:
                    if self.module_list['msfm'] == 'rtb':
                        x, x_dual = self.rtb(x)
                        
        encoder_features = encoder_features[::-1]
        decoder_features=[]
        encoder_features=encoder_features[1:]
        head_feature = x
        
        if self.dual and self.args.seg_vessel:
            decoder_features_dual=[]

        for i, decoder_block in enumerate(self.blocks):
            if self.skip_channels[i] != 0:
                x = decoder_block(x, skip=encoder_features[i])
                if self.dual and self.args.seg_vessel:
                    if x_dual == None:
                        x_dual = head_feature
                    x_dual = self.blocks_dual[i](x_dual, skip=encoder_features[i])
                    if self.module_list:
                        if self.module_list.get('serial_dam', False):
                            x, x_dual = self.dual_cross_att[i](x, x_dual)
                    decoder_features_dual.append(x_dual)
                decoder_features.append(x)
            
        if self.module_list:
            if self.module_list.get('dam', False):
                if self.module_list['dam'] == 'msfa':
                    x = self.msfa(decoder_features)
                if self.dual and self.args.seg_vessel:
                    if self.module_list['dam'] == 'rtb':
                        x, x_dual = self.rtb(x,x_dual)
                        
        x = self.blocks[-1](x)
        
        if self.dual and self.args.seg_vessel:
            x_dual = self.blocks_dual[-1](x_dual)
            return self.final(x), self.final_dual(x_dual)

        return self.final(x)


print('unet.py')

unet.py


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
    
class UnetPlusDecoderBlock(nn.Module):
    def __init__(
        self,
        in_channels,
        skip_channels,
        out_channels
    ):
        super().__init__()
        self.conv = ConvRelu(in_channels+skip_channels, out_channels)

    def forward(self, x, skip=None):
        if skip is not None:
            # print(f'skip:{skip.shape}')
            # print(f'x:{x.shape}')
            if skip.shape[2] == x.shape[2]:
                x = x
            else:
                x = F.interpolate(x, scale_factor=2, mode="nearest")
        else:
            x = F.interpolate(x, scale_factor=2, mode="nearest")
        if skip is not None:
            x = torch.cat([x, skip], dim=1)
        x = self.conv(x)
        return x


class UnetPP(nn.Module):
    def __init__(
        self,
        **kwargs
    ):
        super().__init__()
        self.args=kwargs['args']
        self.resolution=kwargs['resolution']
        self.downsample_factor= self.args.downsample_factor
        self.module_list = self.args.module_list
        self.backbone= self.args.backbone
        self.num_classes= self.args.num_classes
        self.dual=self.args.dual
        upsample_mode=self.args.upsample_mode
        
        if self.backbone[:2] == 'rs':
            self.stages, encoder_channels=get_resnet_stages(self.downsample_factor, self.backbone, self.module_list, self.args)
        elif 'wtconvnext' in self.backbone:
            self.stages, encoder_channels=get_wtconvnext_stages(self.downsample_factor, self.backbone, self.args)
        
        self.stages=nn.ModuleList(self.stages)
        # encoder_channels = [self.conv1_out_c, self.conv2_out_c, self.conv3_out_c, self.conv4_out_c, self.conv5_out_c]
        decoder_channels = [1024,512,256,128,64,32]
        decoder_channels = decoder_channels[6-len(encoder_channels):]
        
        encoder_channels = encoder_channels[::-1]

        # computing blocks input and output channels
        head_channels = encoder_channels[0]
        self.in_channels = [head_channels] + list(decoder_channels[:-1]) # 2048, 256, 128, 64, 32, 16
        self.skip_channels = list(encoder_channels[1:]) + [0] # 1024,512,128,64,0
        self.out_channels = decoder_channels 

        blocks = {}
        for layer_idx in range(len(self.in_channels) - 1):
            for depth_idx in range(layer_idx + 1):
                if depth_idx == 0:
                    in_ch = self.in_channels[layer_idx]
                    skip_ch = self.skip_channels[layer_idx] * (layer_idx + 1)
                    out_ch = self.out_channels[layer_idx]
                else:
                    out_ch = self.skip_channels[layer_idx]
                    skip_ch = self.skip_channels[layer_idx] * (
                        layer_idx + 1 - depth_idx
                    )
                    in_ch = self.skip_channels[layer_idx - 1]
                blocks[f"x_{depth_idx}_{layer_idx}"] = DecoderBlock(
                    in_ch, out_ch, skip_ch, mode=self.args.upsample_mode
                )
        blocks[f"x_{0}_{len(self.in_channels)-1}"] = DecoderBlock(
            self.in_channels[-1], self.out_channels[-1], 0, mode=self.args.upsample_mode, scale_factor=2 if len(encoder_channels) == 5 else 4
        )

        self.final = nn.Conv2d(decoder_channels[-1], self.num_classes, kernel_size=1)
        
        self.blocks = nn.ModuleDict(blocks)
        self.depth = len(self.in_channels) - 1
        
        if self.dual:
            if self.args.seg_vessel:
                self.blocks_dual = nn.ModuleList([
                    DecoderBlock(in_ch, out_ch, sk_ch, mode=upsample_mode) for in_ch, out_ch, sk_ch in zip(self.in_channels, self.out_channels, self.skip_channels)
                ])
                self.final_dual = nn.Conv2d(decoder_channels[-1], 1, 1)
            else:
                self.stages_dual, _=get_resnet_stages(self.downsample_factor, self.backbone, self.module_list, self.args)
                self.stages_dual=nn.ModuleList(self.stages)
                if self.module_list:
                    if self.module_list.get('cross_att', False):
                        if self.module_list['cross_att'] == 'dct':
                            dual_cross_att=DualCrossTransformer
                        elif self.module_list['cross_att'] == 'rtb':
                            dual_cross_att=RTBlock
                    else:
                        dual_cross_att=RTBlock
                else:
                    dual_cross_att=RTBlock
                self.dual_cross_att=nn.ModuleList([dual_cross_att(in_c) for in_c in encoder_channels[1:3][::-1]])
            
        
        
        
        if self.module_list:
            self.att_modules = nn.Sequential(*get_modules(self.module_list, head_channels))
            if self.module_list.get('msfm', False):
                if self.module_list['msfm'] == 'dca':
                    patch=int(self.resolution/min(16,self.downsample_factor))#应该变为crop_size=512，resolution是1024
                    self.dca=DCA(n=1,
                                 features = encoder_channels[::-1][:-1],
                                 patch=patch)
                elif self.module_list['msfm'] == 'msf':
                    patch=int(self.resolution/self.downsample_factor)
                    self.msf=MultiScaleFusion(in_channels=encoder_channels[::-1])
                if self.dual and self.args.seg_vessel:
                    if self.module_list['msfm'] == 'rtb':
                        self.rtb=RTBlock(encoder_channels[0])
            if self.module_list.get('dam', False):
                if self.module_list['dam'] == 'msfa':
                    self.msfa=MultiScaleFreqAttention(decoder_channels[:-1], decoder_channels[-2], freq_method=self.module_list.get('freq_method','dct8'))
                if self.dual and self.args.seg_vessel:
                    if self.module_list['dam'] == 'rtb':
                        self.rtb=RTBlock(decoder_channels[-1], 16)
                if self.module_list['dam'] == 'msf':
                    self.msf=MultiScaleFusion(decoder_channels[:-1], decoder_channels[-2])
        

    def forward(self, x, x_dual=None):
        encoder_features=[]
        if self.dual and not self.args.seg_vessel:
            x_dual = x_dual.unsqueeze(1).repeat(1,3,1,1)
            for i,(m, m_dual) in enumerate(zip(self.stages, self.stages_dual)):
                x=m(x)
                x_dual=m_dual(x_dual)
                encoder_features.append(x)
                if i>=2 and i<4:
                    x, x_dual = self.dual_cross_att[i-2](x, x_dual)
        else:
            for m in self.stages:
                x=m(x)
                encoder_features.append(x)


        if self.module_list:
            encoder_features[-1] = self.att_modules(encoder_features[-1])
            if self.module_list.get('msfm', False):
                if self.module_list['msfm'] == 'dca':
                    encoder_features[:-1] = self.dca(encoder_features[:-1])
                elif self.module_list['msfm'] == 'msf':
                    encoder_features = self.msf(encoder_features)
                if self.dual and self.args.seg_vessel:
                    if self.module_list['msfm'] == 'rtb':
                        x, x_dual = self.rtb(x)
                    
        features = encoder_features[::-1]
        encoder_features = encoder_features[::-1]
        # print('features:\n')
        # for i,f in enumerate(features):
        #     print('features[{}].shape={}'.format(i, f.shape))
            
        # start building dense connections
        dense_x = {}
        for layer_idx in range(len(self.in_channels) - 1):
            for depth_idx in range(self.depth - layer_idx):
                if layer_idx == 0:
                    output = self.blocks[f"x_{depth_idx}_{depth_idx}"](
                        features[depth_idx], features[depth_idx + 1]
                    )
                    # print('\ndense_x[x_{}_{}].shape={},\n input:features[{}].shape={},\n skip:features[{}].shape={}'.format(
                    #         depth_idx, depth_idx, output.shape, 
                    #         depth_idx, features[depth_idx].shape,
                    #         depth_idx+1, features[depth_idx + 1].shape))
                    dense_x[f"x_{depth_idx}_{depth_idx}"] = output
                else:
                    dense_l_i = depth_idx + layer_idx
                    cat_names=['dense[x_{}_{}].shape={}'.format(idx, dense_l_i, dense_x[f"x_{idx}_{dense_l_i}"].shape) for idx in range(depth_idx + 1, dense_l_i + 1)]
                    # print(f'cat_features:\n--------\n{cat_names}\n--------\n')
                    cat_features = [
                        dense_x[f"x_{idx}_{dense_l_i}"]
                        for idx in range(depth_idx + 1, dense_l_i + 1)
                    ]
                    # print('cat_features+=features[{}].shape={}\n'.format(dense_l_i + 1, features[dense_l_i + 1].shape))
                    cat_features = torch.cat(
                        cat_features + [features[dense_l_i + 1]], dim=1
                    )
                    dense_x[f"x_{depth_idx}_{dense_l_i}"] = self.blocks[
                        f"x_{depth_idx}_{dense_l_i}"
                    ](dense_x[f"x_{depth_idx}_{dense_l_i-1}"], cat_features)
                    # print('dense_x[x_{}_{}].shape={},\n input:dense_x[f"x_{}_{}"].shape={},\n skip:cat_features.shape={}'.format(
                    #         depth_idx, dense_l_i, dense_x[f"x_{depth_idx}_{dense_l_i}"].shape, 
                    #         depth_idx, dense_l_i-1, dense_x[f"x_{depth_idx}_{dense_l_i-1}"].shape,
                    #         cat_features.shape))
        dense_x[f"x_{0}_{self.depth}"] = self.blocks[f"x_{0}_{self.depth}"](
            dense_x[f"x_{0}_{self.depth-1}"]
        )
        # print('dense_x[x_{}_{}].shape={}, input:dense_x[f"x_{}_{}"].shape={}'.format(
        #                     0, self.depth, dense_x[f"x_{0}_{self.depth}"].shape, 
        #                     0, self.depth-1, dense_x[f"x_{0}_{self.depth-1}"].shape))

        if self.dual and self.args.seg_vessel:
            decoder_features_dual=[]
            if x_dual == None:
                x_dual = encoder_features[0]
                encoder_features=encoder_features[1:]
            for i, decoder_block in enumerate(self.blocks_dual):
                if self.skip_channels[i] != 0:
                    # print('encoder_features[{}].shape={}'.format(i, encoder_features[i].shape))
                    x_dual = decoder_block(x_dual, skip=encoder_features[i])
                    decoder_features_dual.append(x_dual)

        decoder_features=[dense_x[f'x_{0}_{d}'] for d in range(self.depth)]
        # for df in decoder_features:
        #     print(df.shape)

        x = dense_x[f'x_{0}_{self.depth}']
        
        if self.module_list:
            if self.module_list.get('dam', False):
                if self.module_list['dam'] == 'msfa':
                    x = self.msfa(decoder_features)
                    x = self.blocks[f"x_{0}_{self.depth}"](x)
                if self.module_list['dam'] == 'msf':
                    x = self.msf(decoder_features)
                    x = self.blocks[f"x_{0}_{self.depth}"](x)
                    
        
        
        if self.dual and self.args.seg_vessel:
            x_dual = self.blocks_dual[-1](x_dual)
            return self.final(x), self.final_dual(x_dual)
        
        out = self.final(x)
        return out

print('unetpp.py')

unetpp.py


In [21]:
from torch import nn
from torch.nn import functional as F
import torch


class ResNetLseg(nn.Module):
    def __init__(self, num_classes=1, pretrained=False, module_list = None, downsample_factor=8, task_num=4, **kwargs):
        super(ResNetLseg, self).__init__()
        
        self.task_num=task_num
        self.downsample_factor=downsample_factor
        self.module_list = module_list
        self.backbone=kwargs['backbone']
        self.rsd=[self.downsample_factor<(2**i) for i in range(3,6)]
        if self.backbone == 'rs50':
            self.encoder = resnet50(pretrained=pretrained, replace_stride_with_dilation=self.rsd)
        elif self.backbone == 'rs101':
            self.encoder = resnet101(pretrained=pretrained, replace_stride_with_dilation=self.rsd)
        elif self.backbone == 'rs152':
            self.encoder = resnet152(pretrained=pretrained, replace_stride_with_dilation=self.rsd)
        else:
            assert f'{self.backbone} does not exists!'
        
        self.conv1 = nn.Sequential(self.encoder.conv1,
                                   self.encoder.bn1,
                                   self.encoder.relu)#x/2
        self.conv1_out_c=64
        self.supervision1=nn.Sequential(nn.Conv2d(self.conv1_out_c,self.task_num,1),
                                        nn.Upsample(scale_factor=2))
        
        self.conv2=nn.Sequential(self.encoder.maxpool,
                                 self.encoder.layer1)#x/4
        self.conv2_out_c=256
        self.supervision2=nn.Sequential(nn.Conv2d(self.conv2_out_c,self.task_num,1),
                                        nn.Upsample(scale_factor=4))
        
        self.conv3=nn.Sequential(self.encoder.layer2)#x/8
        self.conv3_out_c=512
        self.supervision3=nn.Sequential(nn.Conv2d(self.conv3_out_c,self.task_num,1),
                                        nn.Upsample(scale_factor=8))
        
        self.conv4=nn.Sequential(self.encoder.layer3)#x/16
        self.conv4_out_c=1024
        self.supervision4=nn.Sequential(nn.Conv2d(self.conv4_out_c,self.task_num,1),
                                        nn.Upsample(scale_factor=min(16,self.downsample_factor)))
        
        self.conv5=nn.Sequential(self.encoder.layer4)
        self.conv5_out_c=2048
        self.supervision5=nn.Sequential(nn.Conv2d(self.conv5_out_c,self.task_num,1),
                                        nn.Upsample(scale_factor=min(32,self.downsample_factor)))
        
        self.conv1x1=nn.ModuleList([nn.Conv2d(5,1,1) for _ in range(self.task_num)])
        
        # self.sigmoid=nn.Sigmoid()
        # self.final = nn.Conv2d(4, num_classes, kernel_size=1)

    
    def forward(self, x):
        conv1 = self.conv1(x)#x/2
        conv2 = self.conv2(conv1)
        conv3 = self.conv3(conv2)
        conv4 = self.conv4(conv3)
        conv5 = self.conv5(conv4)#x/32
        
        feature1 = self.supervision1(conv1)
        feature2 = self.supervision2(conv2)
        feature3 = self.supervision3(conv3)
        feature4 = self.supervision4(conv4)
        feature5 = self.supervision5(conv5)
#         print(f'feature1:{feature1.shape}')
        
        cw = []
        for i in range(self.task_num):
            cw.append(torch.cat((feature1[:,i:i+1],feature2[:,i:i+1],feature3[:,i:i+1],feature4[:,i:i+1],feature5[:,i:i+1]),dim=1))
        
        for i,(c,m) in enumerate(zip(cw,self.conv1x1)):
            cw[i] = m(c)
            
        out = torch.cat((cw), dim=1)
        
        return feature1, feature2, feature3, feature4, feature5, out

print('lseg.py')

lseg.py


In [22]:
import torch
import os
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt


class IDRiDDataset(Dataset):
    def __init__(self, path, dataset_type='train', resolution=[512,700], leisions=['MA','HE','EX','SE'], transform=None, args=None):
        super(IDRiDDataset, self).__init__()
        self.task_dicts = {'MA':'1. Microaneurysms','HE':'2. Haemorrhages','EX':'3. Hard Exudates','SE':'4. Soft Exudates','VES':'5. Vessel/640'}
        self.tasks=self.get_tasks(leisions)
        self.label2color = {'MA':[0,0,255],'HE':[0,255,0],'EX':[255,0,0],'SE':[255,0,255]}

        self.preprocess=args.preprocess
        self.resolution=resolution
        self.dual=args.dual
        self.args=args
        
        
        if self.preprocess:
            preprocess_path = os.path.join(path, 'clahe', '2_8')
            if dataset_type == 'train':
                self.image_root=os.path.join(preprocess_path, 'trn', 'image')
                self.mask_root=os.path.join(preprocess_path, 'trn', 'GT')
            elif dataset_type == 'val':
                self.image_root=os.path.join(preprocess_path, 'val', 'image')
                self.mask_root=os.path.join(preprocess_path, 'val', 'GT')
            else:
                raise EnvironmentError('You should put a valid mode to generate the dataset')
        else:
            if dataset_type == 'train':
                self.image_root=os.path.join(path,'1. Original Images','a. Training Set')
                self.mask_root=os.path.join(path,'2. All Segmentation Groundtruths','a. Training Set')
            elif dataset_type == 'val':
                self.image_root=os.path.join(path,'1. Original Images', 'b. Testing Set')
                self.mask_root=os.path.join(path,'2. All Segmentation Groundtruths','b. Testing Set')
            else:
                raise EnvironmentError('You should put a valid mode to generate the dataset')
        self.image_names=os.listdir(self.image_root)
        self.dataset_type=dataset_type
        self.transform = transform
        print('{} dataset contains {} images at {}'.format(self.dataset_type, len(self.image_names), self.image_root))
        print(self.image_names)

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        'Generate one batch of data'
        sample = self.load(idx)
        
        if self.args.inference:
            image_name=self.image_names[idx].split('.')[0]
            return image_name, sample['image'], sample['masks']
        if self.args.dual:
            return sample['image'], sample['masks']
        return sample['image'], sample['masks'][0]
    
    def get_tasks(self, leisions):
        tasks={}
        for k,v in self.task_dicts.items():
            if k in leisions:
                tasks.update({k:v})
        return tasks

    def eliminate_overlap(self,ms,m):
        # print('masks : {}  m : {}'.format(ms.shape, m.shape))
        _m=m>=1
        _ms=ms>=1
        inter=np.logical_and(_ms,_m)
        # print('inter:{}'.format(inter.reshape(-1).tolist().count(True)))
        m=np.where(inter,0,m)
        return m
    
    def read_mask(self,path):
        mask=Image.open(path)
        mask=np.array(mask)
        #这里不用归一化，因为掩码值已经在0-1内
        mask = cv2.resize(mask, self.resolution)
        mask=mask.astype(np.int32)#这里必须是long，否则传入损失函数时会报错
#         print('read mask shape:{}'.format(mask.shape))
        return mask
    
    def read_image(self,path):
        image = Image.open(path)
        image = np.array(image)
        image = cv2.resize(image, self.resolution)
        image = np.transpose(image, (2,0,1))#(512, 512, 3)->(3, 512, 512)
        image = image/255.0 #归一化 (3, 512, 512)
        image = image.astype(np.float32)
#         print('read image np shape:{}'.format(image.shape))
        image = torch.from_numpy(image)
        return image
        
    def load(self, idx):
        # Get masks from a particular idx
        image_name=self.image_names[idx]
        image_path = os.path.join(self.image_root, image_name)
        image = self.read_image(image_path)
        
        idx=image_name.split('_')[1][:2]

        mask=np.zeros(image.shape[1:3],dtype=np.int32)
#         out_str='\n'
        for i,task in enumerate(self.tasks.items()):
            suffix = '.tif'
            mask_name = 'IDRiD_'+ idx + '_' + task[0] + suffix  # if idx = 0. we look for the image 1
            mask_path = os.path.join(self.mask_root, task[1], mask_name)
            if os.path.exists(mask_path):
                m = self.read_mask(mask_path)*(i+1)
                if len(m.shape)==3:
                    m=np.transpose(m,(2,0,1))
                    m=m[0]/765
                    m=m.astype(np.int32)
                m = self.eliminate_overlap(mask,m)
                mask+=m
#                 out_str+='{} - {} - {} mask : {} | masks : {}\n'.format(image_name, mask_name, task[0], np.unique(m), np.unique(mask))
        
        if self.dual:
            suffix='.tif'
            ves_mask_name = 'IDRiD_'+ idx + '_VES' + suffix
            ves_mask_path = os.path.join(self.mask_root, self.task_dicts['VES'], ves_mask_name)
            ves_mask = self.read_mask(ves_mask_path)
            sample = {'image':image, 'masks':[torch.from_numpy(mask),torch.from_numpy(ves_mask)]}
        else:   
            sample = {'image':image , 'masks': [torch.from_numpy(mask)]}
        # If transform apply transformation
        if self.transform:
            sample = self.transform(sample)
        return sample
    
class DDRDataset(Dataset):
    def __init__(self, path, dataset_type='train', resolution=[512,700], leisions=['MA','HE','EX','SE'], transform=None, args=None):
        super(DDRDataset, self).__init__()
        self.task_dicts = {'MA':'MA','HE':'HE','EX':'EX','SE':'SE'}
        self.tasks=self.get_tasks(leisions)
        self.label2color = {'MA':[0,0,255],'HE':[0,255,0],'EX':[255,0,0],'SE':[255,0,255]}

        self.preprocess=args.preprocess
        self.resolution=resolution
        self.dual=args.dual
        self.args=args

        if self.preprocess:
            if dataset_type == 'train':
                self.image_root=os.path.join(path, 'trn', 'image')
                self.mask_root=os.path.join(path, 'trn', 'GT')
            elif dataset_type == 'val':
                self.image_root=os.path.join(path, 'val', 'image')
                self.mask_root=os.path.join(path, 'val', 'GT')
            elif dataset_type == 'test':
                self.image_root=os.path.join(path, 'tst', 'image')
                self.mask_root=os.path.join(path, 'tst', 'GT')
                
            else:
                raise EnvironmentError('You should put a valid mode to generate the dataset')
        else:
            if dataset_type == 'train':
                self.image_root=os.path.join(path, 'train', 'image')
                self.mask_root=os.path.join(path, 'train', 'label')
            elif dataset_type == 'val':
                self.image_root=os.path.join(path, 'valid', 'image')
                self.mask_root=os.path.join(path, 'valid', 'segmentation label')
            elif dataset_type == 'test':
                self.image_root=os.path.join(path, 'test', 'image')
                self.mask_root=os.path.join(path, 'test', 'label')
            else:
                raise EnvironmentError('You should put a valid mode to generate the dataset')
        
            
        
        self.image_names=os.listdir(self.image_root)
        self.dataset_type=dataset_type
        self.transform = transform
        print('{} dataset contains {} images at {}'.format(self.dataset_type, len(self.image_names), self.image_root))
        
    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        'Generate one batch of data'
        sample = self.load(idx)
        
        if self.args.inference:
            image_name=self.image_names[idx].split('.')[0]
            return image_name, sample['image'], sample['masks']
        if self.args.dual:
            return sample['image'], sample['masks']
        return sample['image'], sample['masks'][0]
    
    def get_tasks(self, leisions):
        tasks={}
        for k,v in self.task_dicts.items():
            if k in leisions:
                tasks.update({k:v})
        return tasks

    def eliminate_overlap(self,ms,m):
        # print('masks : {}  m : {}'.format(ms.shape, m.shape))
        _m=m>=1
        _ms=ms>=1
        inter=np.logical_and(_ms,_m)
        # print('inter:{}'.format(inter.reshape(-1).tolist().count(True)))
        m=np.where(inter,0,m)
        return m
    
    def read_mask(self,path):
        mask=Image.open(path)
        mask=np.array(mask)
        #这里不用归一化，因为掩码值已经在0-1内
        mask = cv2.resize(mask, self.resolution)
        if not self.preprocess:
            mask = mask/255.0
        mask=mask.astype(np.int32)#这里必须是long，否则传入损失函数时会报错
#         print('read mask shape:{}'.format(mask.shape))
        return mask
    
    def read_image(self,path):
        image = Image.open(path)
        image = np.array(image)
        image = cv2.resize(image, self.resolution)
        image = np.transpose(image, (2,0,1))#(512, 512, 3)->(3, 512, 512)
        image = image/255.0 #归一化 (3, 512, 512)
        image = image.astype(np.float32)
#         print('read image np shape:{}'.format(image.shape))
        image = torch.from_numpy(image)
        return image
        
    def load(self, idx):
        # Get masks from a particular idx
        image_name=self.image_names[idx]
        image_path = os.path.join(self.image_root, image_name)
        image = self.read_image(image_path)
        
        idx=image_name.split('.')[0]
        
        mask=np.zeros(image.shape[1:3],dtype=np.int32)
#         out_str='\n'
        for i,task in enumerate(self.tasks.items()):
            suffix = '.tif'
            mask_name = idx + suffix  # if idx = 0. we look for the image 1
            mask_path = os.path.join(self.mask_root, task[1], mask_name)
            if os.path.exists(mask_path):
                m = self.read_mask(mask_path)*(i+1)
                if len(m.shape)==3:
                    m=np.transpose(m,(2,0,1))
                    m=m[0]/765
                    m=m.astype(np.int32)
                m = self.eliminate_overlap(mask,m)
                mask+=m
#                 out_str+='{} - {} - {} mask : {} | masks : {}\n'.format(image_name, mask_name, task[0], np.unique(m), np.unique(mask))
        
        sample = {'image':image , 'masks': [torch.from_numpy(mask)]}
        # If transform apply transformation
        if self.transform:
            sample = self.transform(sample)
        return sample
    
class VesselDataset(Dataset):
    def __init__(self, path, dataset_type='train', resolution=[512,700], transform=None, args=None):
        super(VesselDataset, self).__init__()
        self.preprocess=args.preprocess
        self.resolution=resolution
        self.args=args
        self.tasks={'VES':'Vessel'}
        
        if self.preprocess:
            preprocess_path = os.path.join(path, 'clahe', '2_8')
            if dataset_type == 'train':
                self.image_root=os.path.join(preprocess_path, 'trn', 'image')
                self.mask_root=os.path.join(preprocess_path, 'trn', 'GT')
            elif dataset_type == 'val':
                self.image_root=os.path.join(preprocess_path, 'val', 'image')
                self.mask_root=os.path.join(preprocess_path, 'val', 'GT')
            else:
                raise EnvironmentError('You should put a valid mode to generate the dataset')
        else:
            if dataset_type == 'train':
                self.image_root=os.path.join(path,'Training','Images')
                self.mask_root=os.path.join(path,'Training','Masks')
            elif dataset_type == 'val':
                self.image_root=os.path.join(path,'Test','Images')
                self.mask_root=os.path.join(path,'Test','Masks')
            else:
                raise EnvironmentError('You should put a valid mode to generate the dataset')
        self.image_names=os.listdir(self.image_root)
        self.dataset_type=dataset_type
        self.transform = transform
        print('{} dataset contains {} images'.format(self.dataset_type, len(self.image_names)))
        print(self.image_names)
        

    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, idx):
        'Generate one batch of data'
        sample = self.load(idx)
        return sample['image'], sample['mask']

    def read_mask(self,path):
        mask=Image.open(path)
        mask=np.array(mask)
        mask=mask/np.max(mask)
        mask = cv2.resize(mask, self.resolution)
        mask=mask.astype(np.int32)#这里必须是long，否则传入损失函数时会报错
#         print('read mask shape:{}'.format(mask.shape))
        return mask
    
    def read_image(self,path):
        image = Image.open(path)
        image = np.array(image)
        image = cv2.resize(image, self.resolution)
        image = np.transpose(image, (2,0,1))#(512, 512, 3)->(3, 512, 512)
        image = image/255.0 #归一化 (3, 512, 512)
        image = image.astype(np.float32)
#         print('read image np shape:{}'.format(image.shape))
        image = torch.from_numpy(image)
        return image
        
    def load(self, idx):
        # Get masks from a particular idx
        image_name=self.image_names[idx]
        image_path = os.path.join(self.image_root, image_name)
        image = self.read_image(image_path)
        
        name=image_name.split('.')[0]

        if name.split('_')[-1] == 'HRF':
            suffix = '.tif'
        elif name.split('_')[-1] == 'DRIVE':
            suffix = '.gif'
        elif name.split('_')[-1] == 'CHASE':
            suffix = '.png'
        mask_name = name + suffix  # if idx = 0. we look for the image 1
        mask_path = os.path.join(self.mask_root, mask_name)
        if os.path.exists(mask_path):
            mask = self.read_mask(mask_path)
#                 out_str+='{} - {} - {} mask : {} | masks : {}\n'.format(image_name, mask_name, task[0], np.unique(m), np.unique(mask))
        
#         print(out_str)
        sample = {'image':image , 'mask': torch.from_numpy(mask)}
        # If transform apply transformation
        if self.transform:
            sample = self.transform(sample)
        return sample
        
print('dataset.py')

dataset.py


In [23]:
import os
import numpy as np
from PIL import Image
import cv2

import torch
from torch.utils.data import Dataset
import csv
import itertools
from torch.utils.data.sampler import Sampler
from torch.utils.data.dataloader import DataLoader
import torchvision.transforms as transforms

# from utils import *
# from contrast import ImageContraster

NO_LABEL=-1
# img_cter=ImageContraster()

class ImageDataset(Dataset):
    def __init__(self):
        super(ImageDataset, self).__init__()
    
    def read_label(self, path):
        with open(path,'r') as f:
            if isinstance(self.num_classes, list):
                return np.array(list(csv.reader(f,delimiter=',')))[1:,:len(self.num_classes)+1]
            else:
                return np.array(list(csv.reader(f,delimiter=',')))[1:,:2]
    
    def read_image(self,path):
        size=self.resolution#(W,H)
        image = cv2.imread(path, cv2.IMREAD_COLOR)
        # image = image[:,:,::-1] #将BGR变为RGB
        image = cv2.resize(image,size)
        image = np.transpose(image, (2, 0, 1))  ## (3, 512, 512)
        #这里是为了兼容不使用transform的情况
        image = image/255.0 #归一化 (3, 512, 512)
        image = image.astype(np.float32)
        image = torch.from_numpy(image)
        return image
    
    def toImage(self,img):
        img=img.numpy()
        img*=255
        img=np.transpose(img,(1,2,0))#传入Image.fromarray的图片形状必须为（H,W,C），否则会报错
        img=np.uint8(img)#传入Image.fromarray的图片必须是uint8格式，否则报错
        img = Image.fromarray(img)
        return img
    
    def get_class_weight(self, num_classes):
        class_weight=[]
        if isinstance(num_classes, list):
            for i,n in enumerate(num_classes):
                cw=[]
                l=self.label[:,i]
                total = len(l)
                for i in range(n):
                    t=total/len(l[l==i])
                    cw.append(t/n)
                class_weight.append(torch.tensor(cw))
            return class_weight
        total=len(self.label)
        for i in range(num_classes):
            t=total/len(self.label[self.label==i])
            class_weight.append(t/num_classes)
        return torch.tensor(class_weight)
    
    def load_data_from_dir(self):
        label=[]
        image_names=[]
        for i in range(self.num_classes):
            names=os.listdir(os.path.join(self.image_root,str(i)))
            image_names+=names
            label+=[i for _ in range(len(names))]
        l=np.array([image_names,label])
        l=l.transpose()
        np.random.shuffle(l)
        print(l.shape)
        return l
    
    def balance_dataset(self, label,num_classes,factor=1.5):
        l=label[:,1].astype(np.int32)
        l_list=l.tolist()
        cnt=[l_list.count(i) for i in range(num_classes)]
        min_num=min(cnt)
        max_num=int(min_num*factor)
        cnt=np.array(cnt).clip(min_num,max_num)
    
        balanced_l=[]
        for i in range(num_classes):
            t=label[l==i]
            if i == 0:
                balanced_l=t[:cnt[i]]
            else:
                balanced_l=np.vstack((balanced_l,t[:cnt[i]]))
    
        print(balanced_l.shape)
        return balanced_l

class IDataset(ImageDataset):
    def __init__(self, path, dataset_type='train', resolution=512, args=None):
        super(IDataset, self).__init__()
        self.args=args
        self.resolution=resolution
        self.dataset_type=dataset_type
        self.num_classes=args.gan_grading_classes
        self.transform=None
        if dataset_type == 'train':
            self.image_root=os.path.join(path,'1. Original Images','a. Training Set')
            self.label=self.read_label(os.path.join(path,'2. Groundtruths','a. IDRiD_Disease Grading_Training Labels.csv'))
        elif dataset_type == 'val':
            self.image_root=os.path.join(path,'1. Original Images', 'b. Testing Set')
            self.label=self.read_label(os.path.join(path,'2. Groundtruths','b. IDRiD_Disease Grading_Testing Labels.csv'))
        else:
            self.trn_image_root=os.path.join(path,'1. Original Images','a. Training Set')
            trn_label=self.read_label(os.path.join(path,'2. Groundtruths','a. IDRiD_Disease Grading_Training Labels.csv'))
            self.tst_image_root=os.path.join(path,'1. Original Images', 'b. Testing Set')
            tst_label=self.read_label(os.path.join(path,'2. Groundtruths','b. IDRiD_Disease Grading_Testing Labels.csv'))
            self.trn_length=len(trn_label)
            self.label = np.vstack((trn_label,tst_label))

        self.image_names=self.label[:,0]
        self.label=self.label[:,1:].astype(np.int32).squeeze()
        self.count_result=count_class(self.label, self.num_classes)
        self.class_weight=self.get_class_weight(self.num_classes)
        self.dataset_len=len(self.image_names)
        if dataset_type != 'all': print(f'{self.image_root} contains {self.dataset_len} samples.')
        print(self.count_result)

    def __getitem__(self, idx):
        # print(f'idx:{idx}')
        if self.dataset_type == 'all':
            if idx < self.trn_length:
                self.image_root = self.trn_image_root
            else:
                self.image_root = self.tst_image_root
                
        image_path=os.path.join(self.image_root,self.image_names[idx]+'.jpg')
        img=self.read_image(image_path)
#         print(img.shape)
        if self.transform is not None:
            img=self.toImage(img)
            img = self.transform(img)
        
        if isinstance(self.num_classes, list):
            temp=[img]
            for l in self.label[idx]:
                temp.append(int(l))
            return temp
        
        return img,int(self.label[idx])

    def __len__(self):
        return self.dataset_len

class GradingDDRDataset(ImageDataset):
    def __init__(self, path, label, dataset_type='train', resolution=512, args=None, binary=False):
        super(GradingDDRDataset, self).__init__()
        self.args=args
        self.binary=binary
        self.resolution=resolution
        self.dataset_type=dataset_type
        self.image_root=path
        self.num_classes=args.gan_grading_classes
        self.label=label
        self.image_names=self.label[:,0]
        self.label=self.label[:,1].astype(np.int32).squeeze()
        self.count_result=count_class(self.label, self.num_classes)
        self.class_weight=self.get_class_weight(self.num_classes)
        
        self.dataset_len=len(self.label)
        self.transform=None
        print(f'{self.image_root}/{self.dataset_type} contains {self.dataset_len} samples.')
        print(self.count_result)
        self.need_suffix=False
        if len(self.image_names[0].split('.')) == 1:
            self.need_suffix=True

    def __getitem__(self, idx):
#         print(f'idx:{idx}, image_names:{self.image_names[idx]}, label:{self.label[idx]}\n')
        image_path=os.path.join(self.image_root,self.image_names[idx])
        if self.need_suffix:
            image_path=image_path+'.jpeg'
#         print(image_path)
        img=self.read_image(image_path)
        if self.transform is not None:
            img=self.toImage(img)
            img = self.transform(img)
            
        if self.binary:
            if self.label[idx] <= 1:
                temp_l=0
            else:
                temp_l=1
            return img,int(temp_l)
        return img,int(self.label[idx])

    def __len__(self):
        return self.dataset_len
    
class CamDataset(ImageDataset):
    def __init__(self, path, label, dataset_type='train', resolution=512, args=None, binary=False):
        super(CamDataset, self).__init__()
        self.args=args
        self.binary=binary
        self.resolution=resolution
        self.dataset_type=dataset_type
        self.image_root=path
        self.num_classes=args.gan_grading_classes
        self.label=label
        self.image_names=self.label[:,0]
        self.label=self.label[:,1].astype(np.int32).squeeze()
        self.count_result=count_class(self.label, self.num_classes)
        self.class_weight=self.get_class_weight(self.num_classes)
        
        self.dataset_len=len(self.label)
        self.transform=None
        print(f'{self.image_root}/{self.dataset_type} contains {self.dataset_len} samples.')
        print(self.count_result)
        self.need_suffix=False
        if len(self.image_names[0].split('.')) == 1:
            self.need_suffix=True
    
    def read_image(self,path):
        size=np.array([1,1])*self.resolution#(W,H)
        image = cv2.imread(path, cv2.IMREAD_COLOR)
        # image = image[:,:,::-1]
#         image = cv2.resize(image,size)
        image = np.transpose(image, (2, 0, 1))  ## (3, 512, 512)
        image = image/255.0 ## (512, 512, 3)
        image = image.astype(np.float32)
        image = torch.from_numpy(image)
        return image

    def __getitem__(self, idx):
#         print(f'idx:{idx}, image_names:{self.image_names[idx]}, label:{self.label[idx]}\n')
        image_path=os.path.join(self.image_root,self.image_names[idx])
        if self.need_suffix:
            image_path=image_path+'.jpeg'
#         print(image_path)
        img=self.read_image(image_path)
        if self.transform is not None:
            img=self.toImage(img)
            img = self.transform(img)
            
        if self.binary:
            if self.label[idx] <= 1:
                temp_l=0
            else:
                temp_l=1
            return img,int(temp_l)
        return self.image_names[idx],img,int(self.label[idx])

    def __len__(self):
        return self.dataset_len
    
class EyepacDataset(ImageDataset):
    def __init__(self, path, dataset_type='train', resolution=512, args=None):
        super(EyepacDataset, self).__init__()
        self.args=args
        self.dataset_type=dataset_type
        self.num_classes=args.gan_grading_classes
        self.resolution = resolution
        
        self.image_root = os.path.join(path, dataset_type)
        self.label = self.load_data_from_dir()
        self.image_names=self.label[:,0]
        self.label=self.label[:,1].astype(np.int32).squeeze()
        self.count_result=count_class(self.label, self.num_classes)
        self.class_weight=self.get_class_weight(self.num_classes)
        
        self.dataset_len=len(self.label)
        self.transform=None
        print(f'{self.image_root}/{self.dataset_type} contains {self.dataset_len} samples.')
        print(self.count_result)
        
        
    def __getitem__(self, idx):
#         print(f'idx:{idx}, image_names:{self.image_names[idx]}, label:{self.label[idx]}\n')
        image_path=os.path.join(self.image_root,str(self.label[idx]),self.image_names[idx])
#         print(image_path)
        img=self.read_image(image_path)
        if self.transform is not None:
            img=self.toImage(img)
            img = self.transform(img)
            
        return img,int(self.label[idx])

    def __len__(self):
        return self.dataset_len

print('grading_dataset.py')

grading_dataset.py


In [24]:
import torch
import torch.nn as nn
from torch.optim import *

import wandb
from torch.autograd import Variable
import os
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import logger
from torch.utils.data import dataloader
import time


# from utils import utils
# from utils import metrics
# from loss import *

class SLTrainer:
    def __init__(self, model, optimizer, criterion, device, config):
        print("Supervised trainer")
        self.ssl_name='SL'
        self.config=config
        self.model=model
        self.optimizer=optimizer
        self.device=device
        self.criterion=criterion
        self.estimator=Estimator(config.metrics, config.num_classes)
        self.id=config.id
        
        if self.config.dual and self.config.seg_vessel:
            self.criterion_extra=nn.BCEWithLogitsLoss(pos_weight=torch.tensor(10).to(self.device))
        
        self.data_dir=os.path.join(config.checkpoint_dir,'Supervised', self.id)
        self.log_dir=os.path.join(self.data_dir,'log')
        self.cpt_dir=os.path.join(self.data_dir,'checkpoint')
        self.best_checkpoint=''
        self.epoch       = 0
        self.start_epoch = 0
        self.last_epoch = -1
        mkdir(self.data_dir)
        mkdir(self.log_dir)
        mkdir(self.cpt_dir)
        
    def trn_iteration(self, data_loader, scheduler):
        self.model.train()
        self.estimator.reset()
        trn_loss = AverageMeter()
        
        lr=self.optimizer.state_dict()['param_groups'][0]['lr']
        with tqdm(total=len(data_loader), desc=f"Epoch {self.epoch}/{self.config.num_epochs} --- training lr:{lr}", unit='batch') as pbar:
            for step, (x, y) in enumerate(data_loader):
                if self.config.dual:
                    x = x.to(self.device)
                    x = Variable(x)
                    y = [_y.to(self.device) for _y in y]
                    y_extra = Variable(y[1]).float()
                    y = Variable(y[0]).float()
                    if self.config.seg_vessel:
                        y_pred = self.model(x.float())
                        y_extra_pred = y_pred[1].squeeze()
                        y_pred = y_pred[0]
                    else:
                        y_pred = self.model(x.float(), y_extra)
                else:
                    x, y = [t.to(self.device) for t in (x,y)]
                    x, y=Variable(x),Variable(y)
                    y_pred = self.model(x.float())
                    
                loss = self.criterion(y_pred, y.long())
                if self.config.dual and self.config.seg_vessel:
                    loss_extra = self.criterion_extra(y_extra_pred, y_extra)
                    loss += self.config.dual_loss_factor*loss_extra
                
                if self.config.accumulation_step:
                    loss /= self.config.accumulation_step
                    loss.backward()
                    
                    if ((step + 1) % self.config.accumulation_step == 0) or (step + 1 == len(data_loader)):
                        trn_loss.update(loss.item()*self.config.accumulation_step)
                        self.optimizer.step()
                        self.optimizer.zero_grad()
                else: 
                    trn_loss.update(loss.item())
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()
                if not self.config.upd_by_ep and scheduler != None: scheduler.step()
                self.estimator.update(y_pred, y)
                pbar.update(1)
        
        score=self.estimator.get_scores(4)

        for m in set(self.estimator.need_named_metrics)&set(score.keys()):
            val=score.pop(m)
            score.update(self.estimator.name_val(data_loader.dataset.tasks, m, val))
        
        info_dict=dict([('epoch', self.epoch),
                   ('lr', lr),
                   ('trn_loss', round(trn_loss.avg,4))])
        
        info_dict.update({'trn_'+k:v for k,v in score.items()})
        # if self.config.dual:
        #     k=self.config.dual
        #     score=self.estimator_extra.get_scores(4)
        #     if self.config.show_log:
        #         print(f'trn_gt_{k}:{self.estimator_extra.count("gt")}')
        #         print(f'trn_pred_{k}:{self.estimator_extra.count("pred")}')
        #     info_dict.update({'trn_'+_k+'_'+k:_v for _k,_v in score.items()})
        return info_dict
        
    def val_iteration(self, data_loader, val_loss):
        self.model.eval()
        
        self.estimator.reset()
        
        with tqdm(total=len(data_loader), desc=f"Epoch {self.epoch}/{self.config.num_epochs} --- validating", unit='batch') as pbar:
            with torch.no_grad():
                for step, (x, y) in enumerate(data_loader):
                    if self.config.dual:
                        x = x.to(self.device)
                        x = Variable(x)
                        y = [_y.to(self.device) for _y in y]
                        y_extra = Variable(y[1]).float()
                        y = Variable(y[0]).float()
                        if self.config.seg_vessel:
                            y_pred = self.model(x.float())
                            y_extra_pred = y_pred[1].squeeze()
                            y_pred = y_pred[0]
                        else:
                            y_pred = self.model(x.float(), y_extra)
                    
                    elif self.config.grid_size:
                        x, y = [t.to(self.device) for t in (x,y)]
                        x, y=Variable(x),Variable(y)
                        x = x.float()
                        x = x.transpose(0,1) # b, p1, p2, c, h, w
                        y = y.transpose(0,1)
                        
                        y_pred_pieces = []
                        for x_piece in x:
                            y_pred_pieces.append(self.model(x_piece.float()))
                    else:
                        x, y = [t.to(self.device) for t in (x,y)]
                        x, y=Variable(x),Variable(y)
                        y_pred = self.model(x.float())

                    
                    if self.config.grid_size:
                        loss = self.criterion(y_pred_pieces[0], y[0].long())
                        for y_pred,y_true in zip(y_pred_pieces, y):
                            self.estimator.update(y_pred, y_true)
                    else:
                        loss=self.criterion(y_pred, y.long())
                        self.estimator.update(y_pred, y)
                    if self.config.dual and self.config.seg_vessel:
                        loss_extra = self.criterion_extra(y_extra_pred, y_extra)
                        loss += self.config.dual_loss_factor*loss_extra
                        
                    if self.config.accumulation_step:
                        loss /= self.config.accumulation_step
                        if ((step + 1) % self.config.accumulation_step == 0) or (step + 1 == len(data_loader)):
                            val_loss.update(loss.item()*self.config.accumulation_step)
                    else: 
                        val_loss.update(loss.item())

                    pbar.update(1) 
                
        score=self.estimator.get_scores(4)
        
        for m in set(self.estimator.need_named_metrics)&set(score.keys()):
            val=score.pop(m)
            score.update(self.estimator.name_val(data_loader.dataset.tasks, m, val))
        
        info_dict=dict([('val_loss', round(val_loss.avg,4))])
        info_dict.update({'val_'+k:v for k,v in score.items()})
        # info_dict.update({f'roc_e{self.epoch}':self.plot_roc(self.estimator.y, self.estimator.y_pred)})
        
        return info_dict
    
    def loop(self, num_epochs, trn_loader, val_loader):
        logger.configure(dir=self.log_dir, log_suffix=self.id)
        
        if self.config.use_wandb:
            wandb.init(
              project=self.config.experiment_name,
              config=vars(self.config)
            )
            wandb.run.name = self.id
            wandb.run.save()
            
        self.load()
            
        scheduler=get_scheduler(self.config, self.optimizer, last_epoch = self.last_epoch)
        
        best_perf = [sys.float_info.min]*2
        val_loss = AverageMeter()
        last_info=None

        if self.config.use_wandb:
            for k,v in self.model.named_children():
                print(f'{k}:{v}')

        timer = Timer()
        
        for epoch in range(self.start_epoch+1, num_epochs+1):
            self.epoch=epoch
            timer.start_timer(time.time())
            info_dict=self.trn_iteration(trn_loader, scheduler)
            timer.compute(time.time())
            if self.config.upd_by_ep and scheduler != None: scheduler.step()
                    
            if epoch % self.config.save_interval == 0:
                self.save(epoch, self.model.state_dict(), self.optimizer.state_dict())
            
            if epoch % self.config.validation_interval == 0 or epoch == 1:
                timer.start_timer(time.time())
                info_dict.update(self.val_iteration(val_loader, val_loss))
                timer.compute(time.time())
                
                if info_dict['val_'+self.config.best_metric] > best_perf[1]:
                    best_perf[0]=epoch
                    best_perf[1]=info_dict['val_'+self.config.best_metric]
                    self.best_checkpoint = [os.path.join(self.cpt_dir, self.id+f'_e{epoch}.pt')]
                    if epoch != num_epochs:
                        self.best_checkpoint.append(os.path.join(self.cpt_dir, self.id+f'_e{num_epochs}.pt'))
                    if epoch % self.config.save_interval != 0 and info_dict['val_'+self.config.best_metric] > self.config.good_value:
                        self.save(epoch, self.model.state_dict(), self.optimizer.state_dict())
                    
                if epoch == num_epochs:
                    last_info= {self.config.best_metric:info_dict['val_'+self.config.best_metric],'best_cpt':self.best_checkpoint}
                    
            if self.config.use_wandb:
                    wandb.log(info_dict)

            if self.config.show_log:
                logger.logkvs(info_dict)
                logger.dumpkvs()
                str_output = 'epoch & {} \n'.format(self.config.best_metric)
                str_output += f'{best_perf[0]} & {best_perf[1]:.4f} \n'
                logger.log(str_output)
            
            if val_loss.is_overfitting() and self.config.early_stop:
                print('Detected overfitting signs, stop training.')
                break
            
            val_loss.reset()
                
        if self.config.use_wandb:
            wandb.finish()
            
        return last_info
    
    def evaluate(self, val_loader):
        print('-------------------EVALUATING-------------------')
        
        logger.configure(dir=self.log_dir, log_suffix=self.id)

        cpt_name=os.path.split(self.config.resume_checkpoint)[-1].split('.')[0]
        account=cpt_name.split('_')[-3]
        version=cpt_name.split('_')[-2]
        
        if self.config.use_wandb:
            wandb.init(
              project=self.config.experiment_name,
              config=vars(self.config)
            )
            wandb.run.name = f'test_{self.config.dataset_name}_by_{account}_{version}'
            wandb.run.save()
        
        self.load()
        
        val_loss = AverageMeter()
        self.epoch=1
        info_dict=self.val_iteration(val_loader, val_loss)
        
        if self.config.use_wandb:
            wandb.log(info_dict)
        for k,v in info_dict.items():
            print(f'{k}:{v}')
        # logger.logkvs(info_dict)
        # logger.dumpkvs()
        
    def inference(self, data_loader, type):
        print('-------------------INFERENCE-------------------')
        logger.configure(dir=self.log_dir, log_suffix=self.id)
        
        self.load()
        
        self.model.eval()
        
        result_dir=os.path.join(self.config.inference_root_dir, self.config.dataset_name, type)
        mkdir(result_dir)
        
        with tqdm(total=len(data_loader), desc=f"Inference", unit='batch') as pbar:
            with torch.no_grad():
                for step, (n, x, y) in enumerate(data_loader):
                    x, y = x.to(self.device), y.to(self.device)
                    x, y = Variable(x), Variable(y)
    
                    y_pred = self.model(x.float())
                    logits = y_pred.detach().cpu()
                    x_np = x.detach().cpu().numpy()
                    
                    logits = F.softmax(logits, dim=1)
                    predict = np.argmax(logits.numpy(), axis=1).astype(np.uint8)
                    
                    for i,p in enumerate(predict):
                        img = Image.fromarray(p)
                        result_path=os.path.join(result_dir, n[i]+'_'+self.config.inference_suffix+'.tif')
                        print(result_path)
                        img.save(result_path, format='TIFF')
                        
                        o_img=np.transpose(x_np[i],(1,2,0))
                        imgs=[o_img, img]
                        titles=[n[i], 'vessel_mask']
                        fig,axs = plt.subplots(1,len(imgs),figsize=(15,30),sharey=True)
                        for i,ax in enumerate(axs):
                            ax.imshow(imgs[i])
                            ax.set_title(titles[i])
                            ax.axis('off')
                        plt.show()
                    
                    pbar.update(1) 
        
    def plot_roc(self, y, y_pred):
        fpr, tpr, thresholds = roc_curve(y, y_pred)
        roc_auc = auc(fpr, tpr)
        fig=plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'Receiver Operating Characteristic e{self.epoch}')
        plt.legend(loc="lower right")
        import matplotlib_inline
        matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
        plt.show()
        plt.save(f'/kaggle/working/roc_{self.epoch}.svg')
        plt.close()
        return fig
    
    def plot_cm(self,cm,name):
        import seaborn as sns
        plt.figure()
        sns.heatmap(cm, annot=True, cmap="Blues", cbar=False, fmt="2d")

        # 设置标签和标题
        plt.xlabel('Predict')
        plt.ylabel("Ground Truth")
        plt.gca().xaxis.tick_top()
        plt.gca().xaxis.set_label_position('top')
        # plt.title()
#         import matplotlib_inline
#         matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
        plt.savefig(os.path.join(self.data_dir,f'{name}_cm_e{self.epoch}.svg'), format='svg')
#         # 显示图像
#         plt.show()
        plt.close()
        
    def save(self, epoch, model_state, optim_state, name=None):
        if epoch < self.config.skip_epoch:
            return
        mkdir(self.cpt_dir)
        if name != None:
            cpt_name=name
        else: cpt_name=self.id+f'_e{epoch}.pt'
        save_dict={'epoch': epoch, 'args':self.config, 'model_state_dict': model_state}
        if self.config.save_optim:
            save_dict.update({'optim_state_dict': optim_state})
        torch.save(save_dict, os.path.join(self.cpt_dir,cpt_name))
        print(f'model saved in {self.cpt_dir}/{cpt_name}')
        
    def load(self):
        if self.config.resume_checkpoint:
            if os.path.exists(self.config.resume_checkpoint):
                _, cpt_name=os.path.split(self.config.resume_checkpoint)
                if '.pt' in cpt_name:
                    print(f'loading file at {self.config.resume_checkpoint}')
                    cpt_dict = torch.load(self.config.resume_checkpoint, map_location=self.device)
                else:
                    print('please provide a file path with suffix of ".pt"')
                    return
            else:
                print(f'{self.config.resume_checkpoint} does not exists!')
                return
            
            if cpt_dict:
                self.start_epoch=cpt_dict['epoch']
                self.model.load_state_dict(cpt_dict['model_state_dict'])
                if self.config.load_optimizer:
                    op_state=cpt_dict.get('optim_state_dict', -1)
                    if op_state !=-1:
                        self.last_epoch=self.start_epoch-1
                        self.optimizer.load_state_dict(op_state)
                    else:
                        self.last_epoch=-1
                print('Checkpoint has been loaded.')
            else:
                print('Checkpoint has no contents.')
                return

print('sl.py')

sl.py


In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *

import wandb
from torch.autograd import Variable
import os
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt

import logger
from torch.utils.data import dataloader

# from utils import utils
# from utils import metrics
# from loss import *

class MTSLTrainer(SLTrainer):
    def __init__(self, model, optimizer, criterion, device, config):
        print("Multi-Task Supervised trainer")
        self.ssl_name='MTSL'
        self.config=config
        self.model=model
        self.optimizer=optimizer
        self.device=device
        self.criterion=criterion
        self.estimator=[Estimator(config.metrics, 2, binary=True, input_logits=config.input_logits) for _ in range(config.num_classes)]
        self.id=config.id

        if self.config.dual and self.config.seg_vessel:
            self.criterion_extra=nn.BCEWithLogitsLoss(pos_weight=torch.tensor(10).to(self.device))
        
        self.data_dir=os.path.join(config.checkpoint_dir,'Multi-Task Supervised trainer', self.id)
        self.log_dir=os.path.join(self.data_dir,'log')
        self.cpt_dir=os.path.join(self.data_dir,'checkpoint')
        self.best_checkpoint=''
        self.epoch       = 0
        self.start_epoch = 0
        self.last_epoch = -1
        mkdir(self.data_dir)
        mkdir(self.log_dir)
        mkdir(self.cpt_dir)
        
    def trn_iteration(self, data_loader, scheduler):
        self.model.train()
        for e in self.estimator:
            e.reset()
        trn_loss = AverageMeter()
        
        lr=self.optimizer.state_dict()['param_groups'][0]['lr']
        with tqdm(total=len(data_loader), desc=f"Epoch {self.epoch}/{self.config.num_epochs} --- training lr:{lr}", unit='batch') as pbar:
            for step, (x, y) in enumerate(data_loader):
                if self.config.dual:
                    x = x.to(self.device)
                    x = Variable(x)
                    y = [_y.to(self.device) for _y in y]
                    y_extra = Variable(y[1]).float()
                    y = Variable(y[0]).float()
                    if self.config.seg_vessel:
                        y_pred = self.model(x.float())
                        y_extra_pred = y_pred[1].squeeze()
                        y_pred = y_pred[0]
                    else:
                        y_pred = self.model(x.float(), y_extra)
                elif self.config.grid_size:
                    x, y = [t.to(self.device) for t in (x,y)]
                    x, y=Variable(x),Variable(y)
                    x = x.float()
                    x = x.transpose(0,1) # b, p1, p2, c, h, w
                    y = y.transpose(0,1)
                    
                    y_pred_pieces = []
                    for x_piece in x:
                        y_pred_pieces.append(self.model(x_piece.float()))
                    x_coord = self.config.resolution[0]//self.config.grid_size
                    y_coord = self.config.resolution[1]//self.config.grid_size
                    y_pred = torch.cat([
                                        torch.cat(y_pred_pieces[i*x_coord:(i+1)*x_coord],dim=3) for i in range(y_coord)
                                        ], dim=2)
                    y_pieces=[]
                    for y_piece in y:
                        y_pieces.append(y_piece)
                    # print(len(y_pieces))
                    y = torch.cat([
                                    torch.cat(y_pieces[i*x_coord:(i+1)*x_coord],dim=2) for i in range(y_coord)
                                        ], dim=1)
                else:
                    x, y = [t.to(self.device) for t in (x,y)]
                    x, y=Variable(x),Variable(y)

                    # print(y.shape)
                    # print(y[0].shape)
                    # print(np.max(y.cpu().detach().numpy()))
                    # plt.figure()
                    # plt.imshow(y[0].cpu().detach().numpy())
                    # plt.show()
                    # plt.close()

                    
                    y_pred = self.model(x.float())

                #--------lseg------------
                if self.config.netframe=='lseg':
                    features=y_pred[:-1]
                    y_pred=y_pred[-1]
                    targets=[]
                    for i in range(1,self.config.num_classes+1):
                        t=(y==i).float()
                        targets.append(t)
                        o=y_pred[:,i-1]
                        self.estimator[i-1].update(o, t)
                    targets=torch.stack(targets,dim=1)
                    loss = self.criterion(y_pred, targets)
                    for f in features:
                        loss += self.criterion(f, targets)
                else:
                    #--------normal----------
                    targets=[]
                    for i in range(1,self.config.num_classes+1):
                        t=(y==i).float()
                        targets.append(t)
                        o=y_pred[:,i-1]
                        # print(f't:{t.shape}')
                        # print(f'o:{o.shape}')
                        self.estimator[i-1].update(o, t)
                    targets=torch.stack(targets,dim=1)
                    loss = self.criterion(y_pred, targets)
                    if self.config.dual and self.config.seg_vessel:
                        loss_extra = self.criterion_extra(y_extra_pred, y_extra)
                        loss += self.config.dual_loss_factor*loss_extra

                
                

                if self.config.accumulation_step:
                    loss /= self.config.accumulation_step
                    loss.backward()
                    
                    if ((step + 1) % self.config.accumulation_step == 0) or (step + 1 == len(data_loader)):
                        trn_loss.update(loss.item()*self.config.accumulation_step)
                        self.optimizer.step()
                        self.optimizer.zero_grad()
                else: 
                    trn_loss.update(loss.item())
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()
                
                if not self.config.upd_by_ep and scheduler != None: scheduler.step()
                pbar.update(1)
        
        info_dict=dict([('epoch', self.epoch),
                   ('lr', lr),
                   ('trn_loss', round(trn_loss.avg,4))])

        for i,k in enumerate(self.config.leisions):
            score=self.estimator[i].get_scores(4)
            info_dict.update({'trn_'+_k+'_'+k:_v for _k,_v in score.items()})
        
        return info_dict
        
    def val_iteration(self, data_loader, val_loss):
        self.model.eval()
        
        for e in self.estimator:
            e.reset()
        
        with tqdm(total=len(data_loader), desc=f"Epoch {self.epoch}/{self.config.num_epochs} --- validating", unit='batch') as pbar:
            with torch.no_grad():
                for step, (x, y) in enumerate(data_loader):
                    if self.config.dual:
                        x = x.to(self.device)
                        x = Variable(x)
                        y = [_y.to(self.device) for _y in y]
                        y_extra = Variable(y[1]).float()
                        y = Variable(y[0]).float()
                        if self.config.seg_vessel:
                            y_pred = self.model(x.float())
                            y_extra_pred = y_pred[1].squeeze()
                            y_pred = y_pred[0]
                        else:
                            y_pred = self.model(x.float(), y_extra)
                    elif self.config.grid_size:
                        x, y = [t.to(self.device) for t in (x,y)]
                        x, y=Variable(x),Variable(y)
                        x = x.float()
                        x = x.transpose(0,1) # b, p1, p2, c, h, w
                        y = y.transpose(0,1)
                        
                        y_pred_pieces = []
                        for x_piece in x:
                            y_pred_pieces.append(self.model(x_piece.float()))
                        
                    else:
                        x, y = [t.to(self.device) for t in (x,y)]
                        x, y=Variable(x),Variable(y)
                        y_pred = self.model(x.float())

                    #--------lseg------------
                    if self.config.netframe=='lseg':
                        features=y_pred[:-1]
                        y_pred=y_pred[-1]
                        targets=[]
                        for i in range(1,self.config.num_classes+1):
                            t=(y==i).float()
                            targets.append(t)
                            o=y_pred[:,i-1]
                            self.estimator[i-1].update(o, t)
                        targets=torch.stack(targets,dim=1)
                        loss = self.criterion(y_pred, targets)
                        for f in features:
                            loss += self.criterion(f, targets)
                    elif self.config.grid_size:
                        for y_pred,y_true in zip(y_pred_pieces, y):
                            targets=[]
                            for i in range(1,self.config.num_classes+1):
                                t=(y_true==i).float()
                                targets.append(t)
                                o=y_pred[:,i-1]
                                self.estimator[i-1].update(o, t)
                            targets=torch.stack(targets,dim=1)
                            loss = self.criterion(y_pred, targets) #这里的loss有错，应该是所有piece的loss加起来
                    else:
                        #--------normal----------
                        targets=[]
                        for i in range(1,self.config.num_classes+1):
                            t=(y==i).float()
                            targets.append(t)
                            o=y_pred[:,i-1]
                            self.estimator[i-1].update(o, t)
                        targets=torch.stack(targets,dim=1)
                        loss = self.criterion(y_pred, targets)
                        if self.config.dual and self.config.seg_vessel:
                            loss_extra = self.criterion_extra(y_extra_pred, y_extra)
                            loss += self.config.dual_loss_factor*loss_extra

                    if self.config.accumulation_step:
                        loss /= self.config.accumulation_step
                        if ((step + 1) % self.config.accumulation_step == 0) or (step + 1 == len(data_loader)):
                            val_loss.update(loss.item()*self.config.accumulation_step)
                    else: 
                        val_loss.update(loss.item())
                    pbar.update(1) 
                
        info_dict=dict([('epoch', self.epoch),
                   ('val_loss', round(val_loss.avg,4))])

        mean_metrics={}
        for n in set(self.estimator[0].need_named_metrics)&set(self.estimator[0].get_scores(4).keys()):
            mean_metrics[n]=[]

        print('start calculating scores')
        for i,k in enumerate(self.config.leisions):
            timer1=Timer()
            timer1.start_timer(time.time())
            score=self.estimator[i].get_scores(4)
            timer1.compute(time.time(),name='calculate score')
    
            for n in set(self.estimator[i].need_named_metrics)&set(score.keys()):
                mean_metrics[n].append(score[n])
        
            info_dict.update({'val_'+_k+'_'+k:_v for _k,_v in score.items()})
            
        for k,v in mean_metrics.items():
            info_dict.update({'val_'+k:np.round(np.mean(v), 4)})
            
        return info_dict
        

print('bsl.py')

bsl.py


In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *

import wandb
from torch.autograd import Variable
import os
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt

import logger
from torch.utils.data import dataloader
from itertools import cycle

# from utils import utils
# from utils import metrics
# from loss import *

class Discriminator(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        channels = [32, 64, 128, 256]
        channels = [in_channels] + channels
        self.convs = nn.ModuleList([
                self.basic_conv(channels[i], channels[i+1])
                for i in range(len(channels)-1)
            ])
        self.fc = nn.Linear(channels[-1], 1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
    def basic_conv(self, in_channels, out_channels):
        return nn.Sequential(
                    nn.Conv2d(in_channels, out_channels, 3, 1),
                    nn.Conv2d(out_channels, out_channels, 3, 1),
                    nn.BatchNorm2d(out_channels),
                    nn.ReLU(inplace=True),
                    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
                )
        
    def forward(self, x):
        for m in self.convs:
            x = m(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return F.sigmoid(x)

class GANTrainer(SLTrainer):
    def __init__(self, seg_model, grading_model, optimizers, criterions, device, config):
        print("GAN trainer")
        self.ssl_name='GAN'
        self.config=config
        self.seg_model=seg_model  
        self.grading_model=grading_model
        self.optimizerS=optimizers[0]
        self.optimizerG=optimizers[1]
        self.device=device
        self.criterionS=criterions[0]
        self.criterionG=criterions[1]
        self.estimatorS=[Estimator(config.metrics, 2, binary=True, input_logits=config.input_logits) for _ in range(config.num_classes)]
        self.estimatorG=Estimator(config.gan_grading_metrics, config.gan_grading_classes, input_logits=config.input_logits)
        self.id=config.id
        
        
        self.discriminator=Discriminator(config.num_classes).to(self.device)
        self.optimizerD=torch.optim.SGD(self.discriminator.parameters(),lr=1e-3, momentum=0.9, weight_decay=0.0005)
        self.criterionD=torch.nn.BCELoss()
        
        self.data_dir=os.path.join(config.checkpoint_dir,'GAN trainer', self.id)
        self.log_dir=os.path.join(self.data_dir,'log')
        self.cpt_dir=os.path.join(self.data_dir,'checkpoint')
        self.best_checkpoint=''
        self.grading_best_checkpoint=''
        self.epoch       = 0
        self.start_epoch = 0
        self.last_epoch = -1
        mkdir(self.data_dir)
        mkdir(self.log_dir)
        mkdir(self.cpt_dir)
        
    def trn_iteration(self, seg_loader, grading_loader, scheduler):
        self.seg_model.train()
        self.grading_model.train()
        self.discriminator.train()
        
        for e in self.estimatorS:
            e.reset()
        self.estimatorG.reset()
        
        trn_seg_loss = AverageMeter()
        trn_grading_loss = AverageMeter()
        trn_dis_loss = AverageMeter()
        
        seg_loader=iter(cycle(seg_loader))
        
        lr=self.optimizerS.state_dict()['param_groups'][0]['lr']
        grading_lr=self.optimizerG.state_dict()['param_groups'][0]['lr']
        with tqdm(total=len(grading_loader), desc=f"Epoch {self.epoch}/{self.config.num_epochs} --- training seg_lr:{lr} grading_lr:{grading_lr}", unit='batch') as pbar:
            for step, (x, y) in enumerate(grading_loader):
                
                x, y = [t.to(self.device) for t in (x,y)]
                x, y=Variable(x),Variable(y)
                
                s_x, s_y = next(seg_loader)
                s_x, s_y = [t.to(self.device) for t in (s_x,s_y)]
                s_x, s_y=Variable(s_x),Variable(s_y)
                s_y = self.separate_mask(s_y)
                
                # train seg and grading model
                pseudo_masks = self.seg_model(x.float())
                pseudo_masks_cpu = pseudo_masks.detach().cpu().transpose(0,1).contiguous()
                pseudo_maps = []
                for m in pseudo_masks_cpu:
                    pseudo_maps.append(m)
                grading_pred, refined_pseudo_masks = self.grading_model(x.float(), pseudo_maps)
                s_y_pred = self.seg_model(s_x.float())

                grading_loss = self.criterionG(grading_pred, y)
                trn_grading_loss.update(grading_loss.item())
                self.estimatorG.update(grading_pred, y)

                s_y = torch.cat((s_y, refined_pseudo_masks), dim=0)
                s_y_pred = torch.cat((s_y_pred, pseudo_masks), dim=0)
                
                # update seg estimator
                for i in range(0,self.config.num_classes):
                    t=s_y[:,i].float()
                    o=s_y_pred[:,i]
                    # print(f't:{t.shape}')
                    # print(f'o:{o.shape}')
                    self.estimatorS[i-1].update(o, t)

                valid = Variable(torch.FloatTensor(s_y_pred.size(0), 1).fill_(1.0), requires_grad=False).to(self.device)
                fake = Variable(torch.FloatTensor(s_y_pred.size(0), 1).fill_(0.0), requires_grad=False).to(self.device)
                
                seg_loss = self.criterionS(s_y_pred, s_y) - torch.mean(self.discriminator(s_y_pred))
                trn_seg_loss.update(seg_loss.item())
                
                #train discriminator
                dis_loss = -torch.mean(self.discriminator(s_y)) + torch.mean(self.discriminator(s_y_pred))
                trn_dis_loss.update(dis_loss.item())

                loss = grading_loss + seg_loss + dis_loss
                self.optimizerD.zero_grad()
                self.optimizerG.zero_grad()
                self.optimizerS.zero_grad()
                loss.backward()
                self.optimizerS.step()
                self.optimizerG.step()
                self.optimizerD.step()
                
                
                # if self.config.accumulation_step:
                #     loss /= self.config.accumulation_step
                #     loss.backward()
                #
                #     if ((step + 1) % self.config.accumulation_step == 0) or (step + 1 == len(data_loader)):
                #         trn_loss.update(loss.item()*self.config.accumulation_step)
                #         self.optimizer.step()
                #         self.optimizer.zero_grad()
                # else: 
                #     trn_loss.update(loss.item())
                #     self.optimizer.zero_grad()
                #     loss.backward()
                #     self.optimizer.step()
                
                if not self.config.upd_by_ep and scheduler != None: 
                    for s in scheduler:
                        s.step()
                pbar.update(1)
        
        info_dict=dict([('epoch', self.epoch),
                   ('lr', lr),
                   ('trn_seg_loss', round(trn_seg_loss.avg,4)),
                   ('trn_grading_loss', round(trn_grading_loss.avg,4)),
                   ('trn_dis_loss', round(trn_dis_loss.avg,4))])

        for i,k in enumerate(self.config.leisions):
            score=self.estimatorS[i].get_scores(4)
            info_dict.update({'trn_seg_'+_k+'_'+k:_v for _k,_v in score.items()})
        
        grading_score = self.estimatorG.get_scores(4)
        info_dict.update({'trn_grading_'+k:v for k,v in grading_score.items()})
        
        return info_dict
        
    def val_iteration(self, seg_loader, grading_loader, val_loss):
        self.seg_model.eval()
        self.grading_model.eval()
        
        for e in self.estimatorS:
            e.reset()
        self.estimatorG.reset()
        
        with tqdm(total=len(seg_loader), desc=f"Epoch {self.epoch}/{self.config.num_epochs} --- Seg validating", unit='batch') as pbar:
            with torch.no_grad():
                for step, (x, y) in enumerate(seg_loader):
                    x, y = [t.to(self.device) for t in (x,y)]
                    x, y=Variable(x),Variable(y)
                    y_pred = self.seg_model(x.float())
                        
                    #--------normal----------
                    targets=[]
                    for i in range(1,self.config.num_classes+1):
                        t=(y==i).float()
                        targets.append(t)
                        o=y_pred[:,i-1]
                        self.estimatorS[i-1].update(o, t)
                    targets=torch.stack(targets,dim=1)
                    loss = self.criterionS(y_pred, targets)
                    
                    # if self.config.accumulation_step:
                    #     loss /= self.config.accumulation_step
                    #     if ((step + 1) % self.config.accumulation_step == 0) or (step + 1 == len(data_loader)):
                    #         val_loss.update(loss.item()*self.config.accumulation_step)
                    # else: 
                    val_loss.update(loss.item())
                    pbar.update(1) 
                
        info_dict=dict([('epoch', self.epoch),
                   ('val_seg_loss', round(val_loss.avg,4))])

        mean_metrics={}
        for n in set(self.estimatorS[0].need_named_metrics)&set(self.estimatorS[0].get_scores(4).keys()):
            mean_metrics[n]=[]
        
        for i,k in enumerate(self.config.leisions):
            score=self.estimatorS[i].get_scores(4)
    
            for n in set(self.estimatorS[i].need_named_metrics)&set(score.keys()):
                mean_metrics[n].append(score[n])
        
            info_dict.update({'val_seg_'+_k+'_'+k:_v for _k,_v in score.items()})
            
        for k,v in mean_metrics.items():
            info_dict.update({'val_seg_'+k:np.round(np.mean(v), 4)})
            
        val_loss.reset()
        with tqdm(total=len(grading_loader), desc=f"Epoch {self.epoch}/{self.config.num_epochs} --- Grading validating", unit='batch') as pbar:
            with torch.no_grad():
                for step, (x, y) in enumerate(grading_loader):
                    x, y = x.to(self.device), y.to(self.device)
                    x, y=Variable(x),Variable(y)
    
                    y_pred = self.grading_model(x.float())
    
                    loss=self.criterionG(y_pred, y)
                    val_loss.update(loss.item())
    
                    self.estimatorG.update(y_pred, y)
                    pbar.update(1)
                
        score=self.estimatorG.get_scores(4)
        
        info_dict.update(dict([('val_grading_loss', round(val_loss.avg,4))]))
        info_dict.update({'val_grading_'+k:v for k,v in score.items()})
            
        return info_dict

    def loop(self, num_epochs, trn_loader, val_loader):
        logger.configure(dir=self.log_dir, log_suffix=self.id)
        
        if self.config.use_wandb:
            wandb.init(
              project=self.config.experiment_name,
              config=vars(self.config)
            )
            wandb.run.name = self.id
            wandb.run.save()
            
        self.load()
            
        schedulerS=get_scheduler(self.config, self.optimizerS, last_epoch = self.last_epoch)
        schedulerG=lr_scheduler.CosineAnnealingLR(self.optimizerG,T_max=num_epochs,eta_min=self.config.min_lr)
        schedulerD=lr_scheduler.CosineAnnealingLR(self.optimizerD,T_max=num_epochs,eta_min=self.config.min_lr)
        scheduler=[schedulerS,schedulerG,schedulerD]
        
        best_perf = [sys.float_info.min]*2
        grading_best_perf = [sys.float_info.min]*2
        val_loss = AverageMeter()
        last_info=None

        timer = Timer()
        
        for epoch in range(self.start_epoch+1, num_epochs+1):
            self.epoch=epoch
            timer.start_timer(time.time())
            info_dict=self.trn_iteration(trn_loader[0], trn_loader[1], scheduler)
            timer.compute(time.time())
            if self.config.upd_by_ep and scheduler != None: 
                for s in scheduler:
                    s.step()
                    
            if epoch % self.config.save_interval == 0:
                self.save(epoch, self.seg_model.state_dict(), self.optimizerS.state_dict())
                self.save(epoch, self.grading_model.state_dict(), self.optimizerG.state_dict())
            
            if epoch % self.config.validation_interval == 0 or epoch == 1:
                timer.start_timer(time.time())
                info_dict.update(self.val_iteration(val_loader[0], val_loader[1], val_loss))
                timer.compute(time.time())
                
                if info_dict['val_seg_'+self.config.best_metric] > best_perf[1]:
                    best_perf[0]=epoch
                    best_perf[1]=info_dict['val_seg_'+self.config.best_metric]
                    self.best_checkpoint = [os.path.join(self.cpt_dir, self.id+f'_e{epoch}.pt')]
                    if epoch != num_epochs:
                        self.best_checkpoint.append(os.path.join(self.cpt_dir, self.id+f'_e{num_epochs}.pt'))
                    if epoch % self.config.save_interval != 0 and info_dict['val_seg_'+self.config.best_metric] > self.config.good_value:
                        self.save(epoch, self.seg_model.state_dict(), self.optimizerS.state_dict())
                        
                if info_dict['val_grading_'+self.config.gan_grading_best_metric] > grading_best_perf[1]:
                    grading_best_perf[0]=epoch
                    grading_best_perf[1]=info_dict['val_grading_'+self.config.gan_grading_best_metric]
                    self.grading_best_checkpoint = [os.path.join(self.cpt_dir, 'gan_grading_rs50_myffs'+f'_e{epoch}.pt')]
                    if epoch != num_epochs:
                        self.grading_best_checkpoint.append(os.path.join(self.cpt_dir, self.id+f'_e{num_epochs}.pt'))
                    if epoch % self.config.save_interval != 0 and info_dict['val_grading_'+self.config.gan_grading_best_metric] > self.config.gan_grading_good_value:
                        self.save(epoch, self.grading_model.state_dict(), self.optimizerG.state_dict())
                    
            if self.config.use_wandb:
                    wandb.log(info_dict)

            if self.config.show_log:
                logger.logkvs(info_dict)
                logger.dumpkvs()
                str_output = 'epoch & {} \n'.format(self.config.best_metric)
                str_output += f'{best_perf[0]} & {best_perf[1]:.4f} \n'
                grading_str_output = 'epoch & {} \n'.format(self.config.gan_grading_best_metric)
                grading_str_output += f'{grading_best_perf[0]} & {grading_best_perf[1]:.4f} \n'
                logger.log(str_output)
                logger.log(grading_str_output)
            
            if val_loss.is_overfitting() and self.config.early_stop:
                print('Detected overfitting signs, stop training.')
                break
            
            val_loss.reset()
                
        if self.config.use_wandb:
            wandb.finish()
            
        return last_info
        
    def separate_mask(self, masks):
        targets=[]
        for i in range(1,self.config.num_classes+1):
            t=(masks==i).float()
            targets.append(t)
        return torch.stack(targets,dim=1)

    def load(self):
        if self.config.resume_checkpoint:
            if os.path.exists(self.config.resume_checkpoint):
                _, cpt_name=os.path.split(self.config.resume_checkpoint)
                if '.pt' in cpt_name:
                    print(f'loading file at {self.config.resume_checkpoint}')
                    cpt_dict = torch.load(self.config.resume_checkpoint, map_location=self.device)
                else:
                    print('please provide a file path with suffix of ".pt"')
                    return
            else:
                print(f'{self.config.resume_checkpoint} does not exists!')
                return
            
            if cpt_dict:
                # self.start_epoch=cpt_dict['epoch']
                self.seg_model.load_state_dict(cpt_dict['model_state_dict'])
                if self.config.load_optimizer:
                    op_state=cpt_dict.get('optim_state_dict', -1)
                    if op_state !=-1:
                        self.last_epoch=cpt_dict['epoch']-1
                        self.optimizerS.load_state_dict(op_state)
                    else:
                        self.last_epoch=-1
                print('Segmentation Model Checkpoint has been loaded.')
            else:
                print('Segmentation Model Checkpoint has no contents.')
                return

print('gansl.py')

gansl.py


In [27]:
import albumentations.augmentations.geometric as G
import albumentations.augmentations.transforms as A
import albumentations.augmentations.crops as C
import torchvision.transforms as T
import torch
import numpy as np
import random


class OneOf:
    def __init__(self, transforms, choose_one_of=None):
        self.transforms = transforms
        self.choose_one_of = choose_one_of
        transforms_ps = [t.p for t in self.transforms]
        s = sum(transforms_ps)
        self.transforms_ps = [t / s for t in transforms_ps]

    def __call__(self, sample):
        if self.choose_one_of:
            t = self.transforms[self.choose_one_of]
        else:
            t = random.choices(self.transforms, weights=self.transforms_ps)[0]
        return t(sample)

class ToNumpy(object):
    
    def __call__(self, sample):
        image, masks = sample['image'], sample['masks']
        image=image.numpy()
        masks=[m.numpy() for m in masks]
        image=np.transpose(image,(1,2,0))
        new_sample = {'image': image, 'masks': masks}
        return new_sample
    
class Resize(object):
    def __init__(self, size, p=1):
        self.size=size
        self.p=p
    
    def __call__(self, sample):
        if self.size:
            image, masks = sample['image'], sample['masks']
            augmented=G.Resize(height=self.size[0], width=self.size[1])(image=image,masks=masks)
            new_sample = {'image': augmented['image'], 'masks': augmented['masks']}
            return new_sample
        return sample
    
class RandomCrop(object):
    def __init__(self, size, p=1):
        self.size=size
        self.p=p
    
    def __call__(self, sample):
        if self.size:
            image, masks = sample['image'], sample['masks']
            augmented=C.RandomCrop(height=self.size[0], width=self.size[1])(image=image,masks=masks)
            new_sample = {'image': augmented['image'], 'masks': augmented['masks']}
            return new_sample
        return sample

class RandomRotate90(object):
    '''
        Randomly rotates an image
    '''
    def __call__(self, sample):
        image, masks = sample['image'], sample['masks']
        augmented=G.RandomRotate90()(image=image,masks=masks)
        new_sample = {'image': augmented['image'], 'masks': augmented['masks']}
        return new_sample


class HorizontalFlip(object):

    def __call__(self, sample):
        image, masks = sample['image'], sample['masks']
        augmented=G.HorizontalFlip()(image=image,masks=masks)
        new_sample = {'image': augmented['image'], 'masks': augmented['masks']}
        return new_sample

class VerticalFlip(object):

    def __call__(self, sample):
        image, masks = sample['image'], sample['masks']
        augmented=G.VerticalFlip()(image=image,masks=masks)
        new_sample = {'image': augmented['image'], 'masks': augmented['masks']}
        return new_sample
    
class ColorJitter(object):
    
    def __call__(self, sample):
        image, masks = sample['image'], sample['masks']
        augmented=A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=1.0)(image=image)
        new_sample = {'image': augmented['image'], 'masks': masks}
        return new_sample

class CropTo4(object):
    def __init__(self, size):
        self.size=size

    def __call__(self, sample):
        if self.size:
            image, masks = sample['image'], sample['masks']
            augmented_image=self.crop_to_4(image)
            augmented_masks=[]
            for mask in masks:
                augmented_masks.append(self.crop_to_4(mask))
            new_sample = {'image': augmented_image, 'masks': augmented_masks}
            return new_sample
        else:
            return sample
        
    def crop_to_4(self, img):
        y_mid = img.shape[0]//2
        y_end = img.shape[0]
        x_mid = img.shape[1]//2
        x_end = img.shape[1]
        img_lt = img[0:y_mid, 0:x_mid]
        img_lb = img[y_mid:y_end, 0:x_mid]
        img_rt = img[0:y_mid, x_mid:x_end]
        img_rb = img[y_mid:y_end, x_mid:x_end]
        
        img_cat = np.stack([img_lt, img_rt, img_lb, img_rb], axis=0)
        return img_cat
        
class Grid(object):
    def __init__(self, grid_size, p=1):
        self.grid_size=grid_size
        self.p=p
        
    def __call__(self, sample):
        if self.grid_size:
            image, masks = sample['image'], sample['masks']
            augmented_image=self.make_pieces(image)
            augmented_masks=[]
            for mask in masks:
                augmented_masks.append(self.make_pieces(mask))
            new_sample = {'image': augmented_image, 'masks': augmented_masks}
            return new_sample
        else:
            return sample
        
    def make_pieces(self, img):
        y_coord = np.arange(0, img.shape[0], self.grid_size)
        x_coord = np.arange(0, img.shape[1], self.grid_size)
    
        img_pieces=[]
        for y in y_coord:
            for x in x_coord:
                img_pieces.append(img[y:y+self.grid_size, x:x+self.grid_size])
                
        img_cat = np.stack(img_pieces, axis=0)
        return img_cat
        
    
class ToTensor(object):
    
    def __call__(self, sample):
        image, masks = sample['image'], sample['masks']
        if len(image.shape) == 4:
            image=np.transpose(image, (0,3,1,2))
        else:
            image=np.transpose(image,(2,0,1))
        image=torch.from_numpy(image)
        masks=[torch.from_numpy(np.ascontiguousarray(m)) for m in masks]
        new_sample = {'image': image, 'masks': masks}
        return new_sample
    
class Normalize(object):
    def __init__(self):
        self.channel_stats=dict(mean = [0.425753653049469, 0.29737451672554016, 0.21293757855892181],
                         std = [0.27670302987098694, 0.20240527391433716, 0.1686241775751114])
    
    def __call__(self, sample):
        image, masks = sample['image'], sample['masks']
        augmented=T.Normalize(**self.channel_stats)(image)
        new_sample = {'image': augmented, 'masks': masks}
        return new_sample

print('transforms.py')

transforms.py


/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import random

import wandb
from torch.utils.data.dataloader import DataLoader
import traceback

import segmentation_models_pytorch as smp

import logger

# from utils import *
# from metrics import *
# from loss import get_criterion, get_scheduler
# from dataset import IDRiDDataset
# from transform import *
# from model.shufflenetv2_sa import *
# from model.sa_resnet import *
# from model.model import *
# from sl import SLTrainer
# from model.deeplabv3_plus import DeepLab

def train(args):
    if args.load_args:
        args=load_args(args)
    seed_torch(args.seed)
    
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    #数据集
    if args.dataset_name=='idrid':
        args.dataset_path='/kaggle/input/idrid-dataset/A. Segmentation'
        trn_set=IDRiDDataset(args.dataset_path, dataset_type='train', resolution=args.resolution, leisions=args.leisions, args=args)
        val_set=IDRiDDataset(args.dataset_path, dataset_type='val', resolution=args.resolution, leisions=args.leisions, args=args)
    elif args.dataset_name=='vessel':
        args.dataset_path='/kaggle/input/retinal-vessel-segmentation-combined/Retina Vessel Segmentation'
        trn_set=VesselDataset(args.dataset_path, dataset_type='train', resolution=args.resolution, args=args)
        val_set=VesselDataset(args.dataset_path, dataset_type='val', resolution=args.resolution, args=args)
    elif args.dataset_name=='ddr':
        if args.preprocess:
            args.dataset_path='/kaggle/input/cropped-ddr-seg'
        else:
            args.dataset_path='/kaggle/input/ddr-dataset/lesion_segmentation'
        trn_set=DDRDataset(args.dataset_path, dataset_type='train', resolution=args.resolution, args=args)
        val_set=DDRDataset(args.dataset_path, dataset_type='test', resolution=args.resolution, args=args)
    
    if args.use_T:
        trn_set.transform=T.Compose([
                    ToNumpy(),
                    OneOf([
                        Resize(args.crop_size),
                        RandomCrop(args.crop_size),
                        ],
                          choose_one_of=args.choose_one_of),
                    HorizontalFlip(),
                    VerticalFlip(),
                    RandomRotate90(),
                    ColorJitter(),
                    Grid(args.grid_size),
                    ToTensor(),
                    Normalize()
                ])

        val_set.transform=T.Compose([
                    ToNumpy(),
                    Grid(args.grid_size),
                    ToTensor(),
                    Normalize()
        ])
    
    
    print(f'transform:{trn_set.transform}')

    trn_loader=DataLoader(trn_set, batch_size=args.batch_size, shuffle=True, num_workers=4)
    val_loader=DataLoader(val_set, batch_size=args.batch_size, shuffle=True, num_workers=4)

    #损失函数
    criterion = get_criterion(args, device=device)

    #模型
    kwargs={'args':args}
    kwargs['num_classes']=args.num_classes
    kwargs['backbone']=args.backbone
    kwargs['pretrained']=args.pretrained
    kwargs['downsample_factor']=args.downsample_factor
    kwargs['module_list']=args.module_list
    kwargs['resolution']=args.crop_size[0] if args.crop_size else args.resolution[0]
    kwargs['upsample_mode']=args.upsample_mode
    kwargs['dual']=args.dual
    if args.netframe=='deeplabv3':
        model=DeepLab(**kwargs)
    elif args.netframe=='unet':
        if args.backbone=='ds':
            model=UNetDenseNet(**kwargs)
        elif args.backbone[:2]=='rs':
            model=UNetResNet(**kwargs)
        elif args.backbone=='new':
            model=UNetWithResnet50Encoder(n_classes=args.num_classes)
    elif args.netframe=='unetpp':
        model=UnetPP(**kwargs)
    elif args.netframe=='segformer':
        model=Segformer(num_classes=args.num_classes)
    elif args.netframe=='lseg':
        model=ResNetLseg(**kwargs)
    elif args.netframe[:3]=='smp':
        decoder=getattr(smp, args.netframe[4:])
        model = decoder(
            encoder_name=args.backbone,
            classes=args.num_classes,
            encoder_weights='imagenet' if args.pretrained else None
        )
    elif args.netframe=='dwunet':
        model=DWUnet(dwconv='wtconv',**kwargs)
    elif args.netframe=='hiformer':
        model=HiFormer(img_size=args.crop_size[0] if args.crop_size else args.resolution[0],
                       n_classes=args.num_classes)
    elif args.netframe=='transunet':
        model=VisionTransformer(img_size=kwargs['resolution'], 
                                num_classes=args.num_classes, 
                                upsample_mode=args.upsample_mode, 
                                transformer_block=args.transformer_block, 
                                net_type=args.net_type, 
                                decoder=args.transunet_decoder)
    # model=model(**kwargs)
    model.to(device)
    
#     print({name:m for name,m in model.named_children()})
    
    
    #优化器
    if args.optim == 'Adam':
        optimizer=torch.optim.Adam(model.parameters(),lr=args.lr, betas=args.momentum, weight_decay=args.weight_decay)
    elif args.optim == 'SGD':
        optimizer=torch.optim.SGD(model.parameters(),lr=args.lr, momentum=args.momentum, weight_decay=args.weight_decay)
    elif args.optim == 'AdamW':
        optimizer=torch.optim.AdamW(model.parameters(),lr=args.lr, betas=args.momentum, weight_decay=args.weight_decay)
    
    
    # for k,v in args.get_dict().items():
    #     print(f'{k} : {v}')
    print(args.get_dict())
    
    #半监督训练
    if args.ssl=='SL':
        trainer = SLTrainer(model, optimizer, criterion, device, args)
    elif args.ssl=='MTSL':
        trainer = MTSLTrainer(model, optimizer, criterion, device, args)
    if args.ssl=='GANSL':
        if args.gan_grading_dataset=='idrid':
            dataset_path='/kaggle/input/idrid-dataset/B. Disease Grading'
            grading_trn_set=IDataset(dataset_path, dataset_type='train', resolution=args.resolution, args=args)
            grading_val_set=IDataset(dataset_path, dataset_type='val', resolution=args.resolution, args=args)
        
        if args.use_T:
            #使用IMAGENET数据集的均值与标准差进行归一化，有助于网络训练
            channel_stats = dict(mean = [0.425753653049469, 0.29737451672554016, 0.21293757855892181],
                             std = [0.27670302987098694, 0.20240527391433716, 0.1686241775751114])
            grading_trn_set.transform=T.Compose([
    #                     transforms.RandomApply(
    #                         [transforms.RandomCrop(512)],
    #                         p=0.5
    #                     ),
                        T.RandomHorizontalFlip(p=0.5),
                        T.RandomVerticalFlip(p=0.5),
                        T.RandomApply(
                            [T.ColorJitter(
                                brightness=0.2, contrast=0.2,
                                saturation=0, hue=0
                            )],
                            p=0.5
                        ),
                        T.RandomApply(
                            [T.RandomRotation(
                                degrees=[-180,180]
                            )],
                            p=0.5
                        ),
                        T.RandomApply(
                            [T.RandomAffine(
                                degrees=0,
                                translate=[0.2,0.2]
                            )],
                            p=0.5
                        ),
                        T.ToTensor(),
                        T.Normalize(**channel_stats)
                    ])
    
            grading_val_set.transform=T.Compose([
                        T.ToTensor(),
                        T.Normalize(**channel_stats)
            ])
            
            grading_trn_loader=DataLoader(grading_trn_set, batch_size=args.batch_size, shuffle=True, num_workers=4)
            grading_val_loader=DataLoader(grading_val_set, batch_size=args.batch_size, shuffle=False, num_workers=4)
            grading_model = resnet50(pretrained=False, num_classes=args.gan_grading_classes, module_list=args.module_list, weights=args.backbone_weights)
            grading_model.to(device)
            grading_criterion=torch.nn.CrossEntropyLoss()
            optimizerG=torch.optim.SGD(grading_model.parameters(),lr=1e-3, momentum=0.9, weight_decay=0.0005)
            
            trainer = GANTrainer(model, grading_model, [optimizer, optimizerG], [criterion, grading_criterion], device, args)

            trn_loader = [trn_loader, grading_trn_loader]
            val_loader = [val_loader, grading_val_loader]
            
            
    if args.evaluate:
        return trainer.evaluate(val_loader)
    if args.inference:
        trainer.inference(trn_loader, type='trn')
        trainer.inference(val_loader, type='val')
        return
        
    return trainer.loop(args.num_epochs, trn_loader, val_loader)
    
def main():
    args = create_argparser()
    try:
        if args.ablation:
            for selection in args.selection:
                if isinstance(args.ablation, list):
                    assert isinstance(selection, list), 'selection element not list!'
                    args=update_args(args, {a:s for a,s in zip(args.ablation,selection)})
                else:
                    args=update_args(args, {args.ablation:selection})
                train(args)
        else:
            train(args)
    except:
        traceback.print_exc()
    finally:
        if args.use_wandb: wandb.finish()
            
def load_args(args):
    if args.resume_checkpoint:
        if os.path.exists(args.resume_checkpoint):
            _, cpt_name=os.path.split(args.resume_checkpoint)
            if '.pt' in cpt_name:
                print(f'loading args at {args.resume_checkpoint}')
                cpt_dict = torch.load(args.resume_checkpoint, map_location='cpu')
            else:
                return args
        else:
            return args

        if cpt_dict:
            _args = cpt_dict.get('args',-1)
            if _args != -1:
                _args.evaluate=args.evaluate
                _args.inference=args.inference
                _args.inference_root_dir=args.inference_root_dir
                if args.inference:
                    _args.inference_suffix=args.inference_suffix
                    _args.dataset_name=args.dataset_name
                    _args.resolution=args.resolution
                    _args.preprocess=args.preprocess
                    _args.use_T=args.use_T
                _args.num_epochs=args.num_epochs
                _args.skip_epoch=args.skip_epoch
                _args.save_interval=args.save_interval
                _args.validation_interval=args.validation_interval
                _args.good_value=args.good_value
                _args.resume_checkpoint=args.resume_checkpoint
                _args.use_wandb=args.use_wandb
                _args.load_optimizer=args.load_optimizer
                _args.account=args.account
                _args.a_version=args.a_version
                # _args.crop_size=args.crop_size
                _args.choose_one_of=args.choose_one_of
                _args.ablation=args.ablation
                _args.selection=args.selection
                args=adjust_args(_args)
                print('args has been loaded.')
            return args
    return args
            
            
def update_args(args, kwargs):
    args.update(kwargs)
    args=adjust_args(args)
    return args
    
def adjust_args(args):
    if not isinstance(args.resolution, tuple):
        args.resolution=(args.resolution, args.resolution)
    if args.crop_size:
        if not isinstance(args.crop_size, tuple):
            args.crop_size=(args.crop_size, args.crop_size)
    
    if args.ssl == 'MTSL' or args.ssl == 'GANSL':
        args.num_classes=len(args.leisions)

    a=''
    if args.module_list!=None:
        a=list(args.module_list.values())[-1]+'_'

    # if args.ablation:
    #     val = [getattr(args, args.ablation) for a in args.ablation]
    #     val_names = ''
    #     if isinstance(val, dict):
    #         for i,(k,v) in enumerate(val.items()):
    #             val_names+=f'{k}_{v}_'
    #     else:
    #         val_names+=f'{val}_'
    #     args.id=f'{args.ablation}_{val_names}BASE_{args.base_experiment}_{args.account}_{args.a_version}'
    # else:
    args.id=f'{args.dataset_name}_{args.netframe}_{args.backbone}_{a}{args.ssl}_bs{args.batch_size}_lr{str(args.lr).split(".")[-1]}_{args.loss_method}_{args.optim}_{args.account}_{args.a_version}'
    
    return args

def create_argparser():
    # 运行前改
    defaults = dict( 
        seed=random.randint(1,10000),
        account=ACCOUNT,
    #--------------version--------------
        a_version=8,
        a_note=NOTE,
        a_goal='',
        id='',
    #消融实验
        ablation=None,
        selection=[
            {'attention':'fcbam','freq_method':'dct8'},
            {'attention':'cafs','freq_method':'dct8'},
            {'attention':'fsa', 'freq_method':'dct8'}
        ],
    #推理
        inference=False,
        inference_root_dir='/kaggle/working/inference',
        inference_suffix='VES',
    #测试
        evaluate=False,
        enable_test=0,
        test_single=0,
    #数据位置
        dataset_name='ddr',
        dataset_path='',
        preprocess=True,
        one_mask=False,
    #超参数
        resolution=512,#(W,H)
        crop_size=None,
        grid_size=None,
        batch_size=4,  #有bn层，所以bs应该大于1
        lr=2e-4,
        min_lr=1e-5,
        scheduler='multi',
        milestones=[60,120,180,210],
        lr_gamma=0.5,
        step_size=200,
        upd_by_ep=1,
        accumulation_step=4,
    #模型
        leisions=['MA','HE','EX','SE'], # ['MA','HE','EX','SE']
        num_classes=5,
        backbone='rs50',
        netframe='unetpp',
        pretrained=True,
        downsample_factor=32,
        module_list={'attention':'fca', 'freq_method':'dct8'},#不用时置None,使用时必须有attention
        backbone_weights='',
        upsample_mode='interp',
        net_type='linear',
        transunet_decoder='unet',
        transformer_block='vit',
        dual=None,
        dual_loss_factor=0.5,
        seg_vessel=False,
    #损失函数
        loss_method='bce',
        ce_weight=[0.001,1, 0.1, 0.1, 0.1],
        bce_weight=100,
        focal_gamma=2,
    #优化器设置
        optim='Adam',
        momentum=(0.9,0.999), #adam类优化器，也需要betas=(0.9,0.999),第一个类似动量
        weight_decay=0.001,
    #epoch
        num_epochs=240,
    #validation
        metrics=['auprc'],
        best_metric='auprc',
        good_value=0.6,
        validation_interval=1,
        early_stop=0,
    #-----------------------------------
    #--------------use_wandb------------
    #-----------------------------------
        use_wandb=1,
        experiment_name="DR_Segmentation",
    #log
        log_dir='./log',
        log_interval=10,
        show_log=1,
    #保存
        save_interval=40,
        skip_epoch=0,
        checkpoint_dir='./checkpoint',
        save_optim=1,
    #恢复
        resume_checkpoint='',
        load_optimizer=True,
        load_args=False,
    #Transfrom
        use_T=1,
        choose_one_of=None,
    #ssl
        ssl='MTSL',
        input_logits=True,
        task_num=4,
        gan_grading_metrics=['acc','kappa'],
        gan_grading_best_metric='acc',
        gan_grading_good_value=0.8,
        gan_grading_classes=5,
        gan_grading_dataset='idrid'
    )
    args=adjust_args(Aobj(defaults))
    return args
    
    
if __name__ == "__main__":
    main()

train dataset contains 383 images at /kaggle/input/cropped-ddr-seg/trn/image
test dataset contains 225 images at /kaggle/input/cropped-ddr-seg/tst/image
transform:Compose(
)
{'module_list': {'attention': 'cafs', 'freq_method': 'dct8'}, 'weights': '', 'replace_stride_with_dilation': [False, False, False], 'url': 'https://download.pytorch.org/models/resnet50-0676ba61.pth'}
backbone loading Imagenet weights
{'seed': 1222, 'account': 'yuyututa', 'a_version': 7, 'a_note': 'drseg', 'a_goal': '', 'id': 'ddr_unetpp_rs50_dct8_MTSL_bs4_lr0002_bce_Adam_yuyututa_7', 'ablation': None, 'selection': [{'attention': 'fcbam', 'freq_method': 'dct8'}, {'attention': 'cafs', 'freq_method': 'dct8'}, {'attention': 'fsa', 'freq_method': 'dct8'}], 'inference': False, 'inference_root_dir': '/kaggle/working/inference', 'inference_suffix': 'VES', 'evaluate': False, 'enable_test': 0, 'test_single': 0, 'dataset_name': 'ddr', 'dataset_path': '/kaggle/input/cropped-ddr-seg', 'preprocess': True, 'one_mask': False, 'res

Epoch 1/240 --- training lr:0.0002:   3%|▎         | 3/96 [00:08<04:32,  2.93s/batch]
Traceback (most recent call last):
  File "/tmp/ipykernel_23/1387064461.py", line 232, in main
    train(args)
  File "/tmp/ipykernel_23/1387064461.py", line 218, in train
    return trainer.loop(args.num_epochs, trn_loader, val_loader)
  File "/tmp/ipykernel_23/1221795158.py", line 213, in loop
    info_dict=self.trn_iteration(trn_loader, scheduler)
  File "/tmp/ipykernel_23/837679060.py", line 127, in trn_iteration
    self.estimator[i-1].update(o, t)
  File "/tmp/ipykernel_23/1536472188.py", line 71, in update
    targets = targets.detach().cpu().long()
KeyboardInterrupt
